In [4]:
# ========== 完整专业交易策略类定义 ==========
# 版本：实盘交易就绪版 v8.1 + 完整优化参数 + 动态止损 + 多维度选币 + 智能多级止盈止损 + 退市币种屏蔽 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化
# 特点：实盘直接可用 + 完整资金管理 + 实时资产统计 + 智能持仓管理 + 优化参数 + 多级止盈止损 + 退市币种屏蔽 + 重启修复 + 卖出修复 + 真实可交易币种 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化

import pandas as pd
import numpy as np
import time
import threading
from datetime import datetime, timedelta
from binance.client import Client
from binance.enums import *
import os
from dotenv import load_dotenv
import warnings
from collections import deque
import logging
import json
import sys
import random
import signal
import psutil
import gc
from flask import Flask, jsonify, request
import requests
import math
import re

warnings.filterwarnings('ignore')

# ========== 增强日志配置 ==========
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.FileHandler('trading_bot_professional.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# 加载环境变量
load_dotenv()

# ========== 优化后的长期运行配置参数 ==========
class OptimizedLongRunConfig:
    """优化后的长期运行配置参数 - 专业版"""
    
    RISK_CONFIG = {
        "max_portfolio_drawdown": 0.06,
        "max_single_position": 0.20,
        "volatility_adjustment": True,
        "correlation_limit": 0.60,
        "max_single_risk": 0.02,
        "sector_concentration": 0.25,
    }
    
    MARKET_STATE_CONFIG = {
        "bull_confidence_threshold": 0.80,
        "bear_confidence_threshold": 0.75,
        "state_confirmation_count": 3,
        "min_state_duration_hours": 4,
        "trend_threshold": 0.30,
    }
    
    VOLUME_CONFIG = {
        "min_daily_volume": 50000,
        "volume_spike_ratio": 2.0,
        "volume_quality_threshold": 0.75,
        "max_price_increase": 0.15,
    }
    
    POSITION_CONFIG = {
        "bull": {"max_positions": 4, "position_size": 0.20, "total_exposure": 0.80},
        "consolidation": {"max_positions": 3, "position_size": 0.15, "total_exposure": 0.45},
        "bear": {"max_positions": 2, "position_size": 0.15, "total_exposure": 0.30}
    }
    
    STOPLOSS_CONFIG = {
        "base_stop_loss": {"bull": 0.06, "consolidation": 0.04, "bear": 0.03},
        "min_risk_reward": 2.0,
        "trailing_stop_activation": 0.10,
        "trailing_stop_retracement": 0.50,
        "partial_profit_level": 0.25,
        "partial_profit_ratio": 0.25,
    }
    
    # 新增：多级止盈配置
    MULTI_LEVEL_TAKE_PROFIT = [
        {'profit_pct': 0.08, 'sell_ratio': 0.25, 'trail_activation': True},
        {'profit_pct': 0.15, 'sell_ratio': 0.35, 'trail_activation': True},
        {'profit_pct': 0.25, 'sell_ratio': 0.25, 'trail_activation': True},
        {'profit_pct': 0.40, 'sell_ratio': 0.15, 'trail_activation': False}
    ]
    
    # 新增：多级止损配置
    MULTI_LEVEL_STOP_LOSS = [
        {'loss_pct': -0.03, 'sell_ratio': 0.5, 'condition': 'immediate'},
        {'loss_pct': -0.05, 'sell_ratio': 0.3, 'condition': 'volume_spike'},
        {'loss_pct': -0.08, 'sell_ratio': 0.2, 'condition': 'immediate'}
    ]
    
    # 新增：移动止损配置
    TRAILING_STOP_CONFIG = {
        'aggressive': {'activation': 0.10, 'trail_percent': 0.03},
        'moderate': {'activation': 0.15, 'trail_percent': 0.05},
        'conservative': {'activation': 0.20, 'trail_percent': 0.08}
    }
    
    # 新增：全仓风险控制配置
    PORTFOLIO_RISK_LEVELS = [
        {'drawdown_pct': 0.03, 'action': 'reduce_risk', 'reduce_by': 0.3},
        {'drawdown_pct': 0.05, 'action': 'reduce_risk', 'reduce_by': 0.5},
        {'drawdown_pct': 0.08, 'action': 'stop_trading', 'duration_hours': 24}
    ]
    
    LONG_RUN_CONFIG = {
        "main_cycle_interval": 1800,
        "realtime_interval": 600,
        "min_investment": 5,
        "max_holding_days": 21,
        "min_holding_hours": 8,
        "memory_cleanup_interval": 3600,
        "state_save_interval": 300,
        "health_check_interval": 600,
        "max_continuous_errors": 3,
        "error_retry_delay": 30,
    }
    # 🆕 新增：主动优化配置
    PORTFOLIO_OPTIMIZATION_CONFIG = {
        "replacement_threshold": 20, # 替换评分阈值
        "min_holding_hours": 12, # 最小持有时间（小时）
        "max_replacements_per_cycle": 2, # 每周期最大替换数量
        "profit_protection_threshold": 0.05, # 盈利保护阈值（5%）
        "optimize_interval_bull": 3600, # 牛市优化间隔（秒）
        "optimize_interval_bear": 3600, # 熊市优化间隔
        "optimize_interval_consolidation": 7200 # 震荡市优化间隔
    }
    
    # 🆕 新增：动态间隔配置
    ADAPTIVE_INTERVAL_CONFIG = {
        "interval_update_frequency": 21600, # 间隔更新频率（6小时）
        "bull_high_vol_main": 900, # 高波动牛市主策略间隔
        "bull_low_vol_main": 1800, # 低波动牛市主策略间隔
        "bear_main": 1200, # 熊市主策略间隔
        "consolidation_main": 1800 # 震荡市主策略间隔
    }

# ========== 增强程序重启恢复管理器 ==========
class EnhancedRestartManager:
    """增强的程序重启恢复管理器 - 解决重启后状态不一致问题"""
    
    def __init__(self, bot):
        self.bot = bot
        self.recovery_attempts = 0
        self.max_recovery_attempts = 3
    
    def safe_restart(self):
        """安全重启程序"""
        print("🔄 执行安全重启恢复...")
        
        recovery_steps = [
            self._step1_validate_exchange_connection,
            self._step2_sync_actual_positions,
            self._step3_rebuild_position_manager,
            self._step4_validate_funds,
            self._step5_reset_error_counters,
            self._step6_verify_recovery
        ]
        
        for i, step in enumerate(recovery_steps, 1):
            print(f"🔧 重启恢复步骤 {i}/{len(recovery_steps)}...")
            success = step()
            if not success:
                print(f"❌ 重启恢复步骤 {i} 失败")
                if i <= 3: # 前3步失败则重试
                    return self._retry_recovery()
                else:
                    return self._emergency_recovery()
        
        print("✅ 安全重启恢复完成!")
        return True
    
    def _step1_validate_exchange_connection(self):
        """验证交易所连接"""
        try:
            self.bot.data_collector.client.ping()
            print("✅ 交易所连接正常")
            return True
        except Exception as e:
            print(f"❌ 交易所连接失败: {e}")
            # 尝试重新连接
            return self.bot.data_collector.connect_exchange()
    
    def _step2_sync_actual_positions(self):
        """同步实际持仓"""
        try:
            print("🔄 同步实际持仓...")
            
            # 获取交易所实际持仓
            actual_assets = self._get_exchange_assets()
            if not actual_assets:
                print("❌ 无法获取交易所资产")
                return False
            
            # 清理程序持仓记录
            self.bot.position_manager.positions = {}
            
            # 重新建立持仓记录
            recovered_count = 0
            for asset, quantity in actual_assets.items():
                symbol, price = self._find_asset_trading_pair(asset, quantity)
                if symbol and price:
                    # 🆕 增强：尝试从持久化数据获取实际成本价
                    actual_cost_price = self.bot.persistence_manager._get_actual_cost_price_from_history(asset)
                    if actual_cost_price and actual_cost_price > 0:
                        entry_price = actual_cost_price
                        cost_known = True
                    else:
                        entry_price = price
                        cost_known = False
                        print(f"⚠️ 无法获取 {symbol} 的实际成本价，使用当前价格估算")
                    
                    self.bot.position_manager.positions[symbol] = {
                        'symbol': symbol,
                        'quantity': quantity,
                        'entry_price': entry_price,
                        'total_invested': quantity * entry_price,
                        'entry_time': datetime.now(),
                        'restart_recovered': True,
                        'cost_known': cost_known, # 🆕 标记成本价是否准确
                        'original_entry_time': datetime.now(), # 🆕 保存原始时间
                        'original_entry_price': entry_price # 🆕 保存原始价格
                    }
                    self.bot.holding_manager.record_buy(symbol)
                    recovered_count += 1
                    print(f"✅ 恢复持仓: {symbol}, 数量: {quantity:.6f}, 成本价: {'已知' if cost_known else '估算'}")
            
            print(f"📊 持仓同步完成: {recovered_count}个持仓")
            return recovered_count > 0
            
        except Exception as e:
            print(f"❌ 持仓同步失败: {e}")
            return False
    
    def _step3_rebuild_position_manager(self):
        """重建持仓管理器"""
        try:
            # 重新计算持仓价值
            total_position_value = 0
            valid_positions = {}
            
            for symbol, position in self.bot.position_manager.positions.items():
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price and current_price > 0:
                    position_value = position['quantity'] * current_price
                    total_position_value += position_value
                    valid_positions[symbol] = position
                else:
                    print(f"⚠️ 移除无效持仓: {symbol}")
            
            self.bot.position_manager.positions = valid_positions
            self.bot.live_capital_manager.allocated_capital = total_position_value
            
            print(f"💰 持仓重建: {len(valid_positions)}个有效持仓, 总价值${total_position_value:.2f}")
            return True
            
        except Exception as e:
            print(f"❌ 持仓重建失败: {e}")
            return False
    
    def _step4_validate_funds(self):
        """验证资金"""
        try:
            if self.bot.mode_selector.mode == 'live':
                success = self.bot.live_capital_manager._update_live_balance_correct()
                if not success:
                    print("⚠️ 资金验证失败，使用缓存值继续运行")
            return True
        except Exception as e:
            print(f"⚠️ 资金验证异常: {e}")
            return True # 资金验证失败不阻止程序运行
    
    def _step5_reset_error_counters(self):
        """重置错误计数器"""
        self.bot.continuous_errors = 0
        if hasattr(self.bot, 'trading_executor'):
            self.bot.trading_executor.failed_symbols = {}
        print("✅ 错误计数器重置")
        return True
    
    def _step6_verify_recovery(self):
        """验证恢复结果"""
        try:
            # 检查持仓一致性
            actual_assets = self._get_exchange_assets() or {}
            program_positions = len(self.bot.position_manager.positions)
            
            print(f"🔍 恢复验证: 实际资产{len(actual_assets)}个 vs 程序持仓{program_positions}个")
            
            if abs(len(actual_assets) - program_positions) <= 2: # 允许少量差异
                print("✅ 恢复验证通过")
                return True
            else:
                print(f"⚠️ 恢复验证警告: 数量不匹配")
                return True # 不阻止运行
                
        except Exception as e:
            print(f"⚠️ 恢复验证异常: {e}")
            return True
    
    def _get_exchange_assets(self):
        """获取交易所资产"""
        try:
            @self.bot.data_collector.api_retry_manager.retry_api_call
            def get_account_safe():
                return self.bot.data_collector.client.get_account()
            
            account_info = get_account_safe()
            if not account_info:
                return None
            
            assets = {}
            stable_coins = ['USDT', 'BUSD', 'USDC', 'TUSD', 'USDP', 'DAI']
            
            for balance in account_info['balances']:
                asset = balance['asset']
                free = float(balance['free'])
                locked = float(balance['locked'])
                total = free + locked
                
                if total > 0.000001 and not asset.startswith('LD') and asset not in stable_coins:
                    assets[asset] = total
            
            return assets
            
        except Exception as e:
            print(f"❌ 获取交易所资产失败: {e}")
            return None
    
    def _find_asset_trading_pair(self, asset, quantity):
        """为资产寻找交易对"""
        trading_pairs = [f"{asset}USDT", f"{asset}BUSD"]
        
        for symbol in trading_pairs:
            try:
                price = self.bot.data_collector.get_price(symbol)
                if price and price > 0:
                    # 验证交易对状态
                    symbol_info = self.bot.data_collector.client.get_symbol_info(symbol)
                    if symbol_info and symbol_info.get('status') == 'TRADING':
                        return symbol, price
            except:
                continue
        
        return None, None
    
    def _retry_recovery(self):
        """重试恢复"""
        self.recovery_attempts += 1
        if self.recovery_attempts <= self.max_recovery_attempts:
            print(f"🔄 第{self.recovery_attempts}次重试恢复...")
            time.sleep(5)
            return self.safe_restart()
        else:
            print("🚨 达到最大恢复重试次数")
            return self._emergency_recovery()
    
    def _emergency_recovery(self):
        """紧急恢复"""
        print("🆘 执行紧急恢复...")
        
        # 清空所有状态，从零开始
        self.bot.position_manager.positions = {}
        self.bot.holding_manager.buy_times = {}
        self.bot.continuous_errors = 0
        
        # 强制重新初始化资金
        if self.bot.mode_selector.mode == 'live':
            self.bot.live_capital_manager._update_live_balance_correct()
        
        print("✅ 紧急恢复完成 - 程序将从干净状态开始运行")
        return True

# ========== 修复的卖出管理器 ==========
class FixedSellManager:
    """修复的卖出管理器 - 解决止盈止损卖出报错问题"""
    
    def __init__(self, bot):
        self.bot = bot
        self.sell_retry_count = 3
        self.last_sell_attempt = {}
    
    def safe_sell_position(self, symbol, quantity, reason="手动卖出"):
        """安全卖出持仓"""
        try:
            print(f"🔄 尝试卖出 {symbol} - 原因: {reason}")
            
            # 检查持仓是否存在
            if symbol not in self.bot.position_manager.positions:
                print(f"❌ 卖出失败: 未找到持仓 {symbol}")
                return False
            
            position = self.bot.position_manager.positions[symbol]
            actual_quantity = position['quantity']
            
            # 确保卖出数量不超过实际持仓
            sell_quantity = min(quantity, actual_quantity)
            if sell_quantity <= 0:
                print(f"❌ 卖出失败: 无效数量 {sell_quantity}")
                return False
            
            # 获取当前价格
            current_price = self.bot.data_collector.get_price(symbol)
            if not current_price or current_price <= 0:
                print(f"❌ 卖出失败: 无法获取 {symbol} 价格")
                return False
            
            # 精度调整 - 使用增强的精度处理
            adjusted_quantity, adjusted_investment = self.bot.order_manager._enhanced_adjust_quantity(
                symbol, sell_quantity, current_price, 'SELL'
            )
            if adjusted_quantity <= 0:
                print(f"❌ 卖出失败: 精度调整后数量为0")
                return False
            
            # 检查最小卖出金额
            sell_amount = adjusted_quantity * current_price
            if sell_amount < self.bot.order_manager.min_investment:
                print(f"❌ 卖出失败: 卖出金额过小 ${sell_amount:.2f}")
                return False
            
            # 执行卖出
            for attempt in range(self.sell_retry_count):
                print(f"💸 卖出尝试 {attempt+1}/{self.sell_retry_count}: {symbol}, 数量: {adjusted_quantity:.6f}")
                
                sell_order = self.bot.order_manager.create_sell_order(
                    symbol, adjusted_quantity, current_price,
                    self.bot.position_manager, self.bot.live_capital_manager
                )
                
                if sell_order:
                    print(f"✅ 卖出成功: {symbol}, 数量: {adjusted_quantity:.6f}")
                    
                    # 更新持仓时间记录
                    if adjusted_quantity >= actual_quantity * 0.99: # 几乎全部卖出
                        self.bot.holding_manager.record_sell(symbol)
                    
                    # 🆕 立即保存持仓数据
                    self.bot.persistence_manager.save_position_details()
                    
                    # 立即更新资金显示
                    self.bot.account_detail_manager.print_immediate_funds_snapshot()
                    return True
                else:
                    print(f"❌ 卖出尝试 {attempt+1} 失败")
                    if attempt < self.sell_retry_count - 1:
                        time.sleep(2) # 等待后重试
            
            print(f"🚨 所有卖出尝试都失败: {symbol}")
            return False
            
        except Exception as e:
            print(f"❌ 卖出异常 {symbol}: {e}")
            return False
    
    def execute_stop_loss(self, symbol, current_price, reason):
        """执行止损"""
        if symbol not in self.bot.position_manager.positions:
            return False
        
        position = self.bot.position_manager.positions[symbol]
        quantity = position['quantity']
        
        print(f"🛑 执行止损: {symbol}, 价格: ${current_price:.6f}, 原因: {reason}")
        return self.safe_sell_position(symbol, quantity, f"止损: {reason}")
    
    def execute_take_profit(self, symbol, current_price, quantity, reason):
        """执行止盈"""
        print(f"🎯 执行止盈: {symbol}, 数量: {quantity:.6f}, 价格: ${current_price:.6f}, 原因: {reason}")
        return self.safe_sell_position(symbol, quantity, f"止盈: {reason}")
    
    def execute_trailing_stop(self, symbol, current_price, reason):
        """执行移动止损"""
        if symbol not in self.bot.position_manager.positions:
            return False
        
        position = self.bot.position_manager.positions[symbol]
        quantity = position['quantity']
        
        print(f"📉 执行移动止损: {symbol}, 价格: ${current_price:.6f}, 原因: {reason}")
        return self.safe_sell_position(symbol, quantity, f"移动止损: {reason}")

# ========== 修复的风险管理器 ==========
class FixedRiskManager:
    """修复的风险管理器 - 集成卖出修复"""
    
    def __init__(self, data_collector):
        self.data_collector = data_collector
        self.take_profit_system = SmartMultiLevelTakeProfit()
        self.stop_loss_system = SmartMultiLevelStopLoss()
        self.trailing_stop_manager = SmartTrailingStopManager()
        self.volatility_cache = {}
        self.sell_manager = None
        self.peak_prices = {}
        
    def set_sell_manager(self, sell_manager):
        """设置卖出管理器"""
        self.sell_manager = sell_manager
    
    def check_enhanced_risk(self, symbol, position, market_state, holding_hours, current_price):
        """修复的风险检查"""
        try:
            if not self.sell_manager:
                print("❌ 风险检查失败: 未设置卖出管理器")
                return {'action': 'HOLD', 'reason': '系统配置错误'}
            
            entry_price = position['entry_price']
            current_pnl = (current_price - entry_price) / entry_price
            
            # 1. 检查止损条件
            stop_loss_action = self._check_stop_loss_conditions(
                symbol, position, current_price, current_pnl, market_state
            )
            if stop_loss_action:
                return stop_loss_action
            
            # 2. 检查止盈条件  
            take_profit_action = self._check_take_profit_conditions(
                symbol, position, current_price, current_pnl, holding_hours
            )
            if take_profit_action:
                return take_profit_action
            
            # 3. 检查移动止损
            trailing_stop_action = self._check_trailing_stop_conditions(
                symbol, position, current_price, current_pnl
            )
            if trailing_stop_action:
                return trailing_stop_action
            
            return {
                'action': 'HOLD', 
                'reason': f'继续持有: 盈亏{current_pnl:+.2%}'
            }
            
        except Exception as e:
            print(f"❌ 风险检查异常 {symbol}: {e}")
            return {'action': 'HOLD', 'reason': f'检查异常: {e}'}
    
    def _check_stop_loss_conditions(self, symbol, position, current_price, current_pnl, market_state):
        """检查止损条件"""
        try:
            # 动态止损阈值
            stop_loss_threshold = self._get_stop_loss_threshold(market_state)
            
            if current_pnl <= stop_loss_threshold:
                return {
                    'action': 'STOP_LOSS',
                    'symbol': symbol,
                    'price': current_price,
                    'quantity': position['quantity'],
                    'reason': f'止损触发: 亏损{abs(current_pnl):.2%} ≥ {abs(stop_loss_threshold):.2%}'
                }
            
            return None
        except Exception as e:
            print(f"❌ 止损检查异常 {symbol}: {e}")
            return None
    
    def _check_take_profit_conditions(self, symbol, position, current_price, current_pnl, holding_hours):
        """检查止盈条件"""
        try:
            # 多级止盈
            take_profit_levels = [
                {'profit': 0.08, 'ratio': 0.25, 'min_hours': 2},
                {'profit': 0.15, 'ratio': 0.35, 'min_hours': 4},
                {'profit': 0.25, 'ratio': 0.40, 'min_hours': 8}
            ]
            
            for level in take_profit_levels:
                if (current_pnl >= level['profit'] and 
                    holding_hours >= level['min_hours']):
                    
                    sell_quantity = position['quantity'] * level['ratio']
                    return {
                        'action': 'TAKE_PROFIT',
                        'symbol': symbol,
                        'price': current_price,
                        'quantity': sell_quantity,
                        'reason': f'止盈触发: 盈利{current_pnl:.2%} ≥ {level["profit"]:.0%}'
                    }
            
            return None
        except Exception as e:
            print(f"❌ 止盈检查异常 {symbol}: {e}")
            return None
    
    def _check_trailing_stop_conditions(self, symbol, position, current_price, current_pnl):
        """检查移动止损条件"""
        try:
            if current_pnl >= 0.10: # 盈利10%后启动移动止损
                # 简单移动止损：从最高点回撤5%就卖出
                if symbol in self.peak_prices:
                    peak_price = self.peak_prices[symbol]
                    drawdown = (peak_price - current_price) / peak_price
                    
                    if drawdown >= 0.05: # 从高点回撤5%
                        return {
                            'action': 'TRAILING_STOP',
                            'symbol': symbol,
                            'price': current_price,
                            'quantity': position['quantity'],
                            'reason': f'移动止损: 从高点回撤{drawdown:.2%}'
                        }
                
                # 更新最高价
                if symbol not in self.peak_prices or current_price > self.peak_prices[symbol]:
                    self.peak_prices[symbol] = current_price
            
            return None
        except Exception as e:
            print(f"❌ 移动止损检查异常 {symbol}: {e}")
            return None
    
    def _get_stop_loss_threshold(self, market_state):
        """获取动态止损阈值"""
        thresholds = {
            'bull': -0.06, # 牛市容忍更大回撤
            'consolidation': -0.04, # 震荡市正常止损
            'bear': -0.03 # 熊市快速止损
        }
        return thresholds.get(market_state, -0.04)
    
    def _calculate_volatility(self, symbol):
        """计算波动率"""
        try:
            if symbol in self.volatility_cache:
                cached_data = self.volatility_cache[symbol]
                if time.time() - cached_data['timestamp'] < 3600: # 缓存1小时
                    return cached_data['volatility']
            
            data = self.data_collector.get_klines(symbol, '1d', 20)
            if data is None or len(data) < 20:
                return 0.05 # 默认波动率
            
            returns = data['close'].pct_change().dropna()
            volatility = returns.std()
            
            # 缓存结果
            self.volatility_cache[symbol] = {
                'volatility': volatility,
                'timestamp': time.time()
            }
            
            return volatility
            
        except Exception as e:
            print(f"❌ 计算波动率失败 {symbol}: {e}")
            return 0.05
    
    def _get_volume_data(self, symbol):
        """获取成交量数据"""
        try:
            data = self.data_collector.get_klines(symbol, '1h', 24)
            if data is None or len(data) < 24:
                return {'current_volume': 0, 'avg_volume': 1}
            
            current_volume = data['volume'].iloc[-1]
            avg_volume = data['volume'].mean()
            
            return {
                'current_volume': current_volume,
                'avg_volume': avg_volume
            }
            
        except Exception as e:
            print(f"❌ 获取成交量数据失败 {symbol}: {e}")
            return {'current_volume': 0, 'avg_volume': 1}

# ========== 实盘模式选择器 ==========
class TradingModeSelector:
    """实盘模式选择器"""
    
    def __init__(self):
        self.mode = None
        self.client = None
        
    def select_mode(self):
        """选择交易模式"""
        print("🎯 请选择交易模式:")
        print("1. 测试网模式 (Testnet)")
        print("2. 实盘模式 (Live Trading)")
        
        while True:
            try:
                choice = input("请输入选择 (1 或 2): ").strip()
                if choice == '1':
                    self.mode = 'testnet'
                    print("🔬 已选择测试网模式")
                    break
                elif choice == '2':
                    self.mode = 'live'
                    print("🚀 已选择实盘模式 - 请注意风险!")
                    break
                else:
                    print("❌ 无效选择，请输入 1 或 2")
            except Exception as e:
                print(f"❌ 输入错误: {e}")
        
        return self.mode
    
    def create_client(self):
        """创建交易所客户端"""
        try:
            api_key = os.getenv('BINANCE_API_KEY')
            api_secret = os.getenv('BINANCE_SECRET_KEY')
            
            if not api_key or not api_secret:
                print("❌ 未找到API密钥，请检查.env文件")
                return None
            
            # 添加请求配置
            request_params = {
                'timeout': 10,
                'proxies': None
            }
            
            if self.mode == 'testnet':
                self.client = Client(
                    api_key=api_key,
                    api_secret=api_secret,
                    testnet=True,
                    requests_params=request_params
                )
                print("✅ 测试网客户端创建成功")
            else:
                self.client = Client(
                    api_key=api_key,
                    api_secret=api_secret,
                    requests_params=request_params
                )
                print("✅ 实盘客户端创建成功")
            
            # 测试连接
            self.client.ping()
            print("✅ 交易所连接测试成功")
            
            return self.client
            
        except Exception as e:
            print(f"❌ 创建客户端失败: {e}")
            return None

# ========== 增强API重试管理器 ==========
class EnhancedAPIRetryManager:
    """增强API重试管理器"""
    
    def __init__(self):
        self.retry_config = {
            'max_retries': 5,
            'base_delay': 2.0,
            'max_delay': 60.0,
            'backoff_factor': 2.0,
            'retryable_errors': [
                'ConnectionError', 'Timeout', 'APIError', 
                'BinanceAPIException', 'requests.exceptions.ConnectionError',
                'requests.exceptions.Timeout', 'RemoteDisconnected',
                'Read timed out', 'ConnectTimeoutError', 'HTTPSConnectionPool'
            ],
            'non_retryable_errors': [
                'insufficient balance',
                'Account has insufficient balance',
                'MIN_NOTIONAL'
            ]
        }
        self.circuit_breaker = APICircuitBreaker()
        self.stats = {
            'total_calls': 0,
            'successful_calls': 0,
            'failed_calls': 0,
            'retry_attempts': 0
        }
    
    def retry_api_call(self, func):
        """API重试装饰器"""
        def wrapper(*args, **kwargs):
            self.stats['total_calls'] += 1
            last_exception = None
            
            if self.circuit_breaker.is_open():
                print("🚨 API熔断器已打开，暂停请求60秒")
                time.sleep(60)
                if self.circuit_breaker.is_open():
                    raise Exception("API熔断器仍处于打开状态")
            
            for attempt in range(self.retry_config['max_retries'] + 1):
                try:
                    result = func(*args, **kwargs)
                    self.circuit_breaker.record_success()
                    self.stats['successful_calls'] += 1
                    return result
                    
                except Exception as e:
                    last_exception = e
                    error_msg = str(e).lower()
                    
                    for non_retry_error in self.retry_config['non_retryable_errors']:
                        if non_retry_error.lower() in error_msg:
                            print(f"❌ 不可重试错误: {error_msg}")
                            self.stats['failed_calls'] += 1
                            self.circuit_breaker.record_failure()
                            raise e
                    
                    error_type = type(e).__name__
                    
                    if error_type not in self.retry_config['retryable_errors'] and attempt == 0:
                        print(f"❌ 不可重试错误: {error_type} - {e}")
                        self.stats['failed_calls'] += 1
                        self.circuit_breaker.record_failure()
                        raise e
                    
                    if attempt == self.retry_config['max_retries']:
                        self.stats['failed_calls'] += 1
                        self.circuit_breaker.record_failure()
                        print(f"❌ API调用失败，已达最大重试次数: {error_type} - {e}")
                        raise last_exception
                    
                    self.stats['retry_attempts'] += 1
                    delay = self._calculate_backoff_delay(attempt)
                    print(f"⚠️ API调用失败 (尝试 {attempt + 1}/{self.retry_config['max_retries']}): {error_type}")
                    print(f"⏳ {delay:.1f}秒后重试...")
                    time.sleep(delay)
            
            raise last_exception
        return wrapper
    
    def _calculate_backoff_delay(self, attempt):
        """计算指数退避延迟"""
        delay = self.retry_config['base_delay'] * (self.retry_config['backoff_factor'] ** attempt)
        jitter = random.uniform(0.8, 1.2)
        delay = delay * jitter
        return min(delay, self.retry_config['max_delay'])
    
    def get_stats(self):
        """获取统计信息"""
        success_rate = (self.stats['successful_calls'] / self.stats['total_calls'] * 100) if self.stats['total_calls'] > 0 else 0
        return {
            'total_calls': self.stats['total_calls'],
            'successful_calls': self.stats['successful_calls'],
            'failed_calls': self.stats['failed_calls'],
            'retry_attempts': self.stats['retry_attempts'],
            'success_rate': f"{success_rate:.1f}%",
            'circuit_breaker_state': self.circuit_breaker.state
        }

class APICircuitBreaker:
    """API熔断器"""
    
    def __init__(self, failure_threshold=5, recovery_timeout=60):
        self.failure_count = 0
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.last_failure_time = None
        self.state = "CLOSED"
    
    def record_success(self):
        """记录成功"""
        self.failure_count = 0
        self.state = "CLOSED"
        self.last_failure_time = None
    
    def record_failure(self):
        """记录失败"""
        self.failure_count += 1
        self.last_failure_time = datetime.now()
        
        if self.failure_count >= self.failure_threshold:
            self.state = "OPEN"
            print(f"🚨 API熔断器触发! 失败次数: {self.failure_count}")
    
    def is_open(self):
        """检查是否打开"""
        if self.state == "OPEN":
            if self.last_failure_time:
                time_since_failure = (datetime.now() - self.last_failure_time).total_seconds()
                if time_since_failure > self.recovery_timeout:
                    self.state = "HALF_OPEN"
                    print("🔄 熔断器进入半开状态，尝试恢复...")
                    return False
            return True
        return False

# ========== 数据收集器 ==========
class LongRunDataCollector:
    """长期运行数据收集器"""
    
    def __init__(self, mode_selector):
        self.mode_selector = mode_selector
        self.api_retry_manager = EnhancedAPIRetryManager()
        self.client = None
        self.price_cache = {}
        self.volume_cache = {}
        self.last_update = {}
        self.cache_duration = 120
        self.network_health = "HEALTHY"
        self.last_health_check = datetime.now()
        self.consecutive_failures = 0
    
    def connect_exchange(self):
        """连接交易所"""
        try:
            self.client = self.mode_selector.create_client()
            if self.client:
                print("✅ 交易所连接成功")
                return True
            else:
                print("❌ 交易所连接失败")
                return False
        except Exception as e:
            print(f"❌ 交易所连接失败: {e}")
            self.client = None
            return False
    
    @EnhancedAPIRetryManager().retry_api_call
    def get_price(self, symbol):
        """获取价格"""
        try:
            # 检查网络状态
            network_status = self._check_network_health()
            if network_status == "UNHEALTHY":
                cached_price = self.price_cache.get(symbol)
                if cached_price:
                    print(f"🌐 网络不稳定，使用缓存价格: {symbol} = ${cached_price:.6f}")
                    return cached_price
                else:
                    print(f"🌐 网络不稳定且无缓存: {symbol}")
                    return None
            
            if symbol in self.price_cache and time.time() - self.last_update.get(symbol, 0) < self.cache_duration:
                return self.price_cache[symbol]
            
            if self.client is None:
                if not self.connect_exchange():
                    return None
            
            ticker = self.client.get_symbol_ticker(symbol=symbol)
            price = float(ticker['price'])
            
            self.price_cache[symbol] = price
            self.last_update[symbol] = time.time()
            
            return price
            
        except Exception as e:
            print(f"❌ 获取价格失败 {symbol}: {e}")
            return self.price_cache.get(symbol)
    
    @EnhancedAPIRetryManager().retry_api_call
    def get_klines(self, symbol, interval, limit=100):
        """获取K线数据"""
        try:
            if self.client is None:
                if not self.connect_exchange():
                    return None
            
            klines = self.client.get_klines(symbol=symbol, interval=interval, limit=limit)
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
            
            return df
            
        except Exception as e:
            print(f"❌ 获取K线数据失败 {symbol}: {e}")
            return None
    
    def _check_network_health(self):
        """检查网络健康状态"""
        try:
            current_time = datetime.now()
            if (current_time - self.last_health_check).total_seconds() < 300:
                return self.network_health
            
            # 简单ping测试
            self.client.ping()
            self.consecutive_failures = 0
            self.network_health = "HEALTHY"
            self.last_health_check = current_time
            return "HEALTHY"
            
        except Exception as e:
            self.consecutive_failures += 1
            self.last_health_check = datetime.now()
            
            if self.consecutive_failures >= 3:
                self.network_health = "UNHEALTHY"
                print(f"🌐 网络连接异常: 连续{self.consecutive_failures}次失败")
            else:
                self.network_health = "DEGRADED"
            
            return self.network_health
    
    def get_api_health_report(self):
        """获取API健康报告"""
        return self.api_retry_manager.get_stats()

# ========== 修复的动态币种列表管理器 - 只交易USDT交易对 ==========
class DynamicCoinListManager:
    """修复的动态币种列表管理器 - 只交易USDT交易对，正确过滤稳定币"""
    
    def __init__(self, data_collector):
        self.data_collector = data_collector
        self.coin_universe = set()
        self.coin_scores = {}
        self.last_update = None
        self.update_interval = 3600 # 1小时更新一次
        self.min_coins = 50
        self.max_coins = 150
        
        # 严格的限制列表
        self.restricted_coins = [
            'BNBUSD', 'BTCUSD', 'ETHUSD', 'ADAUSD', 'DOTUSD', 'LINKUSD',
            'LTCUSD', 'BCHUSD', 'XLMUSD', 'XRPUSD', 'EOSUSD', 'TRXUSD',
            'ETCUSD', 'XTZUSD', 'ATOMUSD', 'ALGOUSD', 'ZECUSD', 'BATUSD',
            'ZRXUSD', 'ONTUSD', 'IOTAUSD', 'VETUSD', 'THETAUSD', 'MATICUSD'
        ]
        
        # 已知的稳定币和不可交易币种 - 增强稳定币列表
        self.stable_coins = [
            'USDT', 'BUSD', 'USDC', 'TUSD', 'USDP', 'DAI', 'FDUSD', 'UST', 'USDD',
            'PAX', 'HUSD', 'SUSD', 'GUSD', 'EUR', 'GBP', 'AUD', 'CAD', 'JPY', 'KRW'
        ]
        
        # 稳定币交易对黑名单 - 只过滤明确的稳定币交易对
        self.stable_coin_blacklist = [
            'USDCUSDT', 'BUSDUSDT', 'TUSDUSDT', 'USDPUSDT', 'DAIUSDT', 'FDUSDUSDT',
            'USTUSDT', 'USDDUSDT', 'PAXUSDT', 'HUSDUSDT', 'SUSDUSDT', 'GUSDUSDT',
            'EURUSDT', 'GBPUSDT', 'AUDUSDT', 'CADUSDT', 'JPYUSDT', 'KRWUSDT'
        ]
        
        self.delisted_coins = ['AUTO', 'NEBL', 'SRM', 'MOB', 'REP', 'BTCDOWN', 'ETHUP', 'BEAR', 'BULL']
        
    def initialize_coin_list(self):
        """初始化币种列表 - 只获取USDT交易对，正确过滤稳定币"""
        print("🔄 初始化动态币种列表 - 从币安获取USDT交易对，正确过滤稳定币...")
        
        try:
            # 方法1: 从币安获取USDT交易对
            usdt_coins = self._get_binance_usdt_coins(200)
            if usdt_coins:
                self.coin_universe = set(usdt_coins)
                print(f"✅ 成功从币安获取 {len(usdt_coins)} 个USDT交易对 (已过滤稳定币)")
            else:
                # 方法2: 如果失败，使用经过验证的主流USDT币种
                self._initialize_verified_usdt_coins()
                print(f"✅ 使用验证过的主流USDT币种列表: {len(self.coin_universe)} 个币种 (已过滤稳定币)")
            
            # 严格验证所有币种
            self._strict_validate_all_coins()
            
            print(f"🎯 最终有效USDT交易对: {len(self.coin_universe)} 个 (已过滤稳定币)")
            return list(self.coin_universe)
            
        except Exception as e:
            print(f"❌ 初始化币种列表失败: {e}")
            return self._get_fallback_coin_list()
    
    def _get_binance_usdt_coins(self, limit=200):
        """从币安获取USDT交易对 - 修复稳定币过滤"""
        try:
            print("📊 从币安获取交易量排名前{}的USDT交易对...".format(limit))
            
            # 获取24小时行情
            tickers = self.data_collector.client.get_ticker()
            
            # 🛠️ 关键修复：只获取USDT交易对，正确过滤稳定币
            usdt_pairs = []
            for ticker in tickers:
                symbol = ticker['symbol']
                
                # 🎯 核心修复：只处理以USDT结尾的交易对
                if not symbol.endswith('USDT'):
                    continue
                
                # 🚫 过滤稳定币交易对黑名单
                if symbol in self.stable_coin_blacklist:
                    continue
                
                # 检查是否在限制列表中
                if any(restricted in symbol for restricted in self.restricted_coins):
                    continue
                
                # 检查交易状态
                try:
                    symbol_info = self.data_collector.client.get_symbol_info(symbol)
                    if symbol_info and symbol_info.get('status') == 'TRADING':
                        volume = float(ticker['quoteVolume']) # 以USDT计价的交易量
                        usdt_pairs.append((symbol, volume))
                except:
                    continue
            
            # 按交易量排序
            usdt_pairs.sort(key=lambda x: x[1], reverse=True)
            
            # 取前limit个
            top_symbols = [pair[0] for pair in usdt_pairs[:limit]]
            
            print(f"📈 获取到USDT交易对前{len(top_symbols)}的币种 (已过滤稳定币)")
            return top_symbols
            
        except Exception as e:
            print(f"❌ 获取币安USDT币种失败: {e}")
            return None
    
    def _is_stable_coin_pair(self, symbol):
        """修复的稳定币检测 - 只过滤明确的稳定币交易对"""
        # 只检查黑名单中的稳定币交易对
        return symbol in self.stable_coin_blacklist
    
    def _initialize_verified_usdt_coins(self):
        """初始化经过验证的主流USDT币种 - 正确过滤稳定币"""
        # 经过严格筛选的真实可交易主流USDT币种
        verified_usdt_coins = [
            # 大盘币
            'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT', 
            'DOGEUSDT', 'SOLUSDT', 'MATICUSDT', 'DOTUSDT', 'LTCUSDT',
            'BCHUSDT', 'AVAXUSDT', 'LINKUSDT', 'ATOMUSDT', 'XLMUSDT',
            'ETCUSDT', 'XTZUSDT', 'ALGOUSDT', 'FILUSDT', 'EOSUSDT',
            
            # 中盘币
            'NEARUSDT', 'VETUSDT', 'ICPUSDT', 'AAVEUSDT', 'UNIUSDT',
            'THETAUSDT', 'FTMUSDT', 'XMRUSDT', 'SANDUSDT', 'MANAUSDT',
            'CRVUSDT', 'EGLDUSDT', 'KLAYUSDT', 'RUNEUSDT', 'GALAUSDT',
            'ENJUSDT', 'CHZUSDT', 'ONEUSDT', 'IOTAUSDT', 'HBARUSDT',
            
            # 新兴币种
            'APTUSDT', 'OPUSDT', 'ARBUSDT', 'GRTUSDT', 'RNDRUSDT',
            'IMXUSDT', 'STXUSDT', 'MKRUSDT', 'COMPUSDT', 'SNXUSDT',
            'ZILUSDT', 'WAVESUSDT', 'DASHUSDT', 'ZECUSDT', 'BATUSDT',
            
            # 热门币种
            'AXSUSDT', 'SUSHIUSDT', 'YFIUSDT', 'CELRUSDT', 'DYDXUSDT',
            'ENSUSDT', 'LDOUSDT', 'ARUSDT', 'GMTUSDT', 'APEUSDT',
            
            # 补充币种
            'QTUMUSDT', 'IOSTUSDT', 'ONTUSDT', 'ZRXUSDT', 'OMGUSDT',
            'RSRUSDT', 'COTIUSDT', 'OCEANUSDT', 'BANDUSDT', 'NKNUSDT',
            'OXTUSDT', 'STMXUSDT', 'ANKRUSDT', 'PERPUSDT', 'RLCUSDT',
            'TRBUSDT', 'BALUSDT', 'KNCUSDT', 'REEFUSDT', 'CVCUSDT',
            'BELUSDT', 'CTKUSDT', 'TKOUSDT', 'ALPHAUSDT', 'ROSEUSDT',
            'SKLUSDT', 'SRMUSDT', 'SXPUSDT', 'HOTUSDT', 'CHRUSDT',
            'MTLUSDT', 'KAVAUSDT', 'ARPAUSDT', 'LITUSDT', 'DENTUSDT',
            'KEYUSDT', 'DUSKUSDT', 'ATAUSDT', 'PHBUSDT', 'CELOUSDT',
            'TWTUSDT', 'JASMYUSDT', 'RENUSDT', 'PEOPLEUSDT', 'MASKUSDT',
            'AGIXUSDT', 'NMRUSDT', 'API3USDT', 'ANTUSDT', 'LINAUSDT',
            'AUDIOUSDT', 'DODOUSDT', 'TRUUSDT', 'BADGERUSDT', 'FLMUSDT',
            'PONDUSDT', 'ALICEUSDT', 'TLMUSDT', 'DEGOUSDT', 'BICOUSDT',
            'GALUSDT', 'JSTUSDT', 'ACHUSDT', 'IDEXUSDT', 'RADUSDT',
            'SUPERUSDT', 'EPSUSDT', 'AUCTIONUSDT', 'PUNDIXUSDT', 'QIUSDT'
        ]
        
        # 🛠️ 修复：过滤掉稳定币交易对
        filtered_coins = []
        for coin in verified_usdt_coins:
            if not self._is_stable_coin_pair(coin):
                filtered_coins.append(coin)
            else:
                print(f"🚫 过滤稳定币: {coin}")
        
        for coin in filtered_coins:
            self.coin_universe.add(coin)
        
        print(f"✅ 初始化USDT币种完成: {len(filtered_coins)}个 (已过滤稳定币)")
    
    def _strict_validate_all_coins(self):
        """严格验证所有币种 - 确保每个币种都可交易"""
        print("🔍 执行严格币种验证...")
        
        max_attempts = 2
        attempt = 0
        valid_coins = set()
        
        while attempt < max_attempts:
            print(f"🔄 第 {attempt + 1} 轮严格验证...")
            invalid_count = 0
            
            for symbol in list(self.coin_universe):
                if self._strict_validate_single_coin(symbol):
                    valid_coins.add(symbol)
                else:
                    invalid_count += 1
            
            self.coin_universe = valid_coins
            
            if len(self.coin_universe) >= self.min_coins:
                print(f"✅ 严格验证完成: 经过{attempt+1}轮验证, 剩余{len(self.coin_universe)}个有效币种")
                break
                
            if len(self.coin_universe) < self.min_coins and attempt < max_attempts - 1:
                print(f"⚠️ 有效币种不足 {len(self.coin_universe)}，补充新币种...")
                self._add_verified_backup_coins()
                valid_coins = set(self.coin_universe) # 重置为当前列表
            
            attempt += 1
        
        if len(self.coin_universe) < self.min_coins:
            print(f"⚠️ 警告: 最终有效币种数量 {len(self.coin_universe)} 仍低于最小值 {self.min_coins}")
            print("🔧 调整最小币种数量为当前可用数量")
            self.min_coins = max(20, len(self.coin_universe))
    
    def _strict_validate_single_coin(self, symbol):
        """严格验证单个币种 - 三重检查确保可交易"""
        try:
            # 检查1: 交易对信息
            symbol_info = self.data_collector.client.get_symbol_info(symbol)
            if not symbol_info:
                print(f"❌ {symbol}: 无法获取交易对信息")
                return False
            
            # 检查2: 交易状态
            status = symbol_info.get('status')
            if status != 'TRADING':
                print(f"❌ {symbol}: 交易状态异常 - {status}")
                return False
            
            # 检查3: 价格获取
            price = self.data_collector.get_price(symbol)
            if not price or price <= 0:
                print(f"❌ {symbol}: 无法获取有效价格")
                return False
            
            # 检查4: 交易量 - 修改为24小时成交量 > 400万USDT
            klines = self.data_collector.get_klines(symbol, '1h', 24)
            if klines is None or len(klines) < 24:
                print(f"❌ {symbol}: 无法获取K线数据")
                return False
            
            # 计算24小时总成交量（USDT）
            total_volume_24h = (klines['volume'] * klines['close']).sum()
            if total_volume_24h < 4000000: # 400万USDT
                print(f"❌ {symbol}: 24小时成交量不足 - ${total_volume_24h:,.0f} < $4,000,000")
                return False
            
            print(f"✅ {symbol}: 严格验证通过 - 价格${price:.4f}, 24小时成交量${total_volume_24h:,.0f}")
            return True
            
        except Exception as e:
            error_msg = str(e)
            if 'Invalid symbol' in error_msg:
                print(f"❌ {symbol}: 无效交易对")
            elif 'symbol' in error_msg.lower():
                print(f"❌ {symbol}: 交易对错误")
            else:
                print(f"❌ {symbol}: 验证异常 - {e}")
            return False
    
    def _add_verified_backup_coins(self):
        """添加经过验证的备用币种"""
        needed = self.min_coins - len(self.coin_universe)
        if needed <= 0:
            return
        
        print(f"🆕 需要补充 {needed} 个备用币种")
        
        # 经过验证的备用USDT币种
        verified_backup_coins = [
            'BLZUSDT', 'C98USDT', 'GTCUSDT', 'ILVUSDT', 'LQTYUSDT',
            'MCUSDT', 'NEOUSDT', 'POLYXUSDT', 'PORTOUSDT', 'POWRUSDT',
            'RVNUSDT', 'SCUSDT', 'SPELLUSDT', 'STORJUSDT', 'SYSUSDT',
            'UMAUSDT', 'WAXPUSDT', 'XEMUSDT', 'YGGUSDT', '1INCHUSDT',
            'ASTRUSDT', 'CFXUSDT', 'COMBOUSDT', 'COREUSDT', 'CTSIUSDT',
            'DARUSDT', 'DASHUSDT', 'DGBUSDT', 'FLOKIUSDT', 'FLOWUSDT',
            'GFTUSDT', 'HFTUSDT', 'HIFIUSDT', 'HIGHUSDT', 'HIVEUSDT',
            'HOOKUSDT', 'INJUSDT', 'KDAUSDT', 'LOOMUSDT', 'MBLUSDT',
            'MINAUSDT', 'MULTIUSDT', 'NTRNUSDT', 'OGNUSDT', 'ONGUSDT',
            'PNTUSDT', 'QNTUSDT', 'RAREUSDT', 'RDNTUSDT', 'SFPUSDT'
        ]
        
        added_count = 0
        for coin in verified_backup_coins:
            if coin not in self.coin_universe and not self._is_stable_coin_pair(coin):
                self.coin_universe.add(coin)
                added_count += 1
                if added_count >= needed:
                    break
        
        print(f"✅ 补充 {added_count} 个备用USDT币种")
    
    def _get_fallback_coin_list(self):
        """获取备用币种列表"""
        fallback_coins = [
            'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT',
            'DOGEUSDT', 'SOLUSDT', 'MATICUSDT', 'DOTUSDT', 'LTCUSDT',
            'BCHUSDT', 'AVAXUSDT', 'LINKUSDT', 'ATOMUSDT', 'XLMUSDT',
            'ETCUSDT', 'XTZUSDT', 'ALGOUSDT', 'FILUSDT', 'EOSUSDT',
            'NEARUSDT', 'VETUSDT', 'ICPUSDT', 'AAVEUSDT', 'UNIUSDT'
        ]
        self.coin_universe = set(fallback_coins)
        return fallback_coins
    
    def update_coin_scores(self):
        """更新币种评分 - 使用修复的验证"""
        print("🔄 更新币种评分...")
        
        current_time = time.time()
        if self.last_update and current_time - self.last_update < self.update_interval:
            return list(self.coin_universe)
        
        try:
            # 首先重新验证所有币种
            self._strict_validate_all_coins()
            
            # 然后更新评分
            for symbol in list(self.coin_universe):
                try:
                    klines = self.data_collector.get_klines(symbol, '1d', 30)
                    if klines is None or len(klines) < 20:
                        self.coin_scores[symbol] = self.coin_scores.get(symbol, 50) - 10
                        continue
                    
                    score = self._calculate_coin_score(symbol, klines)
                    self.coin_scores[symbol] = score
                    
                    time.sleep(0.05) # 降低频率避免API限制
                    
                except Exception as e:
                    print(f"❌ 评分币种 {symbol} 失败: {e}")
                    self.coin_scores[symbol] = self.coin_scores.get(symbol, 50) - 5
            
            self._remove_low_score_coins()
            
            self.last_update = current_time
            print(f"✅ 币种评分更新完成: {len(self.coin_universe)} 个币种")
            
            return list(self.coin_universe)
            
        except Exception as e:
            print(f"❌ 更新币种评分失败: {e}")
            return list(self.coin_universe)
    
    def _calculate_coin_score(self, symbol, data):
        """计算币种评分"""
        try:
            score = 50
            
            # 技术指标
            current_price = data['close'].iloc[-1]
            price_change_24h = (current_price - data['close'].iloc[-2]) / data['close'].iloc[-2] if len(data) >= 2 else 0
            
            # 价格变化
            if price_change_24h > 0:
                score += min(20, price_change_24h * 200) # 强化正向变化
            else:
                score += max(-15, price_change_24h * 150) # 弱化负向影响
            
            # 交易量分析
            current_volume = data['volume'].iloc[-1]
            avg_volume = data['volume'].rolling(20).mean().iloc[-1]
            if avg_volume > 0:
                volume_ratio = current_volume / avg_volume
                if volume_ratio > 2.0:
                    score += 20
                elif volume_ratio > 1.5:
                    score += 15
                elif volume_ratio > 1.2:
                    score += 10
                elif volume_ratio < 0.5:
                    score -= 10
            
            # 波动性分析
            volatility = data['close'].pct_change().std()
            if 0.02 < volatility < 0.06: # 适中波动性最佳
                score += 15
            elif volatility > 0.10: # 过高波动性扣分
                score -= 10
            
            # 趋势分析
            if len(data) >= 20:
                ma20 = data['close'].rolling(20).mean().iloc[-1]
                ma50 = data['close'].rolling(50).mean().iloc[-1]
                if current_price > ma20:
                    score += 10
                if current_price > ma50:
                    score += 15
                if ma20 > ma50:
                    score += 10
            
            return max(10, min(100, score)) # 确保分数在合理范围内
            
        except Exception as e:
            print(f"❌ 计算币种 {symbol} 评分失败: {e}")
            return 50
    
    def _remove_low_score_coins(self):
        """移除低分币种"""
        try:
            if not self.coin_scores:
                return
            
            avg_score = np.mean(list(self.coin_scores.values()))
            coins_to_remove = []
            
            for symbol, score in self.coin_scores.items():
                # 更严格的淘汰标准
                if (score < avg_score - 15 or score < 40) and len(self.coin_universe) > self.min_coins:
                    coins_to_remove.append(symbol)
            
            for symbol in coins_to_remove:
                self.coin_universe.remove(symbol)
                del self.coin_scores[symbol]
                print(f"🗑️ 淘汰低分币种: {symbol} (评分: {score:.1f})")
                
        except Exception as e:
            print(f"❌ 移除低分币种失败: {e}")
    
    def get_top_coins(self, count=80):
        """获取评分最高的币种"""
        try:
            if not self.coin_scores:
                return list(self.coin_universe)[:count]
            
            # 确保只返回真实可交易的币种
            valid_scores = {}
            for symbol, score in self.coin_scores.items():
                if symbol in self.coin_universe: # 双重确认
                    valid_scores[symbol] = score
            
            sorted_coins = sorted(valid_scores.items(), key=lambda x: x[1], reverse=True)
            return [coin[0] for coin in sorted_coins[:count]]
            
        except Exception as e:
            print(f"❌ 获取高分币种失败: {e}")
            return list(self.coin_universe)[:count]

    def get_coin_universe_stats(self):
        """获取币种列表统计信息"""
        return {
            'total_coins': len(self.coin_universe),
            'scored_coins': len(self.coin_scores),
            'avg_score': np.mean(list(self.coin_scores.values())) if self.coin_scores else 0,
            'top_10_coins': self.get_top_coins(10)
        }

# ========== 智能孤儿持仓恢复器 - 增强退市币种屏蔽版本 ==========
class SmartOrphanPositionRecovery:
    """智能孤儿持仓恢复器 - 恢复失败不计入程序持仓，严格屏蔽退市币种"""
    
    def __init__(self, bot):
        self.bot = bot
        self.orphan_positions = {}
        
        # 退市币种列表 - 严格屏蔽这些币种，不占用持仓名额
        self.delisted_coins = set(['AUTO', 'NEBL']) # 您提到的两个退市币种
        
        # 从环境变量加载更多退市币种
        config_delisted = os.getenv('DELISTED_COINS', '').split(',')
        config_delisted = [coin.strip().upper() for coin in config_delisted if coin.strip()]
        self.delisted_coins.update(config_delisted)
        
        print(f"🛡️ 退市币种屏蔽列表: {sorted(list(self.delisted_coins))}")
    
    def recover_orphan_positions(self):
        """恢复孤儿持仓，但恢复失败的持仓不计入程序管理，严格屏蔽退市币种"""
        print("🔄 开始智能孤儿持仓恢复（严格退市检测）...")
        
        # 获取账户所有实际资产
        actual_assets = self._get_all_actual_assets()
        if not actual_assets:
            print("❌ 无法获取账户资产")
            return False, [] # 返回成功状态和恢复的持仓列表
        
        # 严格过滤退市币种
        filtered_assets = self._strict_filter_delisted_coins(actual_assets)
        
        # 获取程序已知的持仓
        known_positions = set(self.bot.position_manager.positions.keys())
        
        # 识别孤儿持仓（已过滤退市币种）
        orphan_assets = self._identify_orphan_assets(filtered_assets, known_positions)
        
        if not orphan_assets:
            print("✅ 未发现有效的孤儿持仓")
            return True, []
        
        # 恢复孤儿持仓
        recovered_positions = self._recover_orphans_to_positions(orphan_assets)
        
        # 统计恢复结果
        recovered_count = len(recovered_positions)
        total_orphans = len(orphan_assets)
        
        print(f"📊 孤儿持仓恢复结果:")
        print(f" 发现有效孤儿资产: {total_orphans}个")
        print(f" 成功恢复: {recovered_count}个")
        print(f" 恢复失败: {total_orphans - recovered_count}个")
        
        # 重要：报告被屏蔽的退市币种
        self._report_strict_delisted_coins(actual_assets, filtered_assets)
        
        if total_orphans - recovered_count > 0:
            print(f"⚠️ 注意: {total_orphans - recovered_count}个孤儿资产恢复失败，这些持仓将不计入程序管理")
            print(f" 程序将继续管理 {recovered_count}个成功恢复的持仓")
        
        return recovered_count > 0, recovered_positions
    
    def _strict_filter_delisted_coins(self, actual_assets):
        """严格过滤退市币种 - 确保不占用持仓名额"""
        filtered_assets = {}
        removed_count = 0
        
        for asset, quantity in actual_assets.items():
            # 1. 检查是否在已知退市币种列表中
            if asset in self.delisted_coins:
                print(f"🚫 严格屏蔽已知退市币种: {asset} - 数量: {quantity:.8f}")
                removed_count += 1
                continue
            
            # 2. 动态检测币种是否可交易
            if not self._check_coin_strictly_tradable(asset):
                print(f"🚫 严格屏蔽不可交易币种: {asset} - 数量: {quantity:.8f}")
                self.delisted_coins.add(asset) # 添加到退市列表
                removed_count += 1
                continue
            
            filtered_assets[asset] = quantity
        
        if removed_count > 0:
            print(f"🛡️ 严格屏蔽完成: 共移除 {removed_count} 个退市/不可交易币种")
            print(f"🛡️ 剩余有效币种: {len(filtered_assets)} 个")
        
        return filtered_assets
    
    def _check_coin_strictly_tradable(self, asset):
        """严格检测币种是否可交易 - 确保只有真正可交易的币种才被恢复"""
        # 按优先级尝试交易对
        trading_pairs = [
            f"{asset}USDT", # 最高优先级
            f"{asset}BUSD",  
        ]
        
        for symbol in trading_pairs:
            try:
                # 严格检查：交易对必须存在且状态为TRADING
                symbol_info = self.bot.data_collector.client.get_symbol_info(symbol)
                if symbol_info:
                    status = symbol_info.get('status', '')
                    if status == 'TRADING':
                        # 进一步检查：必须能获取到有效价格
                        price = self.bot.data_collector.get_price(symbol)
                        if price and price > 0:
                            print(f"✅ 币种 {asset} 严格验证通过: {symbol} 价格 ${price:.6f}")
                            return True
                        else:
                            print(f"⚠️ 币种 {asset} 无法获取有效价格: {symbol}")
                            return False
                    else:
                        print(f"⚠️ 币种 {asset} 交易对状态异常: {status}")
                        return False
            except Exception as e:
                if "Invalid symbol" in str(e):
                    continue # 交易对不存在，继续尝试下一个
                else:
                    print(f"⚠️ 检查币种 {asset} 时出现异常: {e}")
                    # 其他异常暂时认为不可交易
                    return False
        
        # 所有主要交易对检查都失败，认为币种已退市
        print(f"🚫 币种 {asset} 严格验证失败: 无有效交易对")
        return False
    
    def _report_strict_delisted_coins(self, all_assets, filtered_assets):
        """严格报告被屏蔽的退市币种 - 强调不计入持仓"""
        delisted_assets = set(all_assets.keys()) - set(filtered_assets.keys())
        
        if delisted_assets:
            print(f"\n{'🚫'*50}")
            print("🚫 严格退市币种报告 - 这些币种将完全不计入程序持仓管理")
            print(f"{'🚫'*50}")
            
            total_quantity = 0
            for asset in delisted_assets:
                quantity = all_assets[asset]
                total_quantity += quantity
                
                print(f" ❌ {asset}: {quantity:.8f} (完全不计入持仓)")
            
            print(f"📊 退市币种总数量: {len(delisted_assets)} 个")
            print(f"📊 退市币种总持有量: {total_quantity:.8f}")
            print(f"💡 重要: 这些币种不会占用持仓名额，不影响正常交易逻辑")
            print(f"💡 建议: 需要手动在交易所处理这些退市币种")
            print(f"{'🚫'*50}\n")
            
            # 额外强调对交易的影响
            current_positions = len(self.bot.position_manager.positions)
            max_positions = self.bot.position_manager.get_max_positions(self.bot.current_market_state)
            print(f"🎯 当前程序管理持仓: {current_positions}/{max_positions}")
            print(f"✅ 退市币种屏蔽后，可用持仓名额: {max_positions - current_positions}")
    
    def _get_all_actual_assets(self):
        """获取账户所有实际资产（包括手动购买的）"""
        try:
            @self.bot.data_collector.api_retry_manager.retry_api_call
            def get_account_safe():
                return self.bot.data_collector.client.get_account()
            
            account_info = get_account_safe()
            if not account_info:
                return None
            
            assets = {}
            stable_coins = ['USDT', 'BUSD', 'USDC', 'TUSD', 'USDP', 'DAI']
            
            for balance in account_info['balances']:
                asset = balance['asset']
                free = float(balance['free'])
                locked = float(balance['locked'])
                total = free + locked
                
                if total > 0.000001 and not asset.startswith('LD') and asset not in stable_coins:
                    assets[asset] = total
                    print(f"📦 发现实际资产: {asset} - {total:.8f}")
            
            return assets
            
        except Exception as e:
            print(f"❌ 获取实际资产失败: {e}")
            return None
    
    def _identify_orphan_assets(self, actual_assets, known_positions):
        """识别孤儿资产"""
        orphan_assets = {}
        
        # 将已知持仓转换为资产格式（去掉交易对后缀）
        known_assets = set()
        for symbol in known_positions:
            # 移除USDT、BUSD等后缀得到资产名称
            for suffix in ['USDT', 'BUSD', 'BTC', 'ETH', 'BNB']:
                if symbol.endswith(suffix):
                    asset = symbol[:-len(suffix)]
                    known_assets.add(asset)
                    break
        
        print(f"🔍 资产对比: 实际{len(actual_assets)}个 vs 已知{len(known_assets)}个")
        
        for asset, quantity in actual_assets.items():
            if asset not in known_assets:
                orphan_assets[asset] = quantity
                print(f"🚨 发现孤儿资产: {asset} - {quantity:.8f}")
        
        return orphan_assets
    
    def _recover_orphans_to_positions(self, orphan_assets):
        """将孤儿资产恢复为持仓，返回成功恢复的列表 - 严格退市检测"""
        recovered_positions = []
        
        for asset, quantity in orphan_assets.items():
            try:
                # 三重检查：确保币种可交易
                if not self._check_coin_strictly_tradable(asset):
                    print(f"🚫 最终确认币种 {asset} 不可交易，严格跳过恢复")
                    self.delisted_coins.add(asset)
                    continue
                
                # 寻找合适的交易对
                symbol, current_price = self._find_best_trading_pair(asset)
                if not symbol or not current_price:
                    print(f"🚫 无法为 {asset} 找到有效交易对，严格跳过")
                    self.delisted_coins.add(asset)
                    continue
                
                # 最终交易对状态检查
                try:
                    symbol_info = self.bot.data_collector.client.get_symbol_info(symbol)
                    if symbol_info.get('status') != 'TRADING':
                        print(f"🚫 交易对 {symbol} 最终状态检查失败: {symbol_info.get('status')}")
                        self.delisted_coins.add(asset)
                        continue
                except Exception as e:
                    print(f"🚫 最终交易对状态检查异常 {symbol}: {e}")
                    self.delisted_coins.add(asset)
                    continue
                
                # 计算持仓市值
                position_value = quantity * current_price
                
                # 关键修改：只恢复市值大于等于1美元的持仓
                if position_value < 1.0:
                    print(f"⏩ 跳过小额孤儿资产: {asset} (${position_value:.2f} < $1.00)")
                    continue
                
                # 🆕 增强：尝试从持久化数据获取实际成本价
                actual_cost_price = self.bot.persistence_manager._get_actual_cost_price_from_history(asset)
                if actual_cost_price and actual_cost_price > 0:
                    entry_price = actual_cost_price
                    cost_known = True
                else:
                    entry_price = current_price
                    cost_known = False
                    print(f"⚠️ 无法获取 {symbol} 的实际成本价，使用当前价格估算")
                
                # 创建恢复的持仓记录
                recovered_position = {
                    'symbol': symbol,
                    'quantity': quantity,
                    'entry_price': entry_price,
                    'total_invested': quantity * entry_price,
                    'entry_time': datetime.now(),
                    'recovered': True,
                    'recovery_time': datetime.now().isoformat(),
                    'cost_known': cost_known, # 🆕 标记成本价是否准确
                    'original_entry_time': datetime.now(), # 🆕 保存原始时间
                    'original_entry_price': entry_price # 🆕 保存原始价格
                }
                
                # 添加到持仓管理器 - 只有这里才会真正计入程序持仓
                self.bot.position_manager.positions[symbol] = recovered_position
                
                # 记录到持仓时间管理器
                self.bot.holding_manager.record_buy(symbol)
                
                recovered_positions.append(symbol)
                print(f"✅ 严格验证通过，恢复孤儿持仓: {symbol}, 数量: {quantity:.8f}, 价值: ${position_value:.2f}, 成本价: {'已知' if cost_known else '估算'}")
                      
            except Exception as e:
                print(f"❌ 恢复孤儿资产 {asset} 失败: {e}")
                continue
        
        return recovered_positions
    
    def _find_best_trading_pair(self, asset):
        """为资产寻找最佳交易对"""
        # 按优先级尝试交易对
        trading_pairs = [
            f"{asset}USDT", # 最高优先级
            f"{asset}BUSD",  
            f"{asset}BTC",   
            f"{asset}ETH",   
            f"{asset}BNB",   
        ]
        
        for symbol in trading_pairs:
            try:
                # 首先检查交易对是否存在
                self.bot.data_collector.client.get_symbol_info(symbol)
                # 然后获取价格
                price = self.bot.data_collector.get_price(symbol)
                if price and price > 0:
                    return symbol, price
            except Exception as e:
                if "Invalid symbol" not in str(e):
                    print(f"⚠️ 检查交易对 {symbol} 失败: {e}")
                continue
        
        # 如果标准交易对都不存在，尝试反向查找
        reverse_pairs = [
            f"USDT{asset}",
            f"BUSD{asset}",
            f"BTC{asset}", 
            f"ETH{asset}",
            f"BNB{asset}"
        ]
        
        for symbol in reverse_pairs:
            try:
                self.bot.data_collector.client.get_symbol_info(symbol)
                price = self.bot.data_collector.get_price(symbol)
                if price and price > 0:
                    # 反向交易对需要计算实际价值
                    actual_price = 1 / price if price > 0 else 0
                    return symbol, actual_price
            except Exception:
                continue
        
        return None, None
    
    def create_orphan_recovery_report(self):
        """生成孤儿持仓恢复报告"""
        recovered_positions = {}
        for symbol, position in self.bot.position_manager.positions.items():
            if position.get('recovered'):
                recovered_positions[symbol] = position
        
        report = {
            'recovery_time': datetime.now().isoformat(),
            'recovered_count': len(recovered_positions),
            'recovered_positions': recovered_positions,
            'total_orphan_value': sum(pos['total_invested'] for pos in recovered_positions.values()),
            'delisted_coins_count': len(self.delisted_coins),
            'delisted_coins': list(self.delisted_coins)
        }
        
        return report

# ========== 智能灾难恢复管理器 ==========
class SmartDisasterRecoveryManager:
    """智能灾难恢复管理器 - 处理严重状态不一致"""
    
    def __init__(self, bot):
        self.bot = bot
        self.orphan_recovery = SmartOrphanPositionRecovery(bot)
    
    def execute_disaster_recovery(self):
        """执行智能灾难恢复"""
        print("🚨 执行智能灾难恢复程序...")
        
        recovery_steps = [
            self._step1_backup_current_state,
            self._step2_validate_account_assets,
            self._step3_smart_recover_orphan_positions, # 修改这一步
            self._step4_rebuild_position_manager,
            self._step5_verify_recovery,
            self._step6_create_recovery_report
        ]
        
        for i, step in enumerate(recovery_steps, 1):
            print(f"🔄 灾难恢复步骤 {i}/6...")
            success = step()
            if not success and i == 3: # 孤儿持仓恢复部分失败可以继续
                print("⚠️ 孤儿持仓恢复部分失败，但程序将继续运行")
                continue
            elif not success:
                print(f"❌ 灾难恢复步骤 {i} 失败")
                return False
        
        print("🎉 智能灾难恢复完成!")
        return True
    
    def _step1_backup_current_state(self):
        """步骤1: 备份当前状态"""
        try:
            backup_file = f"disaster_recovery_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(backup_file, 'w') as f:
                import json
                json.dump({
                    'positions': self.bot.position_manager.positions,
                    'timestamp': datetime.now().isoformat()
                }, f, indent=2, default=str)
            print(f"📦 状态备份完成: {backup_file}")
            return True
        except Exception as e:
            print(f"⚠️ 状态备份失败: {e}")
            return True # 继续执行
    
    def _step2_validate_account_assets(self):
        """步骤2: 验证账户资产"""
        try:
            actual_assets = self.orphan_recovery._get_all_actual_assets()
            if not actual_assets:
                print("❌ 无法验证账户资产")
                return False
            
            print(f"📊 账户资产验证: 共{len(actual_assets)}个非稳定币资产")
            for asset, quantity in actual_assets.items():
                print(f" ✅ {asset}: {quantity:.8f}")
            
            return True
        except Exception as e:
            print(f"❌ 账户资产验证失败: {e}")
            return False
    
    def _step3_smart_recover_orphan_positions(self):
        """步骤3: 智能恢复孤儿持仓 - 失败不计入程序持仓，严格屏蔽退市币种"""
        success, recovered_positions = self.orphan_recovery.recover_orphan_positions()
        
        if not success:
            print("⚠️ 孤儿持仓恢复部分失败，失败持仓将不计入程序管理")
            print(f"✅ 成功恢复的 {len(recovered_positions)} 个持仓将计入程序管理")
        
        return True # 总是返回True，让恢复继续
    
    def _step4_rebuild_position_manager(self):
        """步骤4: 重建持仓管理器"""
        try:
            # 重新计算所有持仓价值
            total_positions_value = 0
            valid_positions = {}
            
            for symbol, position in self.bot.position_manager.positions.items():
                try:
                    current_price = self.bot.data_collector.get_price(symbol)
                    if current_price and current_price > 0:
                        position_value = position['quantity'] * current_price
                        total_positions_value += position_value
                        valid_positions[symbol] = position
                    else:
                        print(f"⚠️ 移除无效持仓: {symbol} (无法获取价格)")
                except Exception as e:
                    print(f"⚠️ 检查持仓失败 {symbol}: {e}")
            
            # 更新持仓管理器
            self.bot.position_manager.positions = valid_positions
            
            # 更新资金管理器
            self.bot.live_capital_manager.allocated_capital = total_positions_value
            
            print(f"💰 持仓重建: {len(valid_positions)}个有效持仓, 总价值${total_positions_value:.2f}")
            return True
            
        except Exception as e:
            print(f"❌ 重建持仓管理器失败: {e}")
            return False
    
    def _step5_verify_recovery(self):
        """步骤5: 验证恢复结果"""
        try:
            # 获取实际资产
            actual_assets = self.orphan_recovery._get_all_actual_assets()
            if not actual_assets:
                return False
            
            # 计算程序识别的资产
            program_assets = set()
            for symbol in self.bot.position_manager.positions.keys():
                for suffix in ['USDT', 'BUSD', 'BTC', 'ETH', 'BNB']:
                    if symbol.endswith(suffix):
                        asset = symbol[:-len(suffix)]
                        program_assets.add(asset)
                        break
            
            # 对比
            actual_asset_set = set(actual_assets.keys())
            missing_assets = actual_asset_set - program_assets
            
            if missing_assets:
                # 检查缺失的资产是否为退市币种
                delisted_missing = [asset for asset in missing_assets if asset in self.orphan_recovery.delisted_coins]
                non_delisted_missing = [asset for asset in missing_assets if asset not in self.orphan_recovery.delisted_coins]
                
                if delisted_missing:
                    print(f"✅ 退市币种正确屏蔽: {delisted_missing}")
                
                if non_delisted_missing:
                    print(f"⚠️ 恢复验证: 仍有{len(non_delisted_missing)}个非退市资产未被识别")
                    for asset in non_delisted_missing:
                        print(f" ❌ 缺失: {asset}")
                    return False
                else:
                    print("✅ 恢复验证: 所有非退市资产已被正确识别")
                    return True
            else:
                print("✅ 恢复验证: 所有资产已被正确识别")
                return True
                
        except Exception as e:
            print(f"❌ 恢复验证失败: {e}")
            return False
    
    def _step6_create_recovery_report(self):
        """步骤6: 创建恢复报告"""
        try:
            report = self.orphan_recovery.create_orphan_recovery_report()
            
            print("\n" + "="*60)
            print("📋 智能灾难恢复报告")
            print("="*60)
            print(f"恢复时间: {report['recovery_time']}")
            print(f"恢复持仓数量: {report['recovered_count']}个")
            print(f"恢复持仓总价值: ${report['total_orphan_value']:.2f}")
            print(f"屏蔽退市币种数量: {report['delisted_coins_count']}个")
            
            if report['recovered_positions']:
                print("恢复的持仓详情:")
                for symbol, position in report['recovered_positions'].items():
                    print(f" 📦 {symbol}: {position['quantity']:.6f} "
                          f"@ ${position['entry_price']:.6f} "
                          f"= ${position['total_invested']:.2f}")
            
            if report['delisted_coins']:
                print("屏蔽的退市币种:")
                for coin in report['delisted_coins']:
                    print(f" 🚫 {coin}")
            
            print("="*60)
            
            # 保存报告到文件
            report_file = f"disaster_recovery_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(report_file, 'w') as f:
                import json
                json.dump(report, f, indent=2, default=str)
            
            print(f"📄 恢复报告已保存: {report_file}")
            return True
            
        except Exception as e:
            print(f"⚠️ 创建恢复报告失败: {e}")
            return True

# ========== 实盘资金管理器 - 修正版本 ==========
class LiveTradingCapitalManager:
    """实盘资金管理器 - 修正资金计算逻辑"""
    
    def __init__(self, data_collector, mode_selector):
        self.data_collector = data_collector
        self.mode_selector = mode_selector
        
        # 核心资金变量 - 修正定义
        self.total_assets = 0 # 总资产 = USDT余额 + 所有持仓当前市值
        self.usdt_balance = 0 # USDT可用余额（可操作资金）
        self.trading_capital = 0 # 交易资金 = 总资产 * 资本比率
        self.allocated_capital = 0 # 已分配的交易资金（持仓成本总额）
        
        self.capital_ratio = 0.50
        self.last_balance_update = None
        
    def initialize_capital(self):
        """初始化资金 - 修正逻辑"""
        try:
            if self.mode_selector.mode == 'testnet':
                # 测试网模式
                self.usdt_balance = 10000
                self.total_assets = self.usdt_balance
                self.trading_capital = self.total_assets * self.capital_ratio
                print(f"🔬 测试网模式: 总资产${self.total_assets:.2f}, "
                      f"USDT余额${self.usdt_balance:.2f}, "
                      f"交易资金${self.trading_capital:.2f}")
            else:
                # 实盘模式
                success = self._update_live_balance_correct()
                if not success:
                    print("⚠️ 使用默认资金继续运行")
                    self.usdt_balance = 1000
                    self.total_assets = self.usdt_balance
                    self.trading_capital = self.total_assets * self.capital_ratio
                
            return self.usdt_balance
            
        except Exception as e:
            print(f"❌ 初始化资金失败: {e}")
            self.usdt_balance = 1000
            self.total_assets = self.usdt_balance
            self.trading_capital = self.total_assets * self.capital_ratio
            return self.usdt_balance
    
    def _update_live_balance_correct(self):
        """修正的实盘余额更新 - 增强资产识别"""
        try:
            if self.mode_selector.mode != 'live':
                return True
                
            print("🔄 获取实盘资金信息...")
            
            @self.data_collector.api_retry_manager.retry_api_call
            def get_account_safe():
                return self.data_collector.client.get_account()
            
            account_info = get_account_safe()
            
            if not account_info:
                print("❌ 无法获取账户信息")
                return False
                
            # 重置资金变量
            total_usdt_value = 0
            usdt_balance = 0
            positions_value = 0
            asset_count = 0
            
            # 支持的稳定币列表
            stable_coins = ['USDT', 'BUSD', 'USDC', 'TUSD', 'USDP', 'DAI']
            
            print("📋 账户所有资产:")
            for balance in account_info['balances']:
                asset = balance['asset']
                free = float(balance['free'])
                locked = float(balance['locked'])
                total = free + locked
                
                # 跳过零余额和定期产品
                if total <= 0.000001 or asset.startswith('LD'):
                    continue
                
                asset_count += 1
                
                if asset in stable_coins:
                    total_usdt_value += total
                    if asset == 'USDT':
                        usdt_balance += free
                    print(f"💰 稳定币 {asset}: {total:.6f}")
                else:
                    asset_value = self._calculate_asset_value(asset, total)
                    if asset_value > 0:
                        total_usdt_value += asset_value
                        positions_value += asset_value
                        asset_count += 1
                    else:
                        print(f"⚠️ 无法估值: {asset} - {total:.6f}")
            
            # 关键修正：正确的资金计算
            self.total_assets = total_usdt_value
            self.usdt_balance = usdt_balance
            self.trading_capital = total_usdt_value * self.capital_ratio
            self.allocated_capital = positions_value
            
            self.last_balance_update = datetime.now()
            
            print(f"🚀 资金统计完成:")
            print(f" 总资产: ${self.total_assets:.2f}")
            print(f" USDT余额: ${self.usdt_balance:.2f}")
            print(f" 交易资金: ${self.trading_capital:.2f}")
            print(f" 已分配资金: ${self.allocated_capital:.2f}")
            print(f" 识别资产: {asset_count}个")
            
            return True
            
        except Exception as e:
            print(f"❌ 更新实盘余额失败: {e}")
            import traceback
            traceback.print_exc()
            return False

    def _calculate_asset_value(self, asset, quantity):
        """计算资产价值 - 多策略尝试"""
        # 策略1: 直接USDT交易对
        symbol = f"{asset}USDT"
        try:
            price = self.data_collector.get_price(symbol)
            if price and price > 0:
                value = quantity * price
                print(f"💰 {asset}: {quantity:.6f} ≈ ${value:.2f} (USDT)")
                return value
        except:
            pass
        
        # 策略2: BUSD交易对
        symbol = f"{asset}BUSD"
        try:
            price = self.data_collector.get_price(symbol)
            if price and price > 0:
                value = quantity * price
                print(f"💰 {asset}: {quantity:.6f} ≈ ${value:.2f} (BUSD)")
                return value
        except:
            pass
        
        # 策略3: 通过BTC中转
        try:
            btc_symbol = f"{asset}BTC"
            btc_price = self.data_collector.get_price(btc_symbol)
            btc_usdt_price = self.data_collector.get_price("BTCUSDT")
            
            if btc_price and btc_usdt_price:
                value = quantity * btc_price * btc_usdt_price
                print(f"💰 {asset}: {quantity:.6f} ≈ ${value:.2f} (BTC中转)")
                return value
        except:
            pass
        
        # 策略4: 通过ETH中转
        try:
            eth_symbol = f"{asset}ETH"
            eth_price = self.data_collector.get_price(eth_symbol)
            eth_usdt_price = self.data_collector.get_price("ETHUSDT")
            
            if eth_price and eth_usdt_price:
                value = quantity * eth_price * eth_usdt_price
                print(f"💰 {asset}: {quantity:.6f} ≈ ${value:.2f} (ETH中转)")
                return value
        except:
            pass
        
        # 策略5: 通过BNB中转
        try:
            bnb_symbol = f"{asset}BNB"
            bnb_price = self.data_collector.get_price(bnb_symbol)
            bnb_usdt_price = self.data_collector.get_price("BNBUSDT")
            
            if bnb_price and bnb_usdt_price:
                value = quantity * bnb_price * bnb_usdt_price
                print(f"💰 {asset}: {quantity:.6f} ≈ ${value:.2f} (BNB中转)")
                return value
        except:
            pass
        
        return 0
    
    def recalculate_after_sell(self, symbol, sell_amount, sold_quantity, sold_cost):
        """卖出后重新计算资金"""
        try:
            print(f"🔄 {symbol} 卖出后资金重计算:")
            print(f" 卖出数量: {sold_quantity:.6f}")
            print(f" 实际收到USDT: ${sell_amount:.2f}")
            print(f" 收回成本: ${sold_cost:.2f}")
            print(f" 实现盈亏: ${sell_amount - sold_cost:+.2f}")
            
            if self.mode_selector.mode == 'live':
                return self._update_live_balance_correct()
            else:
                # 测试网模式手动计算
                old_usdt = self.usdt_balance
                old_allocated = self.allocated_capital
                old_total = self.total_assets
                
                self.usdt_balance += sell_amount
                self.allocated_capital -= sold_cost
                self.total_assets = self.usdt_balance + self.allocated_capital
                self.trading_capital = self.total_assets * self.capital_ratio
                
                print(f"💰 资金变化详情:")
                print(f" USDT余额: ${old_usdt:.2f} → ${self.usdt_balance:.2f} (+${sell_amount:.2f})")
                print(f" 已分配资金: ${old_allocated:.2f} → ${self.allocated_capital:.2f} (-${sold_cost:.2f})")
                print(f" 总资产: ${old_total:.2f} → ${self.total_assets:.2f}")
                print(f" 交易资金: ${self.trading_capital:.2f}")
                
                return True
                
        except Exception as e:
            print(f"❌ 卖出后资金重计算失败: {e}")
            return False
    
    def recalculate_after_buy(self, symbol, buy_amount):
        """买入后重新计算资金"""
        try:
            print(f"🔄 {symbol} 买入后资金重计算:")
            
            if self.mode_selector.mode == 'live':
                return self._update_live_balance_correct()
            else:
                old_usdt = self.usdt_balance
                old_allocated = self.allocated_capital
                
                self.usdt_balance -= buy_amount
                self.allocated_capital += buy_amount
                
                print(f"💰 买入后资金更新:")
                print(f" USDT余额: ${old_usdt:.2f} → ${self.usdt_balance:.2f} (-${buy_amount:.2f})")
                print(f" 已分配资金: ${old_allocated:.2f} → ${self.allocated_capital:.2f} (+${buy_amount:.2f})")
                print(f" 总资产: ${self.total_assets:.2f}")
                print(f" 交易资金: ${self.trading_capital:.2f}")
                
                return True
                
        except Exception as e:
            print(f"❌ 买入后资金重计算失败: {e}")
            return False
    
    def get_available_trading_capital(self):
        """获取可用交易资金 - 修正逻辑"""
        try:
            available_from_allocation = max(0, self.trading_capital - self.allocated_capital)
            actual_available = min(available_from_allocation, self.usdt_balance)
            return actual_available
        except Exception as e:
            print(f"❌ 计算可用交易资金失败: {e}")
            return min(self.trading_capital, self.usdt_balance)
    
    def update_trading_capital(self):
        """更新交易资金"""
        try:
            current_time = datetime.now()
            if (self.last_balance_update is None or 
                (current_time - self.last_balance_update).total_seconds() > 3600):
                if self.mode_selector.mode == 'live':
                    self._update_live_balance_correct()
                else:
                    self.total_assets = 10000
                    self.usdt_balance = 10000
                    self.trading_capital = self.total_assets * self.capital_ratio
                    self.allocated_capital = 0
            
            return self.get_available_trading_capital()
            
        except Exception as e:
            print(f"❌ 更新交易资金失败: {e}")
            return self.get_available_trading_capital()

# ========== 资产统计管理器 - 修复版本 ==========
class AssetStatisticsManager:
    """资产统计管理器 - 完整资产报告"""
    
    def __init__(self, bot):
        self.bot = bot
        self.initial_capital = 0
        self.daily_start_balance = 0
        self.last_daily_reset = datetime.now().date()
        self.performance_history = []
    
    def initialize_statistics(self):
        """初始化统计数据"""
        self.initial_capital = self.bot.live_capital_manager.total_assets
        self.daily_start_balance = self.bot.live_capital_manager.total_assets
        print(f"📈 初始总资产: ${self.initial_capital:.2f}")
    
    def generate_complete_asset_report(self):
        """生成完整资产报告"""
        try:
            total_assets = self.bot.live_capital_manager.total_assets
            usdt_balance = self.bot.live_capital_manager.usdt_balance
            trading_capital = self.bot.live_capital_manager.trading_capital
            allocated_capital = self.bot.live_capital_manager.allocated_capital
            
            # 计算总体盈亏
            total_pnl = total_assets - self.initial_capital
            total_pnl_percent = (total_pnl / self.initial_capital * 100) if self.initial_capital > 0 else 0
            
            # 计算当日盈亏
            if datetime.now().date() != self.last_daily_reset:
                self.daily_start_balance = total_assets
                self.last_daily_reset = datetime.now().date()
            
            daily_pnl = total_assets - self.daily_start_balance
            daily_pnl_percent = (daily_pnl / self.daily_start_balance * 100) if self.daily_start_balance > 0 else 0
            
            # 可用交易资金
            available_trading_capital = trading_capital - allocated_capital
            actual_available = min(available_trading_capital, usdt_balance)
            
            report = {
                'timestamp': datetime.now(),
                'total_assets': total_assets,
                'usdt_balance': usdt_balance,
                'trading_capital': trading_capital,
                'allocated_capital': allocated_capital,
                'available_trading_capital': available_trading_capital,
                'actual_available_capital': actual_available,
                'capital_usage_rate': (allocated_capital / trading_capital * 100) if trading_capital > 0 else 0,
                'positions_count': len(self.bot.position_manager.positions),
                'total_pnl': total_pnl,
                'total_pnl_percent': total_pnl_percent,
                'daily_pnl': daily_pnl,
                'daily_pnl_percent': daily_pnl_percent,
                'initial_capital': self.initial_capital,
                'positions_detail': self._get_positions_detail(),
                'buying_power': self._calculate_buying_power(actual_available),
                'risk_metrics': self._calculate_risk_metrics(),
                'funds_breakdown': self._get_funds_breakdown()
            }
            
            return report
            
        except Exception as e:
            print(f"❌ 生成资产报告失败: {e}")
            return self._get_empty_report()
    
    def _get_positions_detail(self):
        """获取持仓详情 - 修复时间计算"""
        positions_detail = {}
        total_invested = 0
        total_current_value = 0
        
        for symbol, position in self.bot.position_manager.positions.items():
            try:
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price and current_price > 0:
                    # 🆕 使用持久化数据计算实际成本
                    performance_data = self.bot.persistence_manager.calculate_holding_performance(symbol)
                    if performance_data and performance_data.get('is_accurate', False):
                        entry_price = performance_data['entry_price']
                        invested = performance_data['invested_value']
                        entry_time = performance_data['entry_time']
                        holding_hours = performance_data['holding_hours']
                    else:
                        # 回退到持仓管理器中的数据
                        entry_price = position['entry_price']
                        invested = position.get('total_invested', position['quantity'] * entry_price)
                        entry_time = position.get('entry_time')
                        if isinstance(entry_time, str):
                            try:
                                entry_time = datetime.fromisoformat(entry_time.replace('Z', '+00:00'))
                            except:
                                entry_time = datetime.now()
                        elif not isinstance(entry_time, datetime):
                            entry_time = datetime.now()
                        holding_hours = (datetime.now() - entry_time).total_seconds() / 3600
                    
                    current_value = position['quantity'] * current_price
                    pnl = current_value - invested
                    pnl_percent = (pnl / invested * 100) if invested > 0 else 0
                    
                    positions_detail[symbol] = {
                        'quantity': position['quantity'],
                        'entry_price': entry_price,
                        'current_price': current_price,
                        'invested_value': invested,
                        'current_value': current_value,
                        'pnl': pnl,
                        'pnl_percent': pnl_percent,
                        'holding_hours': holding_hours,
                        'weight_in_portfolio': (current_value / self.bot.live_capital_manager.total_assets * 100) if self.bot.live_capital_manager.total_assets > 0 else 0,
                        'data_accuracy': '高' if performance_data and performance_data.get('is_accurate') else '估算'
                    }
                    
                    total_invested += invested
                    total_current_value += current_value
                    
            except Exception as e:
                print(f"❌ 获取持仓详情失败 {symbol}: {e}")
                continue
        
        return {
            'positions': positions_detail,
            'summary': {
                'total_invested': total_invested,
                'total_current_value': total_current_value,
                'total_pnl': total_current_value - total_invested,
                'total_pnl_percent': ((total_current_value - total_invested) / total_invested * 100) if total_invested > 0 else 0
            }
        }
    
    def _get_funds_breakdown(self):
        """获取资金细分"""
        try:
            total_assets = self.bot.live_capital_manager.total_assets
            usdt_balance = self.bot.live_capital_manager.usdt_balance
            trading_capital = self.bot.live_capital_manager.trading_capital
            allocated_capital = self.bot.live_capital_manager.allocated_capital
            
            non_trading_capital = total_assets - trading_capital
            available_trading = trading_capital - allocated_capital
            actual_available = min(available_trading, usdt_balance)
            
            return {
                'total_assets': total_assets,
                'usdt_balance': usdt_balance,
                'trading_capital': trading_capital,
                'allocated_capital': allocated_capital,
                'non_trading_capital': non_trading_capital,
                'available_trading_capital': available_trading,
                'actual_available_capital': actual_available,
                'locked_capital': allocated_capital,
                'free_capital': usdt_balance - actual_available
            }
        except Exception as e:
            print(f"❌ 计算资金细分失败: {e}")
            return {}
    
    def _calculate_buying_power(self, actual_available):
        """计算购买能力"""
        try:
            total_assets = self.bot.live_capital_manager.total_assets
            usdt_balance = self.bot.live_capital_manager.usdt_balance
            
            max_positions = self.bot.position_manager.get_max_positions(self.bot.current_market_state)
            current_positions = len(self.bot.position_manager.positions)
            available_slots = max(0, max_positions - current_positions)
            
            position_size = self.bot.position_manager.get_position_size(self.bot.current_market_state)
            
            actual_buying_power = min(actual_available, usdt_balance)
            
            return {
                'usdt_balance': usdt_balance,
                'actual_available_capital': actual_available,
                'position_size': position_size,
                'max_positions': max_positions,
                'current_positions': current_positions,
                'available_slots': available_slots,
                'max_new_investment': position_size * available_slots,
                'actual_buying_power': actual_buying_power,
                'can_buy_more': available_slots > 0 and actual_buying_power >= 5
            }
            
        except Exception as e:
            print(f"❌ 计算购买能力失败: {e}")
            return {
                'usdt_balance': 0,
                'actual_available_capital': 0,
                'position_size': 0,
                'max_positions': 0,
                'current_positions': 0,
                'available_slots': 0,
                'max_new_investment': 0,
                'actual_buying_power': 0,
                'can_buy_more': False
            }
    
    def _calculate_risk_metrics(self):
        """计算风险指标"""
        try:
            total_assets = self.bot.live_capital_manager.total_assets
            positions_detail = self._get_positions_detail()
            
            current_drawdown = 0
            if total_assets < self.initial_capital:
                current_drawdown = (self.initial_capital - total_assets) / self.initial_capital * 100
            
            position_values = [pos['current_value'] for pos in positions_detail['positions'].values()]
            if position_values:
                max_position = max(position_values)
                concentration = (max_position / sum(position_values) * 100) if sum(position_values) > 0 else 0
            else:
                concentration = 0
            
            winning_positions = sum(1 for pos in positions_detail['positions'].values() if pos['pnl'] > 0)
            losing_positions = sum(1 for pos in positions_detail['positions'].values() if pos['pnl'] < 0)
            win_rate = (winning_positions / len(positions_detail['positions']) * 100) if positions_detail['positions'] else 0
            
            return {
                'current_drawdown_percent': current_drawdown,
                'max_position_concentration': concentration,
                'win_rate': win_rate,
                'winning_positions': winning_positions,
                'losing_positions': losing_positions,
                'total_positions': len(positions_detail['positions']),
                'portfolio_volatility': self._calculate_portfolio_volatility()
            }
            
        except Exception as e:
            print(f"❌ 计算风险指标失败: {e}")
            return {
                'current_drawdown_percent': 0,
                'max_position_concentration': 0,
                'win_rate': 0,
                'winning_positions': 0,
                'losing_positions': 0,
                'total_positions': 0,
                'portfolio_volatility': 0
            }
    
    def _calculate_portfolio_volatility(self):
        """计算组合波动率"""
        try:
            if len(self.performance_history) < 2:
                return 0
            
            returns = []
            for i in range(1, len(self.performance_history)):
                prev = self.performance_history[i-1]['total_assets']
                curr = self.performance_history[i]['total_assets']
                if prev > 0:
                    returns.append((curr - prev) / prev)
            
            if returns:
                return np.std(returns) * 100
            return 0
            
        except Exception as e:
            print(f"❌ 计算组合波动率失败: {e}")
            return 0
    
    def _get_empty_report(self):
        """获取空报告"""
        return {
            'timestamp': datetime.now(),
            'total_assets': 0,
            'usdt_balance': 0,
            'trading_capital': 0,
            'allocated_capital': 0,
            'available_trading_capital': 0,
            'actual_available_capital': 0,
            'capital_usage_rate': 0,
            'positions_count': 0,
            'total_pnl': 0,
            'total_pnl_percent': 0,
            'daily_pnl': 0,
            'daily_pnl_percent': 0,
            'initial_capital': self.initial_capital,
            'positions_detail': {'positions': {}, 'summary': {'total_invested': 0, 'total_current_value': 0, 'total_pnl': 0, 'total_pnl_percent': 0}},
            'buying_power': {'usdt_balance': 0, 'actual_available_capital': 0, 'position_size': 0, 'max_positions': 0, 'current_positions': 0, 'available_slots': 0, 'max_new_investment': 0, 'actual_buying_power': 0, 'can_buy_more': False},
            'risk_metrics': {'current_drawdown_percent': 0, 'max_position_concentration': 0, 'win_rate': 0, 'winning_positions': 0, 'losing_positions': 0, 'total_positions': 0, 'portfolio_volatility': 0},
            'funds_breakdown': {}
        }
    
    def print_detailed_asset_report(self):
        """打印详细资产报告"""
        report = self.generate_complete_asset_report()
        
        print("\n" + "="*80)
        print("📊 完整资产统计报告")
        print("="*80)
        
        # 资金概况
        funds = report['funds_breakdown']
        print(f"💰 总资产: ${report['total_assets']:.2f}")
        print(f"💵 USDT可用余额: ${report['usdt_balance']:.2f}")
        print(f"🎯 交易资金: ${report['trading_capital']:.2f} (总资产的{self.bot.live_capital_manager.capital_ratio*100:.0f}%)")
        print(f"📊 资金分配:")
        print(f" - 已分配交易资金: ${report['allocated_capital']:.2f}")
        print(f" - 可用交易资金: ${report['available_trading_capital']:.2f}")
        print(f" - 实际可操作: ${report['actual_available_capital']:.2f}")
        print(f" - 非交易资金: ${funds.get('non_trading_capital', 0):.2f}")
        print(f" - 自由资金: ${funds.get('free_capital', 0):.2f}")
        
        # 盈亏概况
        print(f"\n📈 盈亏概况:")
        print(f" 初始资金: ${report['initial_capital']:.2f}")
        print(f" 总盈亏: ${report['total_pnl']:+.2f} ({report['total_pnl_percent']:+.2f}%)")
        print(f" 当日盈亏: ${report['daily_pnl']:+.2f} ({report['daily_pnl_percent']:+.2f}%)")
        print(f" 资金使用率: {report['capital_usage_rate']:.1f}%")
        
        # 购买能力
        buying_power = report['buying_power']
        print(f"\n🛒 购买能力分析:")
        print(f" 可买仓位: {buying_power['available_slots']}/{buying_power['max_positions']}")
        print(f" 单个仓位: ${buying_power['position_size']:.2f}")
        print(f" 最大新投资: ${buying_power['max_new_investment']:.2f}")
        print(f" 实际购买力: ${buying_power['actual_buying_power']:.2f}")
        print(f" 是否可以买入: {'✅ 是' if buying_power['can_buy_more'] else '❌ 否'}")
        
        # 风险指标
        risk_metrics = report['risk_metrics']
        print(f"\n⚠️ 风险指标:")
        print(f" 当前回撤: {risk_metrics['current_drawdown_percent']:.2f}%")
        print(f" 最大持仓集中度: {risk_metrics['max_position_concentration']:.2f}%")
        print(f" 胜率: {risk_metrics['win_rate']:.1f}%")
        
        # 持仓详情
        positions_detail = report['positions_detail']
        if positions_detail['positions']:
            print(f"\n📋 持仓详情 ({len(positions_detail['positions'])}个):")
            print(" " + "-"*90)
            print(" 币种 数量 成本价 现价 投资额 当前价值 盈亏 权重 数据精度")
            print(" " + "-"*90)
            
            for symbol, position in positions_detail['positions'].items():
                print(f" {symbol:10} {position['quantity']:8.4f} ${position['entry_price']:7.4f} "
                      f"${position['current_price']:7.4f} ${position['invested_value']:6.2f} "
                      f"${position['current_value']:6.2f} {position['pnl_percent']:+.2f}% "
                      f"{position['weight_in_portfolio']:.1f}% {position['data_accuracy']}")
            
            summary = positions_detail['summary']
            print(" " + "-"*90)
            print(f" 持仓汇总: ${summary['total_invested']:.2f} → ${summary['total_current_value']:.2f} "
                  f"({summary['total_pnl_percent']:+.2f}%)")
        else:
            print(f"\n📋 持仓详情: 无持仓")
        
        print("="*80 + "\n")
        
        # 保存到性能历史
        self.performance_history.append(report)
        if len(self.performance_history) > 1000:
            self.performance_history.pop(0)

# ========== 账户详情管理器 - 新增类 ==========
class AccountDetailManager:
    """账户详情管理器 - 统一管理账户信息输出"""
    
    def __init__(self, bot):
        self.bot = bot
        self.last_detailed_report = None
        self.detailed_report_interval = 1800
        self.last_quick_update = None
        self.quick_update_interval = 300
    
    def should_print_detailed_report(self):
        """检查是否需要打印详细报告"""
        if self.last_detailed_report is None:
            return True
        return (datetime.now() - self.last_detailed_report).total_seconds() > self.detailed_report_interval
    
    def should_print_quick_update(self):
        """检查是否需要打印快速更新"""
        if self.last_quick_update is None:
            return True
        return (datetime.now() - self.last_quick_update).total_seconds() > self.quick_update_interval
    
    def print_transaction_summary(self, transaction_type, symbol=None, amount=None, price=None):
        """打印交易摘要"""
        print(f"\n{'='*60}")
        print(f"🔄 {transaction_type}交易摘要 - {datetime.now().strftime('%H:%M:%S')}")
        print(f"{'='*60}")
        
        if symbol and amount and price:
            print(f" 币种: {symbol}")
            print(f" 价格: ${price:.6f}")
            print(f" 金额: ${amount:.2f}")
        
        self.print_immediate_funds_snapshot()
    
    def print_immediate_funds_snapshot(self):
        """打印即时资金快照"""
        try:
            total_assets = self.bot.calculate_total_assets()
            usdt_balance = self.bot.live_capital_manager.usdt_balance
            trading_capital = self.bot.live_capital_manager.trading_capital
            allocated_capital = self.bot.live_capital_manager.allocated_capital
            available_trading = self.bot.live_capital_manager.get_available_trading_capital()
            
            initial_capital = self.bot.asset_stats_manager.initial_capital
            total_pnl = total_assets - initial_capital
            total_pnl_percent = (total_pnl / initial_capital * 100) if initial_capital > 0 else 0
            
            print(f"💰 资金快照:")
            print(f" 总资产: ${total_assets:.2f}")
            print(f" USDT余额: ${usdt_balance:.2f}")
            print(f" 交易资金: ${trading_capital:.2f}")
            print(f" 已分配资金: ${allocated_capital:.2f}")
            print(f" 可用交易资金: ${available_trading:.2f}")
            print(f" 总盈亏: ${total_pnl:+.2f} ({total_pnl_percent:+.2f}%)")
            print(f" 持仓数量: {len(self.bot.position_manager.positions)}个")
            
            buying_power = self.bot.asset_stats_manager._calculate_buying_power(available_trading)
            if buying_power['can_buy_more']:
                print(f"🛒 可买入: {buying_power['available_slots']}仓位, "
                      f"${buying_power['actual_buying_power']:.2f}可用")
            else:
                print(f"⏹️ 暂无购买能力")
                
            print(f"{'='*60}\n")
            
        except Exception as e:
            print(f"❌ 打印资金快照失败: {e}")
    
    def print_detailed_account_report(self, context="定期报告"):
        """打印详细账户报告"""
        try:
            print(f"\n{'='*80}")
            print(f"📊 详细账户报告 - {context} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"{'='*80}")
            
            self.bot.asset_stats_manager.print_detailed_asset_report()
            
            self.last_detailed_report = datetime.now()
            self.last_quick_update = datetime.now()
            
        except Exception as e:
            print(f"❌ 打印详细账户报告失败: {e}")
    
    def print_quick_status_update(self):
        """打印快速状态更新"""
        try:
            report = self.bot.asset_stats_manager.generate_complete_asset_report()
            
            print(f"📈 状态更新 [{datetime.now().strftime('%H:%M')}]: "
                  f"总资产${report['total_assets']:.2f} | "
                  f"USDT${report['usdt_balance']:.2f} | "
                  f"持仓{report['positions_count']} | "
                  f"盈亏{report['total_pnl_percent']:+.2f}%")
            
            if report['positions_count'] > 0:
                positions_summary = report['positions_detail']['summary']
                print(f" 📦 持仓: ${positions_summary['total_invested']:.2f} → "
                      f"${positions_summary['total_current_value']:.2f} "
                      f"({positions_summary['total_pnl_percent']:+.2f}%)")
            
            buying_power = report['buying_power']
            if buying_power['can_buy_more']:
                print(f" 🛒 可买: {buying_power['available_slots']}仓位, "
                      f"${buying_power['actual_buying_power']:.2f}可用")
            
            self.last_quick_update = datetime.now()
            
        except Exception as e:
            print(f"❌ 打印快速状态更新失败: {e}")

# ========== 紧急停止管理器 ==========
class EmergencyStopManager:
    """紧急停止管理器"""
    
    def __init__(self, bot):
        self.bot = bot
        self.max_daily_loss = 0.05
        self.max_single_loss = 0.10
        self.daily_pnl = 0
        self.last_reset = datetime.now().date()
        self.emergency_stopped = False
    
    def check_emergency_stop(self):
        """检查紧急停止条件"""
        if self.emergency_stopped:
            return True
            
        if datetime.now().date() != self.last_reset:
            self.daily_pnl = 0
            self.last_reset = datetime.now().date()
        
        try:
            total_assets = self.bot.calculate_total_assets()
            initial_capital = self.bot.live_capital_manager.total_assets
            
            if initial_capital > 0:
                total_pnl = total_assets - initial_capital
                daily_loss = -total_pnl / initial_capital
                
                if daily_loss > self.max_daily_loss:
                    print(f"🚨 触发单日亏损止损: 亏损{daily_loss:.2%}")
                    self.emergency_stop("单日亏损超过阈值")
                    return True
            
            for symbol, position in self.bot.position_manager.positions.items():
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price:
                    loss_pct = (position['entry_price'] - current_price) / position['entry_price']
                    if loss_pct > self.max_single_loss:
                        print(f"🚨 单笔巨大亏损: {symbol} 亏损{loss_pct:.2%}")
                        self.emergency_stop(f"单笔亏损{symbol}")
                        return True
            
            return False
            
        except Exception as e:
            print(f"❌ 紧急停止检查失败: {e}")
            return False
    
    def emergency_stop(self, reason):
        """执行紧急停止"""
        print(f"🚨 执行紧急停止: {reason}")
        self.emergency_stopped = True
        self.bot.is_running = False
        
        for symbol, position in list(self.bot.position_manager.positions.items()):
            try:
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price:
                    sell_order = self.bot.order_manager.create_sell_order(
                        symbol, position['quantity'], current_price,
                        self.bot.position_manager, self.bot.live_capital_manager
                    )
                    if sell_order:
                        print(f"✅ 紧急平仓: {symbol}")
            except Exception as e:
                print(f"❌ 紧急平仓失败 {symbol}: {e}")

# ========== 事务性资金管理器 ==========
class TransactionalFundManager:
    """事务性资金管理器"""
    
    def __init__(self, position_manager):
        self.position_manager = position_manager
        self.pending_transactions = {}
        self.transaction_lock = threading.Lock()
    
    def reserve_funds(self, symbol, amount):
        """预留资金"""
        with self.transaction_lock:
            if self.position_manager.available_balance >= amount:
                self.pending_transactions[symbol] = amount
                self.position_manager.available_balance -= amount
                return True
            return False
    
    def commit_transaction(self, symbol, success=True):
        """提交或回滚交易"""
        with self.transaction_lock:
            if symbol in self.pending_transactions:
                amount = self.pending_transactions[symbol]
                if not success:
                    self.position_manager.available_balance += amount
                del self.pending_transactions[symbol]

# ========== 分级错误处理器 ==========
class HierarchicalErrorHandler:
    """分级错误处理器"""
    
    def __init__(self, bot):
        self.bot = bot
        self.error_counts = {
            'api_error': 0,
            'network_error': 0, 
            'data_error': 0,
            'logic_error': 0
        }
        self.last_error_time = {}
    
    def handle_error(self, error_type, error, context=""):
        """处理错误"""
        self.error_counts[error_type] += 1
        self.last_error_time[error_type] = datetime.now()
        
        print(f"❌ [{error_type}] {context}: {error}")
        
        if self.error_counts[error_type] > 10:
            self._enter_safe_mode(error_type)
        elif self.error_counts[error_type] > 5:
            self._reduce_trading_activity(error_type)
    
    def _enter_safe_mode(self, error_type):
        """进入安全模式"""
        print(f"🚨 进入安全模式，错误类型: {error_type}")
        self.bot.is_running = False
        
        self.bot.state_manager.save_complete_state(self.bot)
        
        recovery_time = 600
        print(f"⏳ 等待{recovery_time}秒后恢复...")
        time.sleep(recovery_time)
        
        self.error_counts[error_type] = 0
        self.bot.is_running = True
        print("🟢 安全模式解除")
    
    def _reduce_trading_activity(self, error_type):
        """减少交易活动"""
        print(f"⚠️ 减少交易活动，错误类型: {error_type}")

# ========== 性能监控器 ==========
class PerformanceMonitor:
    """性能监控器"""
    
    def __init__(self):
        self.metrics = {
            'api_call_duration': [],
            'memory_usage': [],
            'cycle_execution_time': [],
            'error_rates': []
        }
        self.start_time = datetime.now()
    
    def record_api_call(self, duration):
        """记录API调用时间"""
        self.metrics['api_call_duration'].append(duration)
        if len(self.metrics['api_call_duration']) > 100:
            self.metrics['api_call_duration'].pop(0)
    
    def check_performance_health(self):
        """检查性能健康度"""
        try:
            memory_usage = psutil.virtual_memory().percent
            self.metrics['memory_usage'].append(memory_usage)
            
            if self.metrics['api_call_duration']:
                avg_api_time = np.mean(self.metrics['api_call_duration'])
                if avg_api_time > 2.0:
                    return "API_PERFORMANCE_DEGRADED"
            
            if memory_usage > 85:
                return "HIGH_MEMORY_USAGE"
            
            return "HEALTHY"
            
        except Exception as e:
            print(f"❌ 性能监控失败: {e}")
            return "MONITOR_ERROR"
    
    def generate_performance_report(self):
        """生成性能报告"""
        report = {
            'uptime_hours': (datetime.now() - self.start_time).total_seconds() / 3600,
            'memory_usage_percent': psutil.virtual_memory().percent,
            'cpu_usage_percent': psutil.cpu_percent(),
            'api_call_count': len(self.metrics['api_call_duration'])
        }
        
        if self.metrics['api_call_duration']:
            report['avg_api_time'] = np.mean(self.metrics['api_call_duration'])
            report['max_api_time'] = np.max(self.metrics['api_call_duration'])
        
        return report

# ========== 智能多级止盈系统 ==========
class SmartMultiLevelTakeProfit:
    """智能多级止盈系统"""
    
    def __init__(self):
        self.config = OptimizedLongRunConfig()
        self.activated_levels = {} # 记录每个持仓的激活级别
    
    def check_take_profit(self, symbol, entry_price, current_price, quantity, volatility):
        """检查多级止盈条件"""
        current_profit_pct = (current_price - entry_price) / entry_price
        
        # 根据波动率动态调整止盈级别
        adjusted_levels = self._adjust_levels_by_volatility(volatility)
        
        for i, level in enumerate(adjusted_levels):
            level_key = f"{symbol}_level_{i}"
            
            # 如果达到盈利目标且该级别未激活
            if (current_profit_pct >= level['profit_pct'] and 
                not self.activated_levels.get(level_key, False)):
                
                self.activated_levels[level_key] = True
                
                # 计算卖出数量
                sell_quantity = quantity * level['sell_ratio']
                
                return {
                    'action': 'PARTIAL_TAKE_PROFIT',
                    'symbol': symbol,
                    'quantity': sell_quantity,
                    'price': current_price,
                    'reason': f"第{i+1}级止盈: 盈利{current_profit_pct:.2%} ≥ {level['profit_pct']:.0%}",
                    'level': i + 1,
                    'profit_pct': current_profit_pct
                }
        
        return None
    
    def _adjust_levels_by_volatility(self, volatility):
        """根据波动率动态调整止盈级别"""
        adjusted_levels = []
        
        for level in self.config.MULTI_LEVEL_TAKE_PROFIT:
            adjusted_level = level.copy()
            
            # 高波动性市场提高止盈目标
            if volatility > 0.08: # 高波动
                adjusted_level['profit_pct'] *= 1.2
            elif volatility < 0.03: # 低波动
                adjusted_level['profit_pct'] *= 0.8
            
            adjusted_levels.append(adjusted_level)
        
        return adjusted_levels

# ========== 智能多级止损系统 ==========
class SmartMultiLevelStopLoss:
    """智能多级止损系统"""
    
    def __init__(self):
        self.config = OptimizedLongRunConfig()
        self.breach_history = {} # 记录每个持仓的止损触发历史
    
    def check_stop_loss(self, symbol, entry_price, current_price, quantity, market_condition, volume_data):
        """检查多级止损条件"""
        current_loss_pct = (current_price - entry_price) / entry_price
        
        # 如果是熊市，使用更严格的止损
        adjusted_levels = self._adjust_levels_by_market(market_condition)
        
        for i, level in enumerate(adjusted_levels):
            level_key = f"{symbol}_stop_{i}"
            
            # 如果达到止损目标
            if current_loss_pct <= level['loss_pct']:
                
                # 检查附加条件
                if self._check_stop_condition(level, volume_data):
                    
                    # 避免重复触发同一级别
                    if not self.breach_history.get(level_key, False):
                        self.breach_history[level_key] = True
                        
                        sell_quantity = quantity * level['sell_ratio']
                        
                        return {
                            'action': 'PARTIAL_STOP_LOSS',
                            'symbol': symbol,
                            'quantity': sell_quantity,
                            'price': current_price,
                            'reason': f"第{i+1}级止损: 亏损{abs(current_loss_pct):.2%} ≥ {abs(level['loss_pct']):.0%}",
                            'level': i + 1,
                            'loss_pct': current_loss_pct
                        }
        
        return None
    
    def _adjust_levels_by_market(self, market_condition):
        """根据市场状态调整止损级别"""
        adjusted_levels = []
        
        for level in self.config.MULTI_LEVEL_STOP_LOSS:
            adjusted_level = level.copy()
            
            if market_condition == 'bear':
                # 熊市使用更严格的止损
                adjusted_level['loss_pct'] *= 0.7 # 收紧30%
            elif market_condition == 'bull':
                # 牛市可以容忍更大回撤
                adjusted_level['loss_pct'] *= 1.3 # 放宽30%
            
            adjusted_levels.append(adjusted_level)
        
        return adjusted_levels
    
    def _check_stop_condition(self, level, volume_data):
        """检查止损附加条件"""
        if level['condition'] == 'immediate':
            return True
        elif level['condition'] == 'volume_spike':
            # 只有出现放量下跌时才止损
            current_volume = volume_data.get('current_volume', 0)
            avg_volume = volume_data.get('avg_volume', 1)
            return current_volume / avg_volume > 1.5 # 成交量放大50%
        
        return False

# ========== 智能移动止损管理器 ==========
class SmartTrailingStopManager:
    """智能移动止损管理器"""
    
    def __init__(self):
        self.config = OptimizedLongRunConfig()
        self.peak_prices = {} # 记录每个持仓的最高价
    
    def check_trailing_stop(self, symbol, position, current_price, current_pnl):
        """检查移动止损"""
        if symbol not in self.peak_prices or current_price > self.peak_prices[symbol]:
            self.peak_prices[symbol] = current_price
        
        peak_price = self.peak_prices[symbol]
        
        # 根据波动率选择移动止损策略
        strategy = self._select_trailing_strategy(position.get('volatility', 0.05))
        trailing_config = self.config.TRAILING_STOP_CONFIG[strategy]
        
        # 检查是否达到激活条件
        if current_pnl >= trailing_config['activation']:
            trail_price = peak_price * (1 - trailing_config['trail_percent'])
            
            if current_price <= trail_price:
                return {
                    'action': 'TRAILING_STOP',
                    'symbol': symbol,
                    'quantity': position['quantity'],
                    'price': current_price,
                    'reason': f'移动止损: 从高点回撤{trailing_config["trail_percent"]:.1%}',
                    'strategy': strategy
                }
        
        return None
    
    def _select_trailing_strategy(self, volatility):
        """根据波动率选择移动止损策略"""
        if volatility > 0.08:
            return 'conservative' # 高波动使用保守策略
        elif volatility < 0.03:
            return 'aggressive' # 低波动使用激进策略
        else:
            return 'moderate' # 中等波动使用适中策略

# ========== 全仓风险控制系统 ==========
class PortfolioRiskController:
    """全仓风险控制系统"""
    
    def __init__(self, bot):
        self.bot = bot
        self.config = OptimizedLongRunConfig()
        self.portfolio_high_watermark = 0 # 组合最高水位线
    
    def check_portfolio_risk(self):
        """检查全仓风险"""
        total_assets = self.bot.calculate_total_assets()
        
        # 更新最高水位线
        if total_assets > self.portfolio_high_watermark:
            self.portfolio_high_watermark = total_assets
        
        # 计算当前回撤
        if self.portfolio_high_watermark > 0:
            current_drawdown = (self.portfolio_high_watermark - total_assets) / self.portfolio_high_watermark
        else:
            current_drawdown = 0
        
        # 检查回撤级别
        for level in self.config.PORTFOLIO_RISK_LEVELS:
            if current_drawdown >= level['drawdown_pct']:
                return self._execute_portfolio_action(level, current_drawdown)
        
        return {'action': 'NORMAL', 'drawdown': current_drawdown}
    
    def _execute_portfolio_action(self, level, drawdown):
        """执行全仓风险控制动作"""
        if level['action'] == 'reduce_risk':
            # 减少风险暴露
            self._reduce_risk_exposure(level['reduce_by'])
            return {
                'action': 'REDUCE_RISK',
                'reason': f'全仓回撤{drawdown:.2%} ≥ {level["drawdown_pct"]:.0%}',
                'reduce_by': level['reduce_by']
            }
        
        elif level['action'] == 'stop_trading':
            # 停止交易一段时间
            self.bot.is_running = False
            return {
                'action': 'STOP_TRADING',
                'reason': f'全仓回撤{drawdown:.2%} ≥ {level["drawdown_pct"]:.0%}',
                'duration_hours': level['duration_hours']
            }
        
        return {'action': 'NORMAL', 'drawdown': drawdown}
    
    def _reduce_risk_exposure(self, reduce_ratio):
        """减少风险暴露"""
        print(f"🚨 全仓风险控制: 减少{reduce_ratio:.0%}风险暴露")
        
        # 1. 减少仓位规模
        original_ratio = self.bot.live_capital_manager.capital_ratio
        new_ratio = original_ratio * (1 - reduce_ratio)
        self.bot.live_capital_manager.capital_ratio = new_ratio
        
        # 2. 清理表现最差的持仓
        self._liquidate_weak_positions(reduce_ratio)
        
        print(f"📉 交易资金比例: {original_ratio:.0%} → {new_ratio:.0%}")
    
    def _liquidate_weak_positions(self, reduce_ratio):
        """清理弱势持仓"""
        positions_to_sell = self._identify_weak_positions()
        sell_count = int(len(positions_to_sell) * reduce_ratio)
        
        for symbol in positions_to_sell[:sell_count]:
            try:
                position = self.bot.position_manager.positions[symbol]
                current_price = self.bot.data_collector.get_price(symbol)
                
                if current_price:
                    self.bot.order_manager.create_sell_order(
                        symbol, position['quantity'], current_price,
                        self.bot.position_manager, self.bot.live_capital_manager
                    )
                    print(f"🗑️ 风险控制卖出: {symbol}")
            except Exception as e:
                print(f"❌ 风险控制卖出失败 {symbol}: {e}")
    
    def _identify_weak_positions(self):
        """识别弱势持仓"""
        weak_positions = []
        
        for symbol, position in self.bot.position_manager.positions.items():
            try:
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price:
                    pnl = (current_price - position['entry_price']) / position['entry_price']
                    holding_hours = self.bot.holding_manager.get_holding_hours(symbol)
                    
                    # 评分标准: 亏损 + 持有时间长 = 弱势
                    weakness_score = abs(min(0, pnl)) + (holding_hours / 240) # 10天为基准
                    weak_positions.append((symbol, weakness_score))
            except:
                continue
        
        # 按弱势程度排序
        weak_positions.sort(key=lambda x: x[1], reverse=True)
        return [item[0] for item in weak_positions]

# ========== 增强型动态止损风险管理器 ==========
class EnhancedDynamicRiskManager:
    """增强型动态止损风险经理 - 集成多级止盈止损"""
    
    def __init__(self, data_collector):
        self.data_collector = data_collector
        self.take_profit_system = SmartMultiLevelTakeProfit()
        self.stop_loss_system = SmartMultiLevelStopLoss()
        self.trailing_stop_manager = SmartTrailingStopManager()
        self.volatility_cache = {}
        
    def check_enhanced_risk(self, symbol, position, market_state, holding_hours, current_price):
        """增强风险检查 - 集成多级止盈止损"""
        entry_price = position['entry_price']
        current_pnl = (current_price - entry_price) / entry_price
        
        # 获取波动率数据
        volatility = self._calculate_volatility(symbol)
        
        # 获取成交量数据
        volume_data = self._get_volume_data(symbol)
        
        # 1. 首先检查多级止盈
        take_profit_signal = self.take_profit_system.check_take_profit(
            symbol, entry_price, current_price, position['quantity'], volatility
        )
        
        if take_profit_signal:
            return take_profit_signal
        
        # 2. 检查多级止损
        stop_loss_signal = self.stop_loss_system.check_stop_loss(
            symbol, entry_price, current_price, position['quantity'], market_state, volume_data
        )
        
        if stop_loss_signal:
            return stop_loss_signal
        
        # 3. 检查移动止损
        trailing_signal = self.trailing_stop_manager.check_trailing_stop(
            symbol, position, current_price, current_pnl
        )
        
        if trailing_signal:
            return trailing_signal
        
        # 4. 检查时间止损 (防止长期横盘)
        time_stop_signal = self._check_time_based_stop(holding_hours, current_pnl)
        if time_stop_signal:
            time_stop_signal['symbol'] = symbol
            time_stop_signal['quantity'] = position['quantity']
            time_stop_signal['price'] = current_price
            return time_stop_signal
        
        return {
            'action': 'HOLD',
            'reason': f'继续持有: 盈亏{current_pnl:+.2%}'
        }
    
    def _check_time_based_stop(self, holding_hours, current_pnl):
        """时间止损策略"""
        # 持有超过7天且仍然亏损，考虑止损
        if holding_hours > 168 and current_pnl < -0.05: # 7天 * 24小时
            return {
                'action': 'TIME_STOP_LOSS',
                'reason': f'时间止损: 持有{holding_hours:.1f}小时仍亏损{abs(current_pnl):.2%}'
            }
        
        # 持有超过14天且盈利微薄，止盈离场
        if holding_hours > 336 and current_pnl < 0.03: # 14天
            return {
                'action': 'TIME_TAKE_PROFIT', 
                'reason': f'时间止盈: 持有{holding_hours:.1f}小时盈利仅{current_pnl:.2%}'
            }
        
        return None
    
    def _calculate_volatility(self, symbol):
        """计算波动率"""
        try:
            if symbol in self.volatility_cache:
                cached_data = self.volatility_cache[symbol]
                if time.time() - cached_data['timestamp'] < 3600: # 缓存1小时
                    return cached_data['volatility']
            
            data = self.data_collector.get_klines(symbol, '1d', 20)
            if data is None or len(data) < 20:
                return 0.05 # 默认波动率
            
            returns = data['close'].pct_change().dropna()
            volatility = returns.std()
            
            # 缓存结果
            self.volatility_cache[symbol] = {
                'volatility': volatility,
                'timestamp': time.time()
            }
            
            return volatility
            
        except Exception as e:
            print(f"❌ 计算波动率失败 {symbol}: {e}")
            return 0.05
    
    def _get_volume_data(self, symbol):
        """获取成交量数据"""
        try:
            data = self.data_collector.get_klines(symbol, '1h', 24)
            if data is None or len(data) < 24:
                return {'current_volume': 0, 'avg_volume': 1}
            
            current_volume = data['volume'].iloc[-1]
            avg_volume = data['volume'].mean()
            
            return {
                'current_volume': current_volume,
                'avg_volume': avg_volume
            }
            
        except Exception as e:
            print(f"❌ 获取成交量数据失败 {symbol}: {e}")
            return {'current_volume': 0, 'avg_volume': 1}

# ========== 智能持仓管理器 ==========
class SmartPositionManager:
    """智能持仓管理器 - 只计算程序管理的持仓"""
    
    def __init__(self, initial_balance=1000):
        self.positions = {}
        self.trade_history = []
        
        # 修正资金变量定义
        self.available_balance = initial_balance
        self.allocated_balance = 0
        
        self.config = OptimizedLongRunConfig()
        self.current_market_state = "consolidation"
        self.price_getter = None
        
        # 修改：添加打印控制
        self.last_sync_print = None
        self.sync_print_interval = 1800 # 30分钟打印一次
        
    def set_price_getter(self, price_getter_func):
        """设置价格获取函数"""
        self.price_getter = price_getter_func
    
    def get_program_managed_positions_count(self):
        """获取程序管理的持仓数量（排除恢复失败的孤儿持仓）"""
        # 只计算非恢复状态的持仓，或者所有持仓（因为恢复失败的不会添加）
        return len(self.positions)
    
    def get_effective_positions_count(self, bot):
        """获取有效持仓数量（程序管理 + 成功恢复的孤儿持仓）"""
        # 这里可以添加逻辑来区分哪些是程序管理的，哪些是恢复的
        # 目前简单返回所有持仓数量
        return len(self.positions)
    
    def enforce_smart_position_limits(self, market_state, bot):
        """智能执行持仓限制 - 只管理程序能识别的持仓"""
        try:
            # 只计算程序管理的持仓
            program_positions_count = self.get_program_managed_positions_count()
            max_positions = self.get_max_positions(market_state)
            
            print(f"🔍 智能持仓检查:")
            print(f" 程序管理持仓: {program_positions_count}个")
            print(f" 市场状态: {market_state}")
            print(f" 最大允许持仓: {max_positions}个")
            
            if program_positions_count <= max_positions:
                print("✅ 程序管理持仓数量符合限制")
                return True
                
            # 需要清理的程序持仓数量
            positions_to_sell = program_positions_count - max_positions
            print(f"🚨 需要清理 {positions_to_sell} 个程序管理持仓")
            
            # 只清理程序管理的持仓
            positions_to_sell_list = self._select_program_positions_to_sell(positions_to_sell, bot)
            
            # 执行卖出
            sold_count = 0
            for symbol in positions_to_sell_list:
                if self._sell_program_position(symbol, bot):
                    sold_count += 1
            
            final_count = self.get_program_managed_positions_count()
            print(f"📊 清理完成: 卖出{sold_count}个持仓，剩余程序管理持仓{final_count}个")
            
            return final_count <= max_positions
            
        except Exception as e:
            print(f"❌ 智能持仓限制执行失败: {e}")
            return False
    
    def _select_program_positions_to_sell(self, count, bot):
        """选择要卖出的程序管理持仓 - 添加持仓时间保护"""
        # 只从程序管理的持仓中选择
        program_positions = []
        
        for symbol, position in self.positions.items():
            try:
                current_price = bot.data_collector.get_price(symbol)
                if not current_price:
                    continue
                    
                entry_price = position['entry_price']
                pnl_percent = (current_price - entry_price) / entry_price
                holding_hours = bot.holding_manager.get_holding_hours(symbol)
                
                # 🛡️ 关键修改：检查是否达到最短持仓时间
                min_holding_hours = bot.holding_manager.min_holding_hours
                if holding_hours < min_holding_hours:
                    # 未达到最短持仓时间，给予保护分数
                    protection_score = -100 # 负分数，不会被选中
                    print(f"🛡️ {symbol} 持仓保护: 仅持有{holding_hours:.1f}小时 (<{min_holding_hours}小时)")
                    program_positions.append((symbol, protection_score, pnl_percent, holding_hours))
                    continue
                
                # 计算卖出优先级分数
                score = self._calculate_sell_priority_score(pnl_percent, holding_hours, position)
                program_positions.append((symbol, score, pnl_percent, holding_hours))
                
            except Exception as e:
                print(f"❌ 计算持仓 {symbol} 分数失败: {e}")
                continue
        
        # 按分数排序，分数高的优先卖出
        program_positions.sort(key=lambda x: x[1], reverse=True)
        
        # 过滤掉被保护的持仓（负分数的）
        eligible_positions = [item for item in program_positions if item[1] >= 0]
        
        # 选择前count个
        selected = [item[0] for item in eligible_positions[:count]]
        
        print(f"📋 选择的程序持仓卖出列表: {selected}")
        for symbol, score, pnl, hours in eligible_positions[:count]:
            status = "亏损" if pnl < 0 else "盈利"
            print(f" 🗑️ {symbol}: 分数{score:.1f}, {status}{abs(pnl):.2%}, 持有{hours:.1f}小时")
            
        return selected
    
    def _calculate_sell_priority_score(self, pnl_percent, holding_hours, position):
        """计算卖出优先级分数"""
        # 亏损的持仓优先级高
        base_score = abs(pnl_percent) * 100
        
        # 持有时间长的优先级高
        time_score = holding_hours / 24
        
        # 亏损的额外加分
        if pnl_percent < 0:
            base_score += 50
        
        # 恢复的持仓优先级较高（如果需要清理，先清理恢复的持仓）
        if position.get('recovered'):
            base_score += 30
        
        return base_score + time_score
    
    def _sell_program_position(self, symbol, bot):
        """卖出程序管理持仓"""
        try:
            if symbol not in self.positions:
                return False
            
            position = self.positions[symbol]
        
            # 🆕 使用统一的卖出方法
            return bot.safe_sell_and_check(symbol, position['quantity'], "程序持仓卖出")
            
        except Exception as e:
            print(f"❌ 卖出程序持仓 {symbol} 失败: {e}")
            return False
                
    
    def record_trade(self, symbol, amount, price, quantity, action, live_capital_manager=None):
        """记录交易 - 修正资金逻辑"""
        try:
            trade = {
                'symbol': symbol,
                'action': action,
                'amount': amount,
                'price': price,
                'quantity': quantity,
                'time': datetime.now(),
                'market_state': self.current_market_state
            }
            
            if action in ['BUY', 'REALTIME_BUY']:
                if symbol in self.positions:
                    position = self.positions[symbol]
                    total_quantity = position['quantity'] + quantity
                    total_cost = position['quantity'] * position['entry_price'] + amount
                    average_price = total_cost / total_quantity
                    
                    position['quantity'] = total_quantity
                    position['entry_price'] = average_price
                    position['total_invested'] += amount
                else:
                    self.positions[symbol] = {
                        'symbol': symbol,
                        'quantity': quantity,
                        'entry_price': price,
                        'total_invested': amount,
                        'entry_time': datetime.now(),
                        'original_entry_time': datetime.now(), # 🆕 保存原始时间
                        'original_entry_price': price # 🆕 保存原始价格
                    }
                
                # 修正资金转移逻辑
                self.available_balance -= amount
                self.allocated_balance += amount
                
                print(f"✅ 记录买入: {symbol}, 数量: {quantity:.6f}, "
                      f"价格: ${price:.6f}, 金额: ${amount:.2f}")
                print(f" 资金变化: 可用${self.available_balance:.2f}, "
                      f"已分配${self.allocated_balance:.2f}")
                
                # 🆕 立即保存持仓数据
                if live_capital_manager and hasattr(live_capital_manager, 'bot'):
                    live_capital_manager.bot.persistence_manager.save_position_details()
                
                # 买入后资金重计算
                if live_capital_manager:
                    live_capital_manager.recalculate_after_buy(symbol, amount)
                
            elif action in ['SELL', 'PARTIAL_TAKE_PROFIT']:
                if symbol in self.positions:
                    position = self.positions[symbol]
                    
                    entry_price = position['entry_price']
                    pnl = (price - entry_price) / entry_price
                    pnl_amount = (price - entry_price) * quantity
                    
                    # 区分卖出金额和卖出成本
                    sell_amount = quantity * price
                    sold_cost = quantity * entry_price
                    
                    position['quantity'] -= quantity
                    position['total_invested'] -= sold_cost
                    
                    if position['quantity'] <= 0:
                        del self.positions[symbol]
                    
                    trade['pnl_pct'] = pnl
                    trade['pnl_amount'] = pnl_amount
                    
                    # 修正资金转移逻辑
                    self.available_balance += sell_amount
                    self.allocated_balance -= sold_cost
                    
                    print(f"✅ 记录卖出: {symbol}, 数量: {quantity:.6f}")
                    print(f" 卖出价格: ${price:.6f}, 成本价格: ${entry_price:.6f}")
                    print(f" 实际收到: ${sell_amount:.2f}, 成本收回: ${sold_cost:.2f}")
                    print(f" 实现盈亏: ${pnl_amount:+.2f} ({pnl:+.2%})")
                    print(f" 资金变化: 可用${self.available_balance:.2f}, "
                          f"已分配${self.allocated_balance:.2f}")
                    
                    # 🆕 立即保存持仓数据
                    if live_capital_manager and hasattr(live_capital_manager, 'bot'):
                        live_capital_manager.bot.persistence_manager.save_position_details()
                    
                    # 卖出后资金重计算
                    if live_capital_manager:
                        live_capital_manager.recalculate_after_sell(
                            symbol, sell_amount, quantity, sold_cost
                        )
                else:
                    print(f"❌ 卖出交易错误: 未找到持仓 {symbol}")
                    return
            
            self.trade_history.append(trade)
            
        except Exception as e:
            print(f"❌ 记录交易失败: {e}")
    
    def get_available_funds(self):
        """获取可用资金"""
        return max(0, self.available_balance)
    
    def get_allocated_funds(self):
        """获取已分配资金"""
        return self.allocated_balance
    
    def get_position_size(self, market_state):
        """获取单个仓位大小"""
        config = self.config.POSITION_CONFIG[market_state]
        position_size_ratio = config["position_size"]
        return self.available_balance * position_size_ratio
    
    def get_max_positions(self, market_state):
        """获取最大持仓数"""
        return self.config.POSITION_CONFIG[market_state]["max_positions"]
    
    def update_allocated_capital(self, live_capital_manager):
        """更新已分配资金 - 与资金管理器同步，减少打印频率"""
        try:
            current_time = datetime.now()
            
            # 控制打印频率
            should_print = (self.last_sync_print is None or 
                          (current_time - self.last_sync_print).total_seconds() >= self.sync_print_interval)
            
            current_positions_value = 0
            for symbol, position in self.positions.items():
                if self.price_getter:
                    current_price = self.price_getter(symbol)
                    if current_price:
                        current_positions_value += position['quantity'] * current_price
            
            live_capital_manager.allocated_capital = current_positions_value
            
            if should_print:
                print(f"🔄 同步已分配资金: ${current_positions_value:.2f}")
                self.last_sync_print = current_time
            
        except Exception as e:
            print(f"❌ 更新已分配资金失败: {e}")
    
    def check_overall_stop_loss(self):
        """检查整体止损"""
        try:
            total_invested = sum([pos['total_invested'] for pos in self.positions.values()])
            current_value = 0
            
            for symbol, position in self.positions.items():
                if self.price_getter:
                    current_price = self.price_getter(symbol)
                    if current_price:
                        current_value += position['quantity'] * current_price
            
            if total_invested > 0:
                drawdown = (current_value - total_invested) / total_invested
                if drawdown < -self.config.RISK_CONFIG["max_portfolio_drawdown"]:
                    return True, drawdown
            
            return False, 0
            
        except Exception as e:
            print(f"❌ 检查整体止损失败: {e}")
            return False, 0
    
    def execute_overall_stop_loss(self, data_collector):
        """执行整体止损"""
        print("🚨 执行整体止损")
        
        for symbol, position in list(self.positions.items()):
            try:
                current_price = data_collector.get_price(symbol)
                if current_price:
                    self.record_trade(
                        symbol=symbol,
                        amount=position['quantity'] * current_price,
                        price=current_price,
                        quantity=position['quantity'],
                        action='SELL'
                    )
                    print(f"✅ 整体止损: {symbol}")
                        
            except Exception as e:
                print(f"❌ 整体止损失败 {symbol}: {e}")
    
    def generate_portfolio_report(self):
        """生成组合报告"""
        try:
            total_invested = sum([pos['total_invested'] for pos in self.positions.values()])
            current_value = 0
            positions_detail = {}
            
            for symbol, position in self.positions.items():
                if self.price_getter:
                    current_price = self.price_getter(symbol)
                    if current_price:
                        position_value = position['quantity'] * current_price
                        current_value += position_value
                        entry_price = position['entry_price']
                        pnl = (current_price - entry_price) / entry_price
                        
                        positions_detail[symbol] = {
                            'quantity': position['quantity'],
                            'entry_price': entry_price,
                            'current_price': current_price,
                            'current_value': position_value,
                            'pnl_pct': pnl,
                            'holding_days': (datetime.now() - position['entry_time']).days
                        }
            
            self.available_balance = max(0, self.available_balance)
            
            total_pnl = current_value - total_invested
            total_pnl_pct = total_pnl / total_invested if total_invested > 0 else 0
            
            return {
                'timestamp': datetime.now(),
                'total_balance': self.available_balance + current_value,
                'available_balance': self.available_balance,
                'positions_value': current_value,
                'total_invested': total_invested,
                'total_pnl': total_pnl,
                'total_pnl_pct': total_pnl_pct,
                'positions_count': len(self.positions),
                'positions_detail': positions_detail,
                'drawdown': min(0, total_pnl_pct)
            }
            
        except Exception as e:
            print(f"❌ 生成组合报告失败: {e}")
            return {
                'timestamp': datetime.now(),
                'total_balance': self.available_balance,
                'available_balance': self.available_balance,
                'positions_value': 0,
                'total_invested': 0,
                'total_pnl': 0,
                'total_pnl_pct': 0,
                'positions_count': 0,
                'positions_detail': {},
                'drawdown': 0
            }

# ========== 增强订单管理器 - 增加稳定币过滤和增强精度处理 ==========
class OrderManager:
    """增强订单管理器 - 改进数量精度处理"""
    
    def __init__(self, data_collector):
        self.data_collector = data_collector
        self.min_investment = 5
        self.symbol_precision_cache = {} # 缓存币种精度信息
    
    def create_buy_order(self, symbol, quantity, price, position_manager=None, capital_manager=None):
        """创建买入订单 - 增强精度处理"""
        try:
            investment = quantity * price
            
            if investment < self.min_investment:
                print(f"💰 投资金额过小: ${investment:.2f} < ${self.min_investment}")
                return None
            
            if self.data_collector.client is None:
                print(f"❌ 客户端未连接，无法下单: {symbol}")
                return None
            
            # 🆕 增强精度处理
            adjusted_quantity, adjusted_investment = self._enhanced_adjust_quantity(symbol, quantity, price, 'BUY')
            
            if adjusted_quantity <= 0:
                print(f"❌ 精度调整后数量无效: {symbol}")
                return None
            
            if adjusted_investment < self.min_investment:
                print(f"💰 精度调整后投资金额过小: ${adjusted_investment:.2f} < ${self.min_investment}")
                return None
            
            print(f"🔄 创建买入订单: {symbol}, 数量: {adjusted_quantity:.8f}, 金额: ${adjusted_investment:.2f}")
            
            # 实际下单使用调整后的数量
            order = self.data_collector.client.order_market_buy(
                symbol=symbol,
                quantity=adjusted_quantity
            )
            
            print(f"✅ 买入订单成功: {symbol}, 订单ID: {order['orderId']}")
            
            # 记录交易使用原始数量和价值计算
            if position_manager and capital_manager:
                position_manager.record_trade(
                    symbol=symbol,
                    amount=investment, # 使用原始投资金额记录
                    price=price,
                    quantity=quantity, # 使用原始数量记录
                    action='BUY',
                    live_capital_manager=capital_manager
                )
            
            return {
                'order_id': order['orderId'],
                'symbol': symbol,
                'quantity': adjusted_quantity, # 返回实际下单数量
                'price': price,
                'status': order['status'],
                'original_quantity': quantity, # 保留原始数量信息
                'adjusted': True
            }
            
        except Exception as e:
            print(f"❌ 创建买入订单失败 {symbol}: {e}")
            
            # 测试网模拟
            if self.data_collector.mode_selector.mode == 'testnet':
                return self._create_mock_order(symbol, quantity, price, 'BUY', position_manager, capital_manager)
            else:
                return None
    
    def create_sell_order(self, symbol, quantity, price, position_manager=None, capital_manager=None):
        """创建卖出订单 - 增强精度处理"""
        try:
            investment = quantity * price
            
            if investment < self.min_investment:
                print(f"💰 卖出金额过小: ${investment:.2f} < ${self.min_investment}")
                return None
            
            if self.data_collector.client is None:
                print(f"❌ 客户端未连接，无法下单: {symbol}")
                return None
            
            # 🆕 增强精度处理 - 对于卖出，确保不超过持仓数量
            position_quantity = 0
            if position_manager and symbol in position_manager.positions:
                position_quantity = position_manager.positions[symbol]['quantity']
            
            adjusted_quantity, adjusted_investment = self._enhanced_adjust_quantity(
                symbol, min(quantity, position_quantity), price, 'SELL'
            )
            
            if adjusted_quantity <= 0:
                print(f"❌ 精度调整后卖出数量无效: {symbol}")
                return None
            
            if adjusted_investment < self.min_investment:
                print(f"💰 精度调整后卖出金额过小: ${adjusted_investment:.2f} < ${self.min_investment}")
                return None
            
            print(f"🔄 创建卖出订单: {symbol}, 数量: {adjusted_quantity:.8f}, 金额: ${adjusted_investment:.2f}")
            
            order = self.data_collector.client.order_market_sell(
                symbol=symbol,
                quantity=adjusted_quantity
            )
            
            print(f"✅ 卖出订单成功: {symbol}, 订单ID: {order['orderId']}")
            
            # 记录交易
            if position_manager and capital_manager:
                if symbol in position_manager.positions:
                    position = position_manager.positions[symbol]
                    entry_price = position['entry_price']
                    sold_cost = adjusted_quantity * entry_price # 使用调整后的数量计算成本
                    
                    position_manager.record_trade(
                        symbol=symbol,
                        amount=adjusted_investment, # 使用调整后的金额
                        price=price,
                        quantity=adjusted_quantity, # 使用调整后的数量
                        action='SELL',
                        live_capital_manager=capital_manager
                    )
            
            return {
                'order_id': order['orderId'],
                'symbol': symbol,
                'quantity': adjusted_quantity,
                'price': price,
                'status': order['status'],
                'original_quantity': quantity,
                'adjusted': True
            }
            
        except Exception as e:
            print(f"❌ 创建卖出订单失败 {symbol}: {e}")
            
            if self.data_collector.mode_selector.mode == 'testnet':
                return self._create_mock_order(symbol, quantity, price, 'SELL', position_manager, capital_manager)
            else:
                return None
    
    def _enhanced_adjust_quantity(self, symbol, quantity, price, side):
        """增强数量精度调整 - 修复精度问题"""
        try:
            # 获取币种精度信息
            symbol_info = self._get_symbol_info(symbol)
            if not symbol_info:
                print(f"⚠️ 无法获取 {symbol} 的精度信息，使用默认处理")
                return quantity, quantity * price
            
            # 获取数量精度限制
            lot_size_filter = next((f for f in symbol_info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
            min_notional_filter = next((f for f in symbol_info['filters'] if f['filterType'] == 'MIN_NOTIONAL'), None)
            
            if not lot_size_filter:
                print(f"⚠️ {symbol} 没有LOT_SIZE过滤器，使用默认处理")
                return quantity, quantity * price
            
            # 提取精度参数
            step_size = float(lot_size_filter['stepSize'])
            min_qty = float(lot_size_filter['minQty'])
            max_qty = float(lot_size_filter.get('maxQty', float('inf')))
            
            print(f"🔧 {symbol} 精度参数: 步长={step_size}, 最小数量={min_qty}, 最大数量={max_qty if max_qty != float('inf') else '无限制'}")
            
            # 🛠️ 关键修复：计算步长的小数位数
            step_size_str = str(step_size)
            if 'e-' in step_size_str:
                # 科学计数法处理，如 1e-5
                precision = int(step_size_str.split('e-')[1])
            else:
                # 普通小数处理
                if '.' in step_size_str:
                    precision = len(step_size_str.split('.')[1].rstrip('0'))
                else:
                    precision = 0
            
            print(f"🔧 {symbol} 数量精度: {precision} 位小数")
            
            # 调整数量到合适的精度
            adjusted_quantity = quantity
            
            # 1. 确保数量是步长的整数倍
            adjusted_quantity = math.floor(adjusted_quantity / step_size) * step_size
            
            # 2. 格式化到正确的小数位数
            adjusted_quantity = round(adjusted_quantity, precision)
            
            # 3. 再次确保是步长的整数倍（防止浮点数精度问题）
            adjusted_quantity = math.floor(adjusted_quantity / step_size) * step_size
            adjusted_quantity = round(adjusted_quantity, precision)
            
            # 4. 确保不小于最小数量
            if adjusted_quantity < min_qty:
                if quantity >= min_qty:
                    adjusted_quantity = min_qty
                    adjusted_quantity = math.floor(adjusted_quantity / step_size) * step_size
                    adjusted_quantity = round(adjusted_quantity, precision)
                else:
                    print(f"❌ 数量 {quantity} 小于最小值 {min_qty}")
                    return 0, 0
            
            # 5. 确保不大于最大数量
            if adjusted_quantity > max_qty:
                adjusted_quantity = max_qty
                adjusted_quantity = math.floor(adjusted_quantity / step_size) * step_size
                adjusted_quantity = round(adjusted_quantity, precision)
            
            # 6. 最终金额检查
            final_investment = adjusted_quantity * price
            
            # 检查最小成交金额
            if min_notional_filter:
                min_notional = float(min_notional_filter.get('minNotional', self.min_investment))
                if final_investment < min_notional:
                    # 尝试增加数量以满足最小成交金额
                    required_quantity = min_notional / price
                    required_quantity = math.ceil(required_quantity / step_size) * step_size
                    required_quantity = round(required_quantity, precision)
                    
                    if required_quantity <= max_qty:
                        adjusted_quantity = required_quantity
                        final_investment = adjusted_quantity * price
                        print(f"📈 调整数量以满足最小成交金额: {quantity:.8f} -> {adjusted_quantity:.8f}")
                    else:
                        print(f"❌ 无法满足最小成交金额: ${final_investment:.2f} < ${min_notional:.2f}")
                        return 0, 0
            
            if final_investment < self.min_investment:
                print(f"❌ 最终投资金额不足: ${final_investment:.2f} < ${self.min_investment:.2f}")
                return 0, 0
            
            print(f"✅ {symbol} {side}数量调整: {quantity:.8f} -> {adjusted_quantity:.8f}, "
                  f"金额: ${quantity*price:.2f} -> ${final_investment:.2f}")
            
            return adjusted_quantity, final_investment
            
        except Exception as e:
            print(f"❌ 增强数量调整失败 {symbol}: {e}")
            import traceback
            traceback.print_exc()
            return quantity, quantity * price
    
    def _get_symbol_info(self, symbol):
        """获取币种信息并缓存 - 增强精度信息"""
        try:
            # 检查缓存
            if symbol in self.symbol_precision_cache:
                cached_info = self.symbol_precision_cache[symbol]
                # 缓存1小时
                if time.time() - cached_info['timestamp'] < 3600:
                    return cached_info['symbol_info']
            
            print(f"🔍 获取 {symbol} 的精度信息...")
            symbol_info = self.data_collector.client.get_symbol_info(symbol)
            
            if symbol_info:
                # 解析精度信息
                lot_size_filter = next((f for f in symbol_info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
                if lot_size_filter:
                    step_size = float(lot_size_filter['stepSize'])
                    min_qty = float(lot_size_filter['minQty'])
                    max_qty = float(lot_size_filter.get('maxQty', float('inf')))
                    
                    print(f"📏 {symbol} 精度详情:")
                    print(f" 步长(stepSize): {step_size}")
                    print(f" 最小数量(minQty): {min_qty}")
                    print(f" 最大数量(maxQty): {max_qty if max_qty != float('inf') else '无限制'}")
                
                # 缓存结果
                self.symbol_precision_cache[symbol] = {
                    'symbol_info': symbol_info,
                    'timestamp': time.time()
                }
                
                return symbol_info
            else:
                print(f"❌ 无法获取 {symbol} 的交易对信息")
                return None
                
        except Exception as e:
            print(f"❌ 获取币种信息失败 {symbol}: {e}")
            return None
    
    def _create_mock_order(self, symbol, quantity, price, side, position_manager, capital_manager):
        """创建模拟订单（测试网）"""
        order_id = f"{side.lower()}_{symbol}_{int(time.time())}"
        print(f"⚠️ 使用模拟订单: {symbol}, 订单ID: {order_id}")
        
        # 模拟精度调整
        adjusted_quantity, adjusted_investment = self._enhanced_adjust_quantity(symbol, quantity, price, side)
        
        if adjusted_quantity <= 0:
            return None
        
        if side == 'BUY':
            if position_manager and capital_manager:
                position_manager.record_trade(
                    symbol=symbol,
                    amount=adjusted_investment,
                    price=price,
                    quantity=adjusted_quantity,
                    action='BUY',
                    live_capital_manager=capital_manager
                )
        else: # SELL
            if position_manager and capital_manager:
                if symbol in position_manager.positions:
                    position = position_manager.positions[symbol]
                    entry_price = position['entry_price']
                    sold_cost = adjusted_quantity * entry_price
                    
                    position_manager.record_trade(
                        symbol=symbol,
                        amount=adjusted_investment,
                        price=price,
                        quantity=adjusted_quantity,
                        action='SELL',
                        live_capital_manager=capital_manager
                    )
        
        return {
            'order_id': order_id,
            'symbol': symbol,
            'quantity': adjusted_quantity,
            'price': price,
            'status': 'FILLED',
            'original_quantity': quantity,
            'adjusted': adjusted_quantity != quantity
        }

    def check_market_status(self, symbol):
        """检查市场状态"""
        try:
            if self.data_collector.client is None:
                return True
                
            symbol_info = self.data_collector.client.get_symbol_info(symbol)
            if not symbol_info:
                print(f"❌ 无法获取币种信息: {symbol}")
                return False
                
            status = symbol_info.get('status')
            if status != 'TRADING':
                print(f"❌ 市场未开放: {symbol} - 状态: {status}")
                return False
                
            return True
            
        except Exception as e:
            print(f"❌ 检查市场状态失败 {symbol}: {e}")
            return True

# ========== 增强市场状态分析器 - 使用200个币种进行市场广度分析 ==========
class EnhancedMarketStateAnalyzer:
    """增强市场状态分析器 - 使用200个币种进行市场广度分析"""
    
    def __init__(self, data_collector, coin_list_manager):
        self.data_collector = data_collector
        self.coin_list_manager = coin_list_manager
        self.market_states = ['bull', 'consolidation', 'bear']
        self.state_history = deque(maxlen=10)
        self.config = OptimizedLongRunConfig()
    
    def professional_market_analysis(self):
        """专业市场分析 - 增强版，使用200个币种进行市场广度分析"""
        try:
            # 多币种确认趋势
            btc_data = self.data_collector.get_klines('BTCUSDT', '1h', 100)
            eth_data = self.data_collector.get_klines('ETHUSDT', '1h', 100)
            bnb_data = self.data_collector.get_klines('BNBUSDT', '1h', 100)
            sol_data = self.data_collector.get_klines('SOLUSDT', '1h', 100)
            
            if any(data is None for data in [btc_data, eth_data, bnb_data]):
                return "consolidation", 0.5, 50, {}
            
            # 多时间框架确认
            btc_daily = self.data_collector.get_klines('BTCUSDT', '1d', 50)
            eth_daily = self.data_collector.get_klines('ETHUSDT', '1d', 50)
            
            # 计算多币种趋势强度
            btc_trend = self._analyze_multi_timeframe_trend(btc_data, btc_daily)
            eth_trend = self._analyze_multi_timeframe_trend(eth_data, eth_daily)
            bnb_trend = self._analyze_trend(bnb_data) if bnb_data is not None else 0
            sol_trend = self._analyze_trend(sol_data) if sol_data is not None else 0
            
            # 加权平均趋势（BTC权重40%，ETH30%，BNB20%，SOL10%）
            avg_trend = (btc_trend * 0.4 + eth_trend * 0.3 + 
                        bnb_trend * 0.2 + sol_trend * 0.1)
            
            # 🆕 使用200个币种进行真实市场广度分析
            market_breadth = self._analyze_enhanced_market_breadth()
            
            market_state, confidence = self._determine_enhanced_market_state(
                avg_trend, market_breadth)
            
            market_score = self._calculate_enhanced_market_score(
                avg_trend, market_breadth)
            
            self.state_history.append(market_state)
            
            # 状态确认机制
            if len(self.state_history) >= 3:
                recent_states = list(self.state_history)[-3:]
                if all(s == market_state for s in recent_states):
                    confidence = min(1.0, confidence * 1.2) # 确认状态增加信心度
            
            return market_state, confidence, market_score, market_breadth
            
        except Exception as e:
            print(f"❌ 增强市场分析失败: {e}")
            return "consolidation", 0.5, 50, {}
    
    def _analyze_trend(self, data):
        """分析趋势"""
        try:
            if data is None or len(data) < 20:
                return 0
            
            data['ma20'] = data['close'].rolling(20).mean()
            data['ma50'] = data['close'].rolling(50).mean()
            data['rsi'] = self._calculate_rsi(data['close'])
            
            current_price = data['close'].iloc[-1]
            ma20 = data['ma20'].iloc[-1]
            ma50 = data['ma50'].iloc[-1]
            rsi = data['rsi'].iloc[-1]
            
            bull_signals = 0
            if current_price > ma20: bull_signals += 1
            if current_price > ma50: bull_signals += 1
            if ma20 > ma50: bull_signals += 1
            if rsi > 50: bull_signals += 1
            
            bear_signals = 0
            if current_price < ma20: bear_signals += 1
            if current_price < ma50: bear_signals += 1
            if ma20 < ma50: bear_signals += 1
            if rsi < 50: bear_signals += 1
            
            trend_strength = (bull_signals - bear_signals) / 4.0
            
            return trend_strength
            
        except Exception as e:
            print(f"❌ 趋势分析失败: {e}")
            return 0
    
    def _analyze_multi_timeframe_trend(self, hourly_data, daily_data):
        """多时间框架趋势分析"""
        try:
            hourly_trend = self._analyze_trend(hourly_data)
            
            if daily_data is not None and len(daily_data) >= 20:
                daily_trend = self._analyze_trend(daily_data)
                # 加权平均：小时线60%，日线40%
                combined_trend = hourly_trend * 0.6 + daily_trend * 0.4
                return combined_trend
            
            return hourly_trend
            
        except Exception as e:
            print(f"❌ 多时间框架分析失败: {e}")
            return self._analyze_trend(hourly_data)
    
    def _calculate_rsi(self, prices, period=14):
        """计算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except:
            return 50
    
    def _analyze_enhanced_market_breadth(self):
        """增强市场广度分析 - 使用200个币种"""
        try:
            # 获取所有币种列表
            all_coins = list(self.coin_list_manager.coin_universe)
            
            # 如果币种数量过多，随机采样100个进行分析（避免API限制）
            if len(all_coins) > 100:
                sample_coins = random.sample(all_coins, 100)
                print(f"📊 从{len(all_coins)}个币种中随机采样100个进行市场广度分析")
            else:
                sample_coins = all_coins
                print(f"📊 使用全部{len(all_coins)}个币种进行市场广度分析")
            
            advancing = 0
            declining = 0
            unchanged = 0
            total_analyzed = 0
            
            # 分析每个币种的24小时价格变化
            for coin in sample_coins:
                try:
                    # 获取24小时K线数据
                    data = self.data_collector.get_klines(coin, '1h', 24)
                    if data is None or len(data) < 24:
                        continue
                    
                    # 计算24小时价格变化
                    price_change = (data['close'].iloc[-1] - data['close'].iloc[0]) / data['close'].iloc[0]
                    
                    if price_change > 0.01: # 上涨超过1%
                        advancing += 1
                    elif price_change < -0.01: # 下跌超过1%
                        declining += 1
                    else:
                        unchanged += 1
                    
                    total_analyzed += 1
                    
                    # 添加小延迟避免API限制
                    time.sleep(0.01)
                    
                except Exception as e:
                    print(f"⚠️ 分析币种 {coin} 失败: {e}")
                    continue
            
            if total_analyzed > 0:
                advance_ratio = advancing / total_analyzed
                decline_ratio = declining / total_analyzed
                advance_decline_ratio = advancing / declining if declining > 0 else float('inf')
                
                print(f"📈 市场广度分析结果:")
                print(f" 上涨币种: {advancing} ({advance_ratio:.1%})")
                print(f" 下跌币种: {declining} ({decline_ratio:.1%})")
                print(f" 持平币种: {unchanged} ({(unchanged/total_analyzed):.1%})")
                print(f" 涨跌比率: {advance_decline_ratio:.2f}")
                print(f" 总分析币种: {total_analyzed}")
                
                return {
                    'advancing_issues': advancing,
                    'declining_issues': declining,
                    'unchanged_issues': unchanged,
                    'advance_decline_ratio': advance_decline_ratio,
                    'breadth_strength': advance_ratio,
                    'total_analyzed': total_analyzed,
                    'advance_ratio': advance_ratio,
                    'decline_ratio': decline_ratio
                }
            
            return {
                'advancing_issues': 0,
                'declining_issues': 0,
                'unchanged_issues': 0,
                'advance_decline_ratio': 1.0,
                'breadth_strength': 0.5,
                'total_analyzed': 0,
                'advance_ratio': 0.5,
                'decline_ratio': 0.5
            }
            
        except Exception as e:
            print(f"❌ 增强市场广度分析失败: {e}")
            return {
                'advancing_issues': 50,
                'declining_issues': 50,
                'unchanged_issues': 0,
                'advance_decline_ratio': 1.0,
                'breadth_strength': 0.5,
                'total_analyzed': 100,
                'advance_ratio': 0.5,
                'decline_ratio': 0.5
            }
    
    def _determine_enhanced_market_state(self, trend_strength, market_breadth):
        """增强市场状态判定"""
        try:
            breadth_strength = market_breadth.get('breadth_strength', 0.5)
            advance_ratio = market_breadth.get('advance_ratio', 0.5)
            advance_decline_ratio = market_breadth.get('advance_decline_ratio', 1.0)
            
            # 复合评分：趋势40%，广度40%，涨跌比率20%
            composite_score = (trend_strength * 0.4 + 
                             breadth_strength * 0.4 + 
                             (min(advance_decline_ratio, 3.0) / 3.0) * 0.2)
            
            trend_threshold = self.config.MARKET_STATE_CONFIG["trend_threshold"]
            
            # 强牛市条件：综合评分高且上涨币种比例高
            if composite_score > trend_threshold + 0.1 and advance_ratio > 0.6:
                state = "bull"
                confidence = min(1.0, composite_score * 1.1)
            elif composite_score > trend_threshold:
                state = "bull"
                confidence = min(1.0, composite_score)
            # 强熊市条件：综合评分低且下跌币种比例高
            elif composite_score < -trend_threshold - 0.1 and advance_ratio < 0.4:
                state = "bear"
                confidence = min(1.0, -composite_score * 1.1)
            elif composite_score < -trend_threshold:
                state = "bear" 
                confidence = min(1.0, -composite_score)
            else:
                state = "consolidation"
                confidence = 1.0 - abs(composite_score)
            
            # 成交量确认
            if confidence > 0.7:
                volume_confirmation = self._check_volume_confirmation(state)
                if not volume_confirmation:
                    confidence *= 0.8 # 缺乏成交量确认，降低信心度
            
            print(f"🎯 市场状态判定: {state.upper()}, 信心度: {confidence:.1%}")
            print(f" 综合评分: {composite_score:.3f}, 上涨比例: {advance_ratio:.1%}")
            
            return state, confidence
            
        except Exception as e:
            print(f"❌ 增强市场状态判定失败: {e}")
            return "consolidation", 0.5
    
    def _check_volume_confirmation(self, state):
        """成交量确认"""
        try:
            btc_data = self.data_collector.get_klines('BTCUSDT', '1h', 24)
            if btc_data is None:
                return True
                
            current_volume = btc_data['volume'].iloc[-1]
            avg_volume = btc_data['volume'].mean()
            
            volume_ratio = current_volume / avg_volume if avg_volume > 0 else 1
            
            if state == "bull":
                return volume_ratio >= 1.2 # 牛市需要放量确认
            elif state == "bear":
                return volume_ratio >= 1.1 # 熊市也需要一定成交量
            else:
                return True # 震荡市对成交量要求不高
                
        except:
            return True
    
    def _calculate_enhanced_market_score(self, trend_strength, market_breadth):
        """计算增强市场得分"""
        try:
            breadth_strength = market_breadth.get('breadth_strength', 0.5)
            advance_ratio = market_breadth.get('advance_ratio', 0.5)
            advance_decline_ratio = market_breadth.get('advance_decline_ratio', 1.0)
            
            # 综合评分：趋势40%，广度40%，涨跌比率20%
            score = 50 + (trend_strength * 20) + (breadth_strength * 20) + (min(advance_decline_ratio, 3.0) / 3.0 * 10)
            return max(0, min(100, score))
            
        except Exception as e:
            print(f"❌ 计算增强市场得分失败: {e}")
            return 50

# ========== 增强选币器 - 修改资金面权重为60% ==========
class EnhancedCoinSelector:
    """增强选币器 - 多维度评分 + 板块分散 + 修改资金面权重为60%"""
    
    def __init__(self, data_collector, coin_list_manager):
        self.data_collector = data_collector
        self.coin_list_manager = coin_list_manager
        self.sector_classification = self._initialize_sector_classification()
    
    def _initialize_sector_classification(self):
        """初始化板块分类"""
        return {
            'layer1': ['ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'AVAXUSDT', 'ATOMUSDT', 'DOTUSDT'],
            'defi': ['UNIUSDT', 'AAVEUSDT', 'COMPUSDT', 'MKRUSDT', 'SUSHIUSDT', 'CRVUSDT'],
            'layer2': ['MATICUSDT', 'ARBUSDT', 'OPUSDT', 'IMXUSDT'],
            'meme': ['DOGEUSDT', 'SHIBUSDT', 'PEPEUSDT'],
            'ai': ['AGIXUSDT', 'FETUSDT', 'OCEANUSDT'],
            'gaming': ['SANDUSDT', 'MANAUSDT', 'GALAUSDT', 'ENJUSDT'],
            'oracles': ['LINKUSDT', 'BANDUSDT', 'TRBUSDT'],
            'storage': ['FILUSDT', 'ARUSDT'],
            'payment': ['XRPUSDT', 'XLMUSDT', 'ALGOUSDT'],
            'privacy': ['ZECUSDT', 'DASHUSDT', 'XMRUSDT']
        }
    
    def professional_screening(self, market_state, current_positions=None):
        """专业筛选 - 增强版"""
        try:
            # 获取更新的币种列表和评分
            self.coin_universe = self.coin_list_manager.update_coin_scores()
            
            candidates = []
            top_coins = self.coin_list_manager.get_top_coins(80)
            
            print(f"🔍 开始筛选 {len(top_coins)} 个候选币种...")
            
            for symbol in top_coins:
                try:
                    # 实时验证币种是否可交易
                    if not self._real_time_validation(symbol):
                        continue
                    
                    # 获取K线数据
                    klines = self.data_collector.get_klines(symbol, '1h', 100)
                    if klines is None or len(klines) < 50:
                        continue
                    
                    # 多维度分析 - 修改权重分配
                    analysis = self._enhanced_coin_analysis(symbol, klines, market_state)
                    if analysis:
                        candidates.append(analysis)
                        
                except Exception as e:
                    print(f"❌ 增强分析币种失败 {symbol}: {e}")
                    continue
            
            # 按评分排序并限制数量
            candidates.sort(key=lambda x: x['score'], reverse=True)
            
            print(f"✅ 筛选完成: 从 {len(top_coins)} 个候选中找到 {len(candidates)} 个有效币种")
            return candidates[:50]
            
        except Exception as e:
            print(f"❌ 增强选币筛选失败: {e}")
            return []

    def _real_time_validation(self, symbol):
        """实时验证币种是否可交易"""
        try:
            # 检查交易对状态
            symbol_info = self.data_collector.client.get_symbol_info(symbol)
            if not symbol_info or symbol_info.get('status') != 'TRADING':
                return False
            
            # 检查价格
            price = self.data_collector.get_price(symbol)
            if not price or price <= 0:
                return False
            
            return True
            
        except Exception as e:
            return False
    
    def _enhanced_coin_analysis(self, symbol, data, market_state):
        """增强币种分析 - 修改权重分配：技术面20%，基本面20%，资金面60%"""
        try:
            # 技术面分析 (权重20%)
            technical_score = self._calculate_technical_score(data) * 0.4 # 原来50分制转20分制
            
            # 基本面分析 (权重20%)
            fundamental_score = self._calculate_fundamental_score(symbol) * 0.4 # 原来50分制转20分制
            
            # 资金面分析 (权重60%) - 使用15分钟K线增强版
            volume_score = self._calculate_volume_score_enhanced(symbol, data)
            
            # 综合评分
            total_score = (technical_score + fundamental_score + volume_score)
            
            # 市场状态调整
            if market_state == 'bull':
                total_score = min(100, total_score * 1.1)
            elif market_state == 'bear':
                total_score = total_score * 0.9
            
            current_price = data['close'].iloc[-1]
            price_change_24h = (current_price - data['close'].iloc[-24]) / data['close'].iloc[-24] if len(data) >= 24 else 0
            
            return {
                'symbol': symbol,
                'current_price': current_price,
                'score': total_score,
                'technical_score': technical_score,
                'fundamental_score': fundamental_score,
                'volume_score': volume_score,
                'price_increase': price_change_24h,
                'sector': self._get_coin_sector(symbol),
                'volume_ratio': self._get_15m_volume_ratio(symbol) # 新增15分钟成交量比率
            }
            
        except Exception as e:
            print(f"❌ 增强币种分析失败 {symbol}: {e}")
            return None
    
    def _calculate_volume_score_enhanced(self, symbol, data):
        """增强版资金面分析 - 15分钟K线基准，权重60%"""
        score = 0
        max_score = 60 # 资金面满分60分
        
        try:
            # 1. 获取15分钟K线数据
            klines_15m = self.data_collector.get_klines(symbol, '15m', 25) # 25根 = 过去6.25小时
            
            if klines_15m is None or len(klines_15m) < 21: # 需要至少21根来计算20期平均
                return 30 # 数据不足时给中等分数
                
            # 2. 计算成交量比率
            current_volume = klines_15m['volume'].iloc[-1] # 当前15分钟成交量
            past_5h_volumes = klines_15m['volume'].iloc[-21:-1] # 过去5小时成交量（20个15分钟）
            avg_5h_volume = past_5h_volumes.mean()
            
            if avg_5h_volume <= 0:
                return 30
                
            volume_ratio = current_volume / avg_5h_volume
            
            # 3. 根据成交量放大倍数打分
            if volume_ratio >= 5.0:
                score += 25 # 爆量5倍以上
            elif volume_ratio >= 4.0:
                score += 20 # 放量4倍
            elif volume_ratio >= 3.0:
                score += 15 # 放量3倍  
            elif volume_ratio >= 2.0:
                score += 10 # 放量2倍
            elif volume_ratio >= 1.5:
                score += 5 # 温和放量
            else:
                score += 0 # 无量
            
            # 4. 🛡️ 防止高位放量接盘 - 价格位置分析
            price_safety_score = self._calculate_price_safety_score(symbol, klines_15m, volume_ratio)
            score += price_safety_score
            
            # 5. 成交量持续性分析
            continuity_score = self._calculate_volume_continuity_score(klines_15m)
            score += continuity_score
            
            return min(max_score, max(0, score))
            
        except Exception as e:
            print(f"❌ 增强资金面分析失败 {symbol}: {e}")
            return 30
    
    def _calculate_price_safety_score(self, symbol, klines_15m, volume_ratio):
        """计算价格安全分数 - 防止高位接盘"""
        safety_score = 0
        
        try:
            current_price = klines_15m['close'].iloc[-1]
            
            # 1. 相对强度分析 - 对比BTC
            btc_data = self.data_collector.get_klines('BTCUSDT', '15m', 25)
            if btc_data is not None and len(btc_data) >= 25:
                btc_current = btc_data['close'].iloc[-1]
                btc_24h_ago = btc_data['close'].iloc[-25] # 24小时前（24个15分钟）
                btc_change = (btc_current - btc_24h_ago) / btc_24h_ago
                
                coin_24h_ago = klines_15m['close'].iloc[-25]
                coin_change = (current_price - coin_24h_ago) / coin_24h_ago
                
                # 如果币种涨幅远大于BTC，可能过热
                relative_strength = coin_change - btc_change
                if relative_strength > 0.15: # 相对BTC超额涨幅15%
                    safety_score -= 10
                elif relative_strength < -0.05: # 相对BTC弱势
                    safety_score += 5
            
            # 2. 价格位置分析
            high_24h = klines_15m['high'].max() # 24小时内最高价
            low_24h = klines_15m['low'].min() # 24小时内最低价
            
            if high_24h != low_24h: # 避免除零
                price_position = (current_price - low_24h) / (high_24h - low_24h)
                
                # 价格在相对高位且放量，风险较高
                if price_position > 0.7 and volume_ratio > 2.0:
                    safety_score -= 15 # 高位放量，大幅扣分
                elif price_position > 0.8 and volume_ratio > 3.0:
                    safety_score -= 25 # 极高位置爆量，风险极大
                elif price_position < 0.3 and volume_ratio > 2.0:
                    safety_score += 10 # 低位放量，安全性高
                elif price_position < 0.5:
                    safety_score += 5 # 中低位相对安全
            
            # 3. 波动率过滤 - 避免在异常波动时买入
            volatility = klines_15m['close'].pct_change().std()
            if volatility > 0.08: # 15分钟线波动率超过8%
                safety_score -= 8
            
            # 4. 成交量与价格背离检测
            divergence_score = self._detect_volume_price_divergence(klines_15m)
            safety_score += divergence_score
            
            return safety_score
            
        except Exception as e:
            print(f"❌ 价格安全分析失败 {symbol}: {e}")
            return 0
    
    def _calculate_volume_continuity_score(self, klines_15m):
        """分析成交量持续性 - 避免单根放量骗炮"""
        continuity_score = 0
        
        try:
            recent_volumes = klines_15m['volume'].iloc[-6:] # 最近1.5小时（6个15分钟）
            avg_earlier_volume = klines_15m['volume'].iloc[-21:-6].mean() # 之前4.5小时平均
            
            if avg_earlier_volume <= 0:
                return 0
                
            # 检查持续性：最近1.5小时是否有多个周期放量
            sustained_volume_count = 0
            for volume in recent_volumes:
                if volume / avg_earlier_volume > 1.8: # 持续1.8倍以上放量
                    sustained_volume_count += 1
            
            if sustained_volume_count >= 3: # 至少3个周期持续放量
                continuity_score += 10
            elif sustained_volume_count >= 2:
                continuity_score += 5
            elif sustained_volume_count == 1:
                continuity_score -= 5 # 单根放量，可能骗炮
                
            return continuity_score
            
        except Exception as e:
            print(f"❌ 成交量持续性分析失败: {e}")
            return 0
    
    def _detect_volume_price_divergence(self, klines_15m):
        """检测量价背离 - 重要的风险信号"""
        divergence_score = 0
        
        try:
            # 最近6个周期（1.5小时）的价格和成交量
            recent_prices = klines_15m['close'].iloc[-6:]
            recent_volumes = klines_15m['volume'].iloc[-6:]
            
            # 计算价格趋势和成交量趋势
            price_trend = (recent_prices.iloc[-1] - recent_prices.iloc[0]) / recent_prices.iloc[0]
            volume_trend = (recent_volumes.iloc[-1] - recent_volumes.iloc[0]) / recent_volumes.iloc[0]
            
            # 量价背离检测
            if price_trend > 0.03 and volume_trend < -0.2: # 价升量缩
                divergence_score -= 12 # 顶背离，风险信号
            elif price_trend < -0.02 and volume_trend > 0.3: # 价跌量增
                divergence_score -= 8 # 放量下跌，风险信号
            elif price_trend > 0.02 and volume_trend > 0.3: # 价升量增
                divergence_score += 8 # 健康上涨
            elif price_trend < -0.03 and volume_trend < -0.2: # 价跌量缩
                divergence_score += 5 # 缩量调整，相对健康
                
            return divergence_score
            
        except Exception as e:
            print(f"❌ 量价背离分析失败: {e}")
            return 0
    
    def _get_15m_volume_ratio(self, symbol):
        """获取15分钟成交量比率"""
        try:
            klines_15m = self.data_collector.get_klines(symbol, '15m', 25)
            if klines_15m is None or len(klines_15m) < 21:
                return 1.0
                
            current_volume = klines_15m['volume'].iloc[-1]
            past_5h_volumes = klines_15m['volume'].iloc[-21:-1]
            avg_5h_volume = past_5h_volumes.mean()
            
            return current_volume / avg_5h_volume if avg_5h_volume > 0 else 1.0
            
        except Exception as e:
            print(f"❌ 获取15分钟成交量比率失败 {symbol}: {e}")
            return 1.0
    
    def _calculate_technical_score(self, data):
        """计算技术面评分"""
        try:
            score = 0
            
            # 移动平均线分析
            data['ma20'] = data['close'].rolling(20).mean()
            data['ma50'] = data['close'].rolling(50).mean()
            
            current_price = data['close'].iloc[-1]
            ma20 = data['ma20'].iloc[-1]
            ma50 = data['ma50'].iloc[-1]
            
            if current_price > ma20: score += 15
            if current_price > ma50: score += 20
            if ma20 > ma50: score += 15
            
            # RSI分析
            rsi = self._calculate_rsi(data['close'])
            if 30 < rsi < 70: score += 10 # 避免超买超卖
            elif 40 < rsi < 60: score += 15 # 强势区间
            
            # 动量分析
            momentum_5h = (current_price - data['close'].iloc[-5]) / data['close'].iloc[-5] if len(data) >= 5 else 0
            if momentum_5h > 0: score += 10
            
            return min(50, score) # 技术面满分50
            
        except:
            return 25
    
    def _calculate_fundamental_score(self, symbol):
        """计算基本面评分"""
        try:
            score = 25 # 基础分
            
            # 市值分类
            large_caps = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT']
            mid_caps = ['SOLUSDT', 'DOTUSDT', 'DOGEUSDT', 'MATICUSDT', 'LTCUSDT']
            
            if symbol in large_caps:
                score += 20 # 大盘币加分
            elif symbol in mid_caps:
                score += 15 # 中盘币加分
            else:
                score += 10 # 小盘币基础分
            
            # 板块强度分析（简化版）
            strong_sectors = ['layer1', 'defi', 'layer2']
            coin_sector = self._get_coin_sector(symbol)
            if coin_sector in strong_sectors:
                score += 5
            
            return min(50, score) # 基本面满分50
            
        except:
            return 25
    
    def _calculate_rsi(self, prices, period=14):
        """计算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except:
            return 50
    
    def _get_coin_sector(self, symbol):
        """获取币种所属板块"""
        for sector, coins in self.sector_classification.items():
            if symbol in coins:
                return sector
        return 'other'

# ========== 持仓时间管理器 ==========
class PositionHoldingManager:
    """持仓时间管理器"""
    
    def __init__(self, min_holding_hours=8): # 从6提高到8小时
        self.min_holding_hours = min_holding_hours
        self.buy_times = {}
    
    def record_buy(self, symbol):
        """记录买入时间"""
        self.buy_times[symbol] = datetime.now()
    
    def record_sell(self, symbol):
        """记录卖出时间"""
        if symbol in self.buy_times:
            del self.buy_times[symbol]
    
    def get_holding_hours(self, symbol):
        """获取持仓小时数"""
        if symbol in self.buy_times:
            buy_time = self.buy_times[symbol]
            holding_hours = (datetime.now() - buy_time).total_seconds() / 3600
            return holding_hours
        return 0

# ========== 状态管理器 - 修复版本 ==========
class EnhancedStateManager:
    """增强状态管理器 - 修复时间序列化问题"""
    
    def __init__(self):
        self.state_file = "trading_bot_state.json"
        self.backup_dir = "state_backups"
        self.max_backups = 10
        
        if not os.path.exists(self.backup_dir):
            os.makedirs(self.backup_dir)
    
    def save_complete_state(self, bot):
        """保存完整状态 - 修复时间序列化"""
        try:
            state = {
                'timestamp': datetime.now().isoformat(),
                'position_manager': self._extract_position_state(bot.position_manager),
                'market_state': bot.current_market_state,
                'trade_count': bot.trade_count,
                'continuous_errors': bot.continuous_errors,
                'start_time': bot.start_time.isoformat(),
                'total_running_hours': (datetime.now() - bot.start_time).total_seconds() / 3600
            }
            
            with open(self.state_file, 'w', encoding='utf-8') as f:
                json.dump(state, f, indent=2, default=self._json_serializer)
            
            self._create_backup(state)
            
            print("✅ 状态保存完成")
            return True
            
        except Exception as e:
            print(f"❌ 状态保存失败: {e}")
            return False
    
    def load_complete_state(self, bot):
        """加载完整状态"""
        try:
            if not os.path.exists(self.state_file):
                print("🆕 无保存状态文件，从初始状态开始运行")
                return False
            
            with open(self.state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            
            print("🔄 加载保存的状态...")
            
            state_timestamp_str = state.get('timestamp')
            if state_timestamp_str:
                state_timestamp = datetime.fromisoformat(state_timestamp_str)
                current_time = datetime.now()
                time_diff_hours = (current_time - state_timestamp).total_seconds() / 3600
                
                if time_diff_hours > 24:
                    print(f"⚠️ 状态文件已过期 ({time_diff_hours:.1f}小时)，仅恢复配置状态")
                    bot.current_market_state = state.get('market_state', 'consolidation')
                    bot.trade_count = state.get('trade_count', 0)
                    bot.continuous_errors = state.get('continuous_errors', 0)
                    
                    start_time_str = state.get('start_time')
                    if start_time_str:
                        bot.start_time = datetime.fromisoformat(start_time_str)
                    
                    print("✅ 仅恢复配置状态，保留原有持仓")
                    return True
            
            if 'position_manager' in state:
                self._restore_position_state(bot.position_manager, state['position_manager'])
            
            bot.current_market_state = state.get('market_state', 'consolidation')
            bot.trade_count = state.get('trade_count', 0)
            bot.continuous_errors = state.get('continuous_errors', 0)
            
            start_time_str = state.get('start_time')
            if start_time_str:
                bot.start_time = datetime.fromisoformat(start_time_str)
            
            print("✅ 状态恢复完成")
            return True
            
        except Exception as e:
            print(f"❌ 状态恢复失败: {e}")
            return False

    def _json_serializer(self, obj):
        """JSON序列化处理器"""
        if isinstance(obj, datetime):
            return obj.isoformat()
        raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

    def _extract_position_state(self, position_manager):
        """提取持仓管理器状态"""
        return {
            'positions': position_manager.positions,
            'trade_history': position_manager.trade_history,
            'available_balance': position_manager.available_balance,
            'allocated_balance': position_manager.allocated_balance
        }
    
    def _restore_position_state(self, position_manager, state_data):
        """恢复持仓管理器状态 - 修复时间反序列化"""
        try:
            position_manager.positions = state_data.get('positions', {})
            position_manager.trade_history = state_data.get('trade_history', [])
            position_manager.available_balance = state_data.get('available_balance', 1000)
            position_manager.allocated_balance = state_data.get('allocated_balance', 0)
            
            # 修复：安全转换时间字段
            self._safe_convert_timestamps(position_manager)
            
            print(f"✅ 状态恢复: {len(position_manager.positions)}个持仓，可用资金${position_manager.available_balance:.2f}")
            
        except Exception as e:
            print(f"❌ 恢复持仓状态失败: {e}")

    def _safe_convert_timestamps(self, position_manager):
        """安全转换时间戳"""
        # 转换持仓时间
        for symbol, position in position_manager.positions.items():
            if 'entry_time' in position:
                position['entry_time'] = self._parse_datetime_safe(position['entry_time'])
        
        # 转换交易历史时间
        for trade in position_manager.trade_history:
            if 'time' in trade:
                trade['time'] = self._parse_datetime_safe(trade['time'])

    def _parse_datetime_safe(self, dt_str):
        """安全解析datetime"""
        if isinstance(dt_str, datetime):
            return dt_str
        
        try:
            # 处理各种时间格式
            if 'Z' in str(dt_str):
                return datetime.fromisoformat(dt_str.replace('Z', '+00:00'))
            elif '+' in str(dt_str):
                return datetime.fromisoformat(dt_str)
            else:
                return datetime.fromisoformat(dt_str + '+00:00')
        except:
            print(f"⚠️ 时间解析失败，使用当前时间: {dt_str}")
            return datetime.now()
    
    def _create_backup(self, state):
        """创建状态备份"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_file = os.path.join(self.backup_dir, f"state_backup_{timestamp}.json")
            
            with open(backup_file, 'w', encoding='utf-8') as f:
                json.dump(state, f, indent=2, default=self._json_serializer)
            
            self._cleanup_old_backups()
            
        except Exception as e:
            print(f"⚠️ 创建备份失败: {e}")
    
    def _cleanup_old_backups(self):
        """清理旧备份"""
        try:
            backups = []
            for file in os.listdir(self.backup_dir):
                if file.startswith("state_backup_") and file.endswith(".json"):
                    file_path = os.path.join(self.backup_dir, file)
                    backups.append((file_path, os.path.getctime(file_path)))
            
            backups.sort(key=lambda x: x[1], reverse=True)
            
            for backup in backups[self.max_backups:]:
                os.remove(backup[0])
                print(f"🗑️ 删除旧备份: {os.path.basename(backup[0])}")
                
        except Exception as e:
            print(f"⚠️ 清理备份失败: {e}")

# ========== 粉尘持仓管理器 ==========
class DustPositionManager:
    """粉尘持仓管理器 - 解决粉尘占用资源问题"""
    
    def __init__(self, bot, dust_threshold=0.3):
        self.bot = bot
        self.dust_threshold = dust_threshold # 0.3美元以下的持仓视为粉尘
        self.dust_positions = set()
    
    def identify_dust_positions(self):
        """识别粉尘持仓"""
        print("🔍 扫描粉尘持仓...")
        dust_positions = []
        
        for symbol, position in self.bot.position_manager.positions.items():
            try:
                current_price = self.bot.data_collector.get_price(symbol)
                if current_price and current_price > 0:
                    position_value = position['quantity'] * current_price
                    
                    if position_value < self.dust_threshold:
                        dust_positions.append({
                            'symbol': symbol,
                            'quantity': position['quantity'],
                            'current_price': current_price,
                            'position_value': position_value,
                            'holding_hours': self.bot.holding_manager.get_holding_hours(symbol)
                        })
                        print(f"🚫 发现粉尘持仓: {symbol}, 价值: ${position_value:.4f}")
                        
            except Exception as e:
                print(f"❌ 检查粉尘持仓失败 {symbol}: {e}")
                continue
        
        self.dust_positions = set([pos['symbol'] for pos in dust_positions])
        print(f"📊 粉尘持仓统计: 共发现 {len(dust_positions)} 个粉尘持仓")
        return dust_positions
    
    def get_effective_positions(self):
        """获取有效持仓（排除粉尘）"""
        effective_positions = {}
        dust_count = 0
        
        for symbol, position in self.bot.position_manager.positions.items():
            if symbol not in self.dust_positions:
                effective_positions[symbol] = position
            else:
                dust_count += 1
        
        print(f"📊 有效持仓: {len(effective_positions)}个, 粉尘持仓: {dust_count}个")
        return effective_positions
    
    def is_dust_position(self, symbol):
        """判断是否为粉尘持仓"""
        return symbol in self.dust_positions
    
    def attempt_dust_cleanup(self):
        """尝试清理粉尘持仓"""
        dust_positions = self.identify_dust_positions()
        cleaned_count = 0
        
        for dust_info in dust_positions:
            symbol = dust_info['symbol']
            
            # 只有持有时间超过24小时的粉尘才尝试清理
            if dust_info['holding_hours'] >= 24:
                print(f"🧹 尝试清理粉尘持仓: {symbol}")
                
                # 使用修复的卖出管理器尝试卖出
                if self.bot.sell_manager.safe_sell_position(
                    symbol, dust_info['quantity'], "粉尘清理"
                ):
                    cleaned_count += 1
                    print(f"✅ 成功清理粉尘持仓: {symbol}")
                else:
                    print(f"❌ 清理粉尘持仓失败: {symbol}")
        
        print(f"📊 粉尘清理完成: 成功清理 {cleaned_count} 个粉尘持仓")
        return cleaned_count

# ========== 持久化持仓数据管理器 - 增强版本 ==========
class PositionPersistenceManager:
    """持久化持仓数据管理器 - 解决重启后计算不准确问题"""
    
    def __init__(self, bot):
        self.bot = bot
        self.persistence_file = "position_persistence.json"
        self.backup_dir = "position_backups"
        
        if not os.path.exists(self.backup_dir):
            os.makedirs(self.backup_dir)
    
    def save_position_details(self):
        """保存完整的持仓信息 - 增强版本"""
        try:
            position_data = {
                'save_time': datetime.now().isoformat(),
                'positions': {}
            }
            
            for symbol, position in self.bot.position_manager.positions.items():
                position_data['positions'][symbol] = {
                    'symbol': symbol,
                    'quantity': position['quantity'],
                    'entry_price': position['entry_price'],
                    'total_invested': position.get('total_invested', 0),
                    'entry_time': position.get('entry_time', datetime.now()).isoformat(),
                    'original_entry_time': position.get('original_entry_time', position.get('entry_time', datetime.now())).isoformat(),
                    'original_entry_price': position.get('original_entry_price', position.get('entry_price', 0)),
                    'is_recovered': position.get('recovered', False),
                    'recovery_time': position.get('recovery_time', None),
                    'cost_known': position.get('cost_known', True), # 🆕 标记成本价是否准确
                    'data_source': 'program' if not position.get('recovered') else 'recovered' # 🆕 数据来源
                }
            
            with open(self.persistence_file, 'w', encoding='utf-8') as f:
                json.dump(position_data, f, indent=2, ensure_ascii=False)
            
            self._create_position_backup(position_data)
            print(f"✅ 持仓数据持久化完成: 保存了 {len(position_data['positions'])} 个持仓的详细信息")
            return True
            
        except Exception as e:
            print(f"❌ 持仓数据持久化失败: {e}")
            return False
    
    def load_position_details(self):
        """加载持仓信息 - 增强版本"""
        try:
            if not os.path.exists(self.persistence_file):
                print("🆕 无持久化持仓数据文件")
                return False
            
            with open(self.persistence_file, 'r', encoding='utf-8') as f:
                position_data = json.load(f)
            
            print("🔄 加载持久化持仓数据...")
            
            loaded_count = 0
            accuracy_improved_count = 0
            
            for symbol, saved_position in position_data['positions'].items():
                if symbol in self.bot.position_manager.positions:
                    # 恢复原始买入时间和成本价
                    current_position = self.bot.position_manager.positions[symbol]
                    
                    # 🆕 增强：只有当持久化数据更准确时才恢复
                    current_cost_known = current_position.get('cost_known', True)
                    saved_cost_known = saved_position.get('cost_known', True)
                    
                    # 如果持久化数据成本已知而当前数据成本未知，则恢复
                    if saved_cost_known and not current_cost_known:
                        # 恢复原始买入时间
                        if 'original_entry_time' in saved_position:
                            try:
                                original_time = datetime.fromisoformat(saved_position['original_entry_time'])
                                current_position['original_entry_time'] = original_time
                                current_position['entry_time'] = original_time # 使用原始时间
                            except:
                                pass
                        
                        # 恢复原始成本价
                        if 'original_entry_price' in saved_position:
                            current_position['original_entry_price'] = saved_position['original_entry_price']
                            current_position['entry_price'] = saved_position['original_entry_price'] # 使用原始价格
                        
                        current_position['cost_known'] = True
                        accuracy_improved_count += 1
                        print(f"✅ 提高 {symbol} 成本价准确性")
                    
                    loaded_count += 1
            
            print(f"✅ 持仓数据恢复完成: 恢复了 {loaded_count} 个持仓的原始数据，提高了 {accuracy_improved_count} 个持仓的准确性")
            return True
            
        except Exception as e:
            print(f"❌ 加载持仓数据失败: {e}")
            return False
    
    def _get_actual_cost_price_from_history(self, asset):
        """从交易历史获取实际成本价 - 增强版本"""
        try:
            # 尝试从币安API获取交易历史
            symbol_variants = [f"{asset}USDT", f"{asset}BUSD"]
            
            for symbol in symbol_variants:
                try:
                    # 获取最近的历史订单
                    orders = self.bot.data_collector.client.get_all_orders(symbol=symbol, limit=10)
                    
                    buy_orders = []
                    for order in orders:
                        if (order['status'] == 'FILLED' and 
                            order['side'] == 'BUY' and
                            float(order['executedQty']) > 0):
                            
                            executed_qty = float(order['executedQty'])
                            # 计算平均成交价格
                            if 'fills' in order and order['fills']:
                                total_cost = 0
                                total_qty = 0
                                for fill in order['fills']:
                                    fill_qty = float(fill['qty'])
                                    fill_price = float(fill['price'])
                                    total_cost += fill_qty * fill_price
                                    total_qty += fill_qty
                                
                                if total_qty > 0:
                                    avg_price = total_cost / total_qty
                                    buy_orders.append({
                                        'quantity': executed_qty,
                                        'price': avg_price,
                                        'time': datetime.fromtimestamp(order['time'] / 1000)
                                    })
                            else:
                                # 如果没有fills信息，使用order价格
                                price = float(order['price'])
                                buy_orders.append({
                                    'quantity': executed_qty,
                                    'price': price,
                                    'time': datetime.fromtimestamp(order['time'] / 1000)
                                })
                    
                    if buy_orders:
                        # 计算加权平均成本
                        total_investment = 0
                        total_quantity = 0
                        for order in buy_orders:
                            total_investment += order['quantity'] * order['price']
                            total_quantity += order['quantity']
                        
                        if total_quantity > 0:
                            weighted_avg_price = total_investment / total_quantity
                            print(f"✅ 从交易历史获取 {symbol} 实际成本价: ${weighted_avg_price:.6f}")
                            return weighted_avg_price
                            
                except Exception as e:
                    if "Invalid symbol" not in str(e):
                        print(f"⚠️ 获取 {symbol} 交易历史失败: {e}")
                    continue
            
            return None
            
        except Exception as e:
            print(f"❌ 获取实际成本价失败: {e}")
            return None
    
    def calculate_holding_performance(self, symbol):
        """基于实际买入数据计算持仓表现 - 增强版本"""
        if symbol not in self.bot.position_manager.positions:
            return None
        
        position = self.bot.position_manager.positions[symbol]
        current_price = self.bot.data_collector.get_price(symbol)
        
        if not current_price or current_price <= 0:
            return None
        
        # 使用保存的原始数据计算
        entry_price = position.get('original_entry_price', position['entry_price'])
        entry_time_str = position.get('original_entry_time', position.get('entry_time'))
        
        # 安全解析时间
        if isinstance(entry_time_str, str):
            try:
                entry_time = datetime.fromisoformat(entry_time_str.replace('Z', '+00:00'))
            except:
                entry_time = datetime.now()
        else:
            entry_time = entry_time_str
        
        # 计算持仓时长
        holding_hours = (datetime.now() - entry_time).total_seconds() / 3600
        
        # 计算盈亏
        current_pnl = (current_price - entry_price) / entry_price
        current_pnl_amount = (current_price - entry_price) * position['quantity']
        
        return {
            'symbol': symbol,
            'current_price': current_price,
            'entry_price': entry_price,
            'quantity': position['quantity'],
            'current_value': position['quantity'] * current_price,
            'invested_value': position['quantity'] * entry_price,
            'pnl_percent': current_pnl,
            'pnl_amount': current_pnl_amount,
            'holding_hours': holding_hours,
            'entry_time': entry_time.isoformat(),
            'is_accurate': position.get('cost_known', True) # 🆕 使用cost_known字段
        }
    
    def generate_recovery_validation_report(self):
        """生成恢复验证报告 - 新增功能"""
        try:
            report = {
                'recovery_time': datetime.now().isoformat(),
                'positions_recovered': len(self.bot.position_manager.positions),
                'validation_results': []
            }
            
            for symbol, position in self.bot.position_manager.positions.items():
                performance = self.calculate_holding_performance(symbol)
                is_accurate = position.get('cost_known', True)
                
                report['validation_results'].append({
                    'symbol': symbol,
                    'quantity': position['quantity'],
                    'entry_price': position['entry_price'],
                    'current_price': performance['current_price'] if performance else 0,
                    'pnl_percent': performance['pnl_percent'] if performance else 0,
                    'holding_hours': performance['holding_hours'] if performance else 0,
                    'data_accuracy': '高' if is_accurate else '估算',
                    'recovered_from_persistence': 'original_entry_price' in position,
                    'cost_known': is_accurate
                })
            
            # 计算统计信息
            accurate_positions = sum(1 for result in report['validation_results'] if result['data_accuracy'] == '高')
            total_positions = len(report['validation_results'])
            accuracy_rate = (accurate_positions / total_positions * 100) if total_positions > 0 else 0
            
            report['accuracy_statistics'] = {
                'total_positions': total_positions,
                'accurate_positions': accurate_positions,
                'estimated_positions': total_positions - accurate_positions,
                'accuracy_rate': accuracy_rate
            }
            
            return report
            
        except Exception as e:
            print(f"❌ 生成恢复验证报告失败: {e}")
            return None
    
    def _create_position_backup(self, position_data):
        """创建持仓数据备份"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_file = os.path.join(self.backup_dir, f"position_backup_{timestamp}.json")
            
            with open(backup_file, 'w', encoding='utf-8') as f:
                json.dump(position_data, f, indent=2, ensure_ascii=False)
            
        except Exception as e:
            print(f"⚠️ 创建持仓备份失败: {e}")
# ========== 主动持仓优化器 - 新增类 ==========
class ActivePortfolioOptimizer:
    """主动持仓优化器 - 用表现更好的币种替换表现差的持仓"""

    def __init__(self, bot):
        self.bot = bot
        self.replacement_threshold = 20 # 评分差超过20分才替换
        self.min_holding_hours_for_replacement = 12 # 12小时后才允许替换
        self.max_replacements_per_cycle = 2 # 每周期最多替换2个持仓
        self.last_optimization = None

    def optimize_portfolio(self):
        """主动优化持仓组合 - 使用统一评分方法"""
        try:
            if len(self.bot.position_manager.positions) == 0:
                return

            print(" 执行主动持仓优化（统一评分版）...")

            # 获取当前持仓评分 - 使用统一的实时评分方法
            position_scores = {}
            for symbol in self.bot.position_manager.positions:
                # 检查持仓时间（保持原有时间设置）
                holding_hours = self.bot.holding_manager.get_holding_hours(symbol)
            
                # 原有的时间保护检查保持不变
                if holding_hours < self.min_holding_hours_for_replacement:
                    print(f"⏳ {symbol} 持仓保护: 仅持有{holding_hours:.1f}小时 (<{self.min_holding_hours_for_replacement}小时)，跳过优化")
                    continue
            
                # 修改这里：使用统一的实时评分方法
                score = self._calculate_unified_real_time_score(symbol)
                current_pnl = self._get_current_pnl(symbol)
            
                position_scores[symbol] = {
                    'score': score,
                    'holding_hours': holding_hours,
                    'current_pnl': current_pnl,
                    'symbol': symbol
                }

            # 如果所有持仓都被保护，直接返回
            if not position_scores:
                print("✅ 所有持仓都在保护期内，跳过本次优化")
                return

            # 修改这里：使用统一评分的候选币种
            candidates = self._get_real_time_candidates()
        
            if not candidates:
                print("⚠️ 无合适候选币种，跳过优化")
                return

            # 找出需要替换的持仓
            replacements = self._find_replacement_opportunities(
                position_scores, candidates
            )

            # 执行替换
            replacements_executed = 0
            for old_symbol, new_candidate in replacements:
                if replacements_executed >= self.max_replacements_per_cycle:
                    break
                if self._execute_replacement(old_symbol, new_candidate):
                    replacements_executed += 1

            if replacements_executed > 0:
                print(f"✅ 主动持仓优化完成：替换了 {replacements_executed} 个持仓")
            else:
                print("✅ 主动持仓优化完成：无需要替换的持仓")

            self.last_optimization = datetime.now()

        except Exception as e:
            print(f"❌ 持仓优化执行失败: {e}")

    def _calculate_current_position_score(self, symbol):
        """计算当前持仓的实时评分"""
        try:
            # 获取K线数据
            klines = self.bot.data_collector.get_klines(symbol, '1h', 100)
            if klines is None or len(klines) < 50:
                return 0

            # 使用选币器的分析方法计算当前持仓的评分
            analysis = self.bot.coin_selector._enhanced_coin_analysis(
                symbol, klines, self.bot.current_market_state
            )
            if analysis:
                return analysis['score']
            else:
                return 0
        except Exception as e:
            print(f"❌ 计算持仓 {symbol} 评分失败: {e}")
            return 0

    def _get_current_pnl(self, symbol):
        """获取当前盈亏"""
        try:
            position = self.bot.position_manager.positions[symbol]
            current_price = self.bot.data_collector.get_price(symbol)
            if current_price and position['entry_price'] > 0:
                return (current_price - position['entry_price']) / position['entry_price']
            return 0
        except:
            return 0

    def _find_replacement_opportunities(self, position_scores, candidates):
        """找出替换机会 - 使用统一评分"""
        replacements = []

        # 按评分排序持仓（从差到好）
        sorted_positions = sorted(position_scores.items(), 
                                 key=lambda x: x[1]['score'])
    
        # 按评分排序候选币种（从好到差）
        sorted_candidates = sorted(candidates, 
                                 key=lambda x: x['score'], reverse=True)

        for old_symbol, position_info in sorted_positions:
            # 原有的时间检查保持不变
            holding_hours = position_info['holding_hours']
            if holding_hours < self.min_holding_hours_for_replacement:
                continue

            # 如果是盈利的持仓，设置更高的替换阈值
            current_threshold = self.replacement_threshold
            if position_info['current_pnl'] > 0.05:  # 盈利超过5%
                current_threshold = self.replacement_threshold + 10

            for candidate in sorted_candidates:
                if candidate['symbol'] in self.bot.position_manager.positions:
                    continue  # 跳过已持有的

                # 检查评分差异是否达到阈值
                score_diff = candidate['score'] - position_info['score']
                if score_diff > current_threshold:
                    replacements.append((old_symbol, candidate))
                    print(f" 发现替换机会: {old_symbol}({position_info['score']:.1f}, {position_info['holding_hours']:.1f}小时) -> {candidate['symbol']}({candidate['score']:.1f})")
                    break  # 每个差持仓只替换一次

        return replacements

    def _execute_replacement(self, old_symbol, new_candidate):
        """执行替换：卖出旧持仓，买入新候选"""
        try:
            print(f"🔄 执行替换: {old_symbol} -> {new_candidate['symbol']}")

            # 卖出旧持仓
            position = self.bot.position_manager.positions[old_symbol]
            sell_success = self.bot.sell_manager.safe_sell_position(
                old_symbol, position['quantity'], 
                f"主动优化替换为 {new_candidate['symbol']}"
            )

            if not sell_success:
                print(f"❌ 卖出 {old_symbol} 失败，替换中止")
                return False

            # 等待资金结算
            time.sleep(2)

            # 买入新候选
            symbol = new_candidate['symbol']
            price = new_candidate['current_price']
            
            # 计算投资金额（使用可用资金）
            available_funds = self.bot.live_capital_manager.get_available_trading_capital()
            position_size = self.bot.position_manager.get_position_size(self.bot.current_market_state)
            investment = min(available_funds, position_size)

            if investment < self.bot.order_manager.min_investment:
                print(f"💰 资金不足，无法买入 {symbol}")
                return False

            quantity = investment / price

            # 调整数量精度
            adjusted_quantity, adjusted_investment = self.bot.order_manager._enhanced_adjust_quantity(
                symbol, quantity, price, 'BUY'
            )

            if adjusted_quantity <= 0:
                print(f"⚠️ 调整后数量无效: {symbol}")
                return False

            # 创建买入订单
            buy_order = self.bot.order_manager.create_buy_order(
                symbol, adjusted_quantity, price,
                self.bot.position_manager, self.bot.live_capital_manager
            )

            if buy_order:
                self.bot.holding_manager.record_buy(symbol)
                print(f"✅ 主动优化替换完成: {old_symbol} -> {symbol}")
                return True
            else:
                print(f"❌ 买入 {symbol} 失败")
                return False

        except Exception as e:
            print(f"❌ 执行替换 {old_symbol} -> {new_candidate['symbol']} 失败: {e}")
            return False
    def _calculate_unified_real_time_score(self, symbol):
        """统一实时评分方法 - 与选币器使用相同的评分逻辑"""
        try:
            # 获取实时K线数据
            klines = self.bot.data_collector.get_klines(symbol, '1h', 100)
            if klines is None or len(klines) < 50:
                print(f"⚠️ {symbol} 数据不足，无法计算实时评分")
                return 0

            # 使用与选币器完全相同的分析方法
            analysis = self.bot.coin_selector._enhanced_coin_analysis(
                symbol, klines, self.bot.current_market_state
            )
            
            if analysis:
                print(f"📊 {symbol} 实时评分: {analysis['score']:.1f} "
                      f"(技术{analysis['technical_score']:.1f}+"
                      f"基本面{analysis['fundamental_score']:.1f}+"
                      f"资金面{analysis['volume_score']:.1f})")
                return analysis['score']
            else:
                print(f"❌ {symbol} 实时评分失败")
                return 0
                
        except Exception as e:
            print(f"❌ 计算 {symbol} 统一实时评分失败: {e}")
            return 0

    def _get_real_time_candidates(self):
        """获取实时候选币种 - 使用统一评分方法"""
        try:
            # 获取候选币种列表
            candidates = self.bot.coin_selector.professional_screening(
                self.bot.current_market_state, 
                list(self.bot.position_manager.positions.keys())
            )
            
            # 为每个候选币种计算实时评分
            real_time_candidates = []
            for candidate in candidates:
                symbol = candidate['symbol']
                
                # 跳过已持有的币种
                if symbol in self.bot.position_manager.positions:
                    continue
                
                # 计算实时评分
                real_time_score = self._calculate_unified_real_time_score(symbol)
                
                # 更新候选币种的评分
                updated_candidate = candidate.copy()
                updated_candidate['score'] = real_time_score
                updated_candidate['real_time_price'] = self.bot.data_collector.get_price(symbol)
                
                real_time_candidates.append(updated_candidate)
                
            print(f"✅ 实时候选币种评分完成: {len(real_time_candidates)}个币种")
            return real_time_candidates
            
        except Exception as e:
            print(f"❌ 获取实时候选币种失败: {e}")
            return []        
# ========== 动态间隔管理器 - 新增类 ==========
class AdaptiveIntervalManager:
    """动态间隔管理器 - 根据市场状态调整监控频率"""

    def __init__(self, bot):
        self.bot = bot
        self.last_interval_update = datetime.now()
        self.current_intervals = self._get_default_intervals()

    def get_current_intervals(self):
        """获取当前监控间隔"""
        # 每6小时更新一次间隔设置
        current_time = datetime.now()
        if (current_time - self.last_interval_update).total_seconds() > 21600: # 6小时
            self.current_intervals = self._calculate_adaptive_intervals()
            self.last_interval_update = current_time
        
        return self.current_intervals

    def _calculate_adaptive_intervals(self):
        """计算自适应间隔"""
        try:
            market_volatility = self._calculate_market_volatility()
            market_state = self.bot.current_market_state
            
            print(f"📊 计算自适应间隔: 状态={market_state}, 波动率={market_volatility:.3f}")

            if market_state == 'bull':
                if market_volatility > 0.10: # 高波动牛市
                    return {
                        'main_cycle': 900, # 15分钟
                        'realtime_monitor': 300, # 5分钟
                        'portfolio_optimize': 1800, # 30分钟
                        'market_analysis': 1800 # 30分钟
                    }
                else: # 平稳牛市
                    return {
                        'main_cycle': 1800, # 30分钟
                        'realtime_monitor': 600, # 10分钟
                        'portfolio_optimize': 3600, # 60分钟
                        'market_analysis': 3600 # 60分钟
                    }
            elif market_state == 'bear':
                return {
                    'main_cycle': 1200, # 20分钟
                    'realtime_monitor': 300, # 5分钟
                    'portfolio_optimize': 3600, # 60分钟
                    'market_analysis': 1800 # 30分钟
                }
            else: # 震荡市
                return {
                    'main_cycle': 1800, # 30分钟
                    'realtime_monitor': 600, # 10分钟
                    'portfolio_optimize': 7200, # 120分钟
                    'market_analysis': 3600 # 60分钟
                }

        except Exception as e:
            print(f"❌ 计算自适应间隔失败: {e}")
            return self._get_default_intervals()

    def _calculate_market_volatility(self):
        """计算市场波动率"""
        try:
            # 使用BTC的1小时K线计算波动率
            btc_data = self.bot.data_collector.get_klines('BTCUSDT', '1h', 24)
            if btc_data is None or len(btc_data) < 24:
                return 0.05

            returns = btc_data['close'].pct_change().dropna()
            volatility = returns.std()
            return volatility
        except:
            return 0.05

    def _get_default_intervals(self):
        """获取默认间隔"""
        return {
            'main_cycle': 1800, # 30分钟
            'realtime_monitor': 600, # 10分钟
            'portfolio_optimize': 3600, # 60分钟
            'market_analysis': 3600 # 60分钟
        }

# ========== 完整优化版交易机器人 - 最终修复版本 ==========
class FullyOptimizedTradingBot:
    """完整优化版交易机器人 - 融合所有优化和修复"""
    
    def __init__(self):
        print("🛡️ 初始化实盘就绪版交易机器人 v8.1 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化...")
        self.start_time = datetime.now()
        self.is_running = True
        
        # 选择交易模式
        self.mode_selector = TradingModeSelector()
        self.mode_selector.select_mode()
        
        # 初始化核心组件
        self.data_collector = LongRunDataCollector(self.mode_selector)
        
        # 连接交易所
        print("🔄 正在连接交易所...")
        if not self.data_collector.connect_exchange():
            print("❌ 交易所连接失败，程序无法继续")
            return
        
        # 实盘资金管理器 - 修正版本
        self.live_capital_manager = LiveTradingCapitalManager(self.data_collector, self.mode_selector)
        initial_usdt = self.live_capital_manager.initialize_capital()
        
        print(f"💰 资金初始化: 总资产${self.live_capital_manager.total_assets:.2f}, "
              f"USDT余额${initial_usdt:.2f}, 交易资金${self.live_capital_manager.trading_capital:.2f} (50%)")
        
        # 初始化动态币种列表管理器 - 使用修复版本（只交易USDT交易对）
        self.coin_list_manager = DynamicCoinListManager(self.data_collector)
        self.coin_list_manager.initialize_coin_list()
        
        # 初始化交易组件 - 使用增强订单管理器
        self.order_manager = OrderManager(self.data_collector)
        self.position_manager = SmartPositionManager(initial_balance=initial_usdt)
        self.position_manager.set_price_getter(self.data_collector.get_price)

        # 初始化分析组件 - 使用优化版本
        self.market_analyst = EnhancedMarketStateAnalyzer(self.data_collector, self.coin_list_manager) # 修改这里
        self.coin_selector = EnhancedCoinSelector(self.data_collector, self.coin_list_manager)
        
        # 使用修复的风险管理器 - 替换原来的EnhancedDynamicRiskManager
        self.risk_manager = FixedRiskManager(self.data_collector)
        self.holding_manager = PositionHoldingManager(min_holding_hours=8) # 优化为8小时
        
        # 初始化修复的卖出管理器
        self.sell_manager = FixedSellManager(self)
        self.risk_manager.set_sell_manager(self.sell_manager)
        
        # 初始化全仓风险控制器
        self.portfolio_risk_controller = PortfolioRiskController(self)
        
        # 优化组件
        self.transaction_manager = TransactionalFundManager(self.position_manager)
        self.performance_monitor = PerformanceMonitor()
        self.error_handler = HierarchicalErrorHandler(self)
        self.state_manager = EnhancedStateManager()
        self.emergency_stop_manager = EmergencyStopManager(self)
        
        # 智能灾难恢复系统 - 使用增强版本（包含退市币种屏蔽）
        self.disaster_recovery = SmartDisasterRecoveryManager(self)
        
        # 资产统计
        self.asset_stats_manager = AssetStatisticsManager(self)
        self.asset_stats_manager.initialize_statistics()
        self.account_detail_manager = AccountDetailManager(self)
        
        # 新增：增强重启恢复管理器
        self.restart_manager = EnhancedRestartManager(self)
        
        # 新增：粉尘持仓管理器
        self.dust_manager = DustPositionManager(self)
        
        # 新增：持久化持仓数据管理器 - 增强版本
        self.persistence_manager = PositionPersistenceManager(self)
        
        # 🆕 新增：主动持仓优化器
        self.portfolio_optimizer = ActivePortfolioOptimizer(self)
        
        # 🆕 新增：动态间隔管理器
        self.adaptive_interval_manager = AdaptiveIntervalManager(self)
        
        # 关键修复：强制重新同步
        self._execute_enhanced_recovery_initialization()
        
        # 新增：启动时强制更新币种评分
        print("🔄 启动时强制更新币种评分...")
        self.coin_list_manager.last_update = None
        self.coin_list_manager.update_coin_scores()
        
        # 初始化状态变量
        self.current_market_state = "consolidation"
        self.trade_count = 0
        self.continuous_errors = 0
        self.max_continuous_errors = 3 # 优化为3次
        
        # 打印控制变量
        self.last_status_print = datetime.now() - timedelta(minutes=10)
        self.status_print_interval = 5
        
        print("✅ 实盘就绪版交易机器人 v8.1 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化 初始化完成!")
        
        # 打印初始详细账户报告
        self.account_detail_manager.print_detailed_account_report("初始状态")
        
        self._print_enhanced_system_info()
    
    def _get_actual_cost_price(self, symbol, position):
        """获取实际成本价 - 简化版本"""
        try:
            # 首先检查是否有准确标记的成本价
            if position.get('cost_known', False):
                return position.get('original_entry_price', position['entry_price'])
        
            # 尝试从持久化数据获取
            persisted_data = self.persistence_manager._get_persisted_position_data(symbol)
            if persisted_data and persisted_data.get('cost_known'):
                return persisted_data.get('original_entry_price')
        
            # 最后使用持仓记录的成本价
            return position['entry_price']
        
        except Exception as e:
            print(f"❌ 获取成本价失败 {symbol}: {e}")
            return position['entry_price']
   
    def generate_immediate_profit_summary(self):
        """立即生成盈亏汇总报告 - 基于实际买入价计算"""
        print("\n📊 实时盈亏汇总报告")
        print("-" * 50)
    
        try:
            total_invested = 0
            total_current_value = 0
            position_count = 0
            profit_positions = 0
            loss_positions = 0
        
            print("各币种盈亏详情:")
            print("币种 数量 成本价 现价 投资额 当前价值 盈亏% 持仓时间")
            print("-" * 80)
        
            for symbol, position in self.position_manager.positions.items():
                # 跳过粉尘持仓
                if self.dust_manager.should_skip_for_trading(symbol):
                    continue
                
                # 获取当前价格
                current_price = self.data_collector.get_price(symbol)
                if not current_price or current_price <= 0:
                    continue
            
                # 获取实际成本价
                quantity = position['quantity']
                actual_cost_price = self._get_actual_cost_price(symbol, position)
            
                # 计算各项数值
                invested = quantity * actual_cost_price
                current_value = quantity * current_price
                pnl_amount = current_value - invested
                pnl_percent = (pnl_amount / invested * 100) if invested > 0 else 0
            
                # 获取持仓时间
                holding_hours = self.holding_manager.get_holding_hours(symbol)
            
                # 统计盈亏
                if pnl_amount > 0:
                    profit_positions += 1
                elif pnl_amount < 0:
                    loss_positions += 1
            
                total_invested += invested
                total_current_value += current_value
                position_count += 1
            
                # 打印单个币种详情
                print(f"{symbol:10} {quantity:8.4f} ${actual_cost_price:7.4f} "
                      f"${current_price:7.4f} ${invested:6.2f} ${current_value:6.2f} "
                      f"{pnl_percent:+.2f}% {holding_hours:6.1f}h")
        
            # 计算总体盈亏
            total_pnl = total_current_value - total_invested
            total_pnl_percent = (total_pnl / total_invested * 100) if total_invested > 0 else 0
        
            print("-" * 80)
            print(f"📈 汇总统计:")
            print(f" 总持仓数量: {position_count}个")
            print(f" 盈利持仓: {profit_positions}个")
            print(f" 亏损持仓: {loss_positions}个")
            print(f" 总投入: ${total_invested:.2f}")
            print(f" 当前总值: ${total_current_value:.2f}")
            print(f" 总盈亏: ${total_pnl:+.2f} ({total_pnl_percent:+.2f}%)")
        
            # 计算胜率
            win_rate = (profit_positions / position_count * 100) if position_count > 0 else 0
            print(f" 胜率: {win_rate:.1f}%")
        
        except Exception as e:
            print(f"❌ 生成盈亏汇总失败: {e}")
    
    def print_decision_ready_status(self):
        """打印便于决策的当前状态"""
        print("\n🎯 决策准备状态")
        print("-" * 40)
    
        try:
            # 获取当前市场状态
            market_state = self.current_market_state
        
            # 计算有效持仓（排除粉尘）
            effective_positions = self.dust_manager.get_effective_positions_count()
            max_positions = self.position_manager.get_max_positions(market_state)
        
            # 获取可用资金
            available_funds = self.live_capital_manager.get_available_trading_capital()
            min_investment = self.order_manager.min_investment
        
            # 计算可用仓位
            available_slots = max(0, max_positions - effective_positions)
        
            print(f"🏷️ 市场状态: {market_state.upper()}")
            print(f"📊 持仓情况: {effective_positions}/{max_positions} (有效/最大)")
            print(f"💰 可用资金: ${available_funds:.2f}")
            print(f"🎯 可用仓位: {available_slots}个")
            print(f"💵 最小投资: ${min_investment:.2f}")
        
            # 决策建议
            print("\n🤖 决策建议:")
        
            if available_slots > 0 and available_funds >= min_investment:
                print(" ✅ 可以加仓 - 有可用仓位和资金")
            else:
                if available_slots <= 0:
                    print(" ⏹️ 无法加仓 - 持仓已满")
                if available_funds < min_investment:
                    print(" ⏹️ 无法加仓 - 资金不足")
        
            # 检查是否需要调仓
            if effective_positions > 0:
                print(" 🔄 可以考虑调仓 - 有持仓可优化")
            else:
                print(" ➕ 建议寻找新机会 - 无持仓")
            
            # 粉尘情况
            dust_count = len(self.dust_manager.dust_positions)
            if dust_count > 0:
                print(f" 🧹 需要清理粉尘 - {dust_count}个粉尘持仓")
        
        except Exception as e:
            print(f"❌ 决策状态分析失败: {e}")
    
    def after_sell_immediate_check(self, sold_symbol, sell_reason):
        """卖出后立即检查：更新账户、过滤粉尘、汇总盈亏"""
        print(f"\n🎯 卖出后立即检查: {sold_symbol} - {sell_reason}")
        print("=" * 60)
    
        try:
            # 1. 立即更新资金情况
            print("1. 更新资金情况...")
            self.update_available_balance()
        
            # 2. 立即过滤粉尘持仓
            print("2. 过滤粉尘持仓...")
            dust_count = self.dust_manager.update_dust_positions()
            if dust_count > 0:
                print(f" 发现 {dust_count} 个粉尘持仓")
        
            # 3. 生成详细的盈亏汇总报告
            print("3. 生成盈亏汇总报告...")
            self.generate_immediate_profit_summary()
        
            # 4. 打印当前状态，便于后续决策
            print("4. 当前状态总结...")
            self.print_decision_ready_status()
        
            print("✅ 卖出后检查完成!")
            print("=" * 60)
        
        except Exception as e:
            print(f"❌ 卖出后检查失败: {e}")
   
    def safe_sell_and_check(self, symbol, quantity, reason="卖出"):
        """统一的卖出方法，包含卖出后检查"""
        try:
            print(f"🔄 执行卖出: {symbol}, 原因: {reason}")
        
            # 获取当前价格
            current_price = self.data_collector.get_price(symbol)
            if not current_price or current_price <= 0:
                print(f"❌ 无法获取 {symbol} 价格")
                return False
        
            # 使用 OrderManager 执行卖出
            sell_order = self.order_manager.create_sell_order(
                symbol, 
                quantity, 
                current_price,
                self.position_manager, 
                self.live_capital_manager
            )
        
            if sell_order:
                # 🆕 卖出后立即检查
                self.after_sell_immediate_check(symbol, reason)
                return True
            else:
                print(f"❌ 卖出失败: {symbol}")
                return False
            
        except Exception as e:
            print(f"❌ 卖出异常 {symbol}: {e}")
            return False

    def _execute_enhanced_recovery_initialization(self):
        """执行增强的恢复初始化"""
        print("🔄 执行增强恢复初始化...")
        
        # 🆕 首先加载持久化数据
        self.persistence_manager.load_position_details()
        
        # 然后执行安全重启恢复
        restart_success = self.restart_manager.safe_restart()
        
        if not restart_success:
            print("🚨 安全重启恢复失败，执行智能灾难恢复...")
            recovery_success = self.disaster_recovery.execute_disaster_recovery()
            
            if not recovery_success:
                print("🚨 智能灾难恢复失败，执行紧急恢复...")
                self._execute_emergency_recovery()
        
        # 🆕 立即重新计算所有持仓状态
        self._recalculate_all_positions_immediately()
        
        # 重新初始化资产统计
        self.asset_stats_manager.initialize_statistics()
        
        # 🆕 生成恢复验证报告
        validation_report = self.persistence_manager.generate_recovery_validation_report()
        if validation_report:
            self._print_recovery_validation_report(validation_report)
        
        # 验证恢复结果
        self._validate_enhanced_recovery_result()

    def _recalculate_all_positions_immediately(self):
        """重启后立即重新计算所有持仓状态 - 新增功能"""
        print("🔄 立即重新计算所有持仓状态...")
        
        try:
            for symbol in self.position_manager.positions:
                performance = self.persistence_manager.calculate_holding_performance(symbol)
                if performance:
                    print(f"📊 {symbol} 重新计算: 成本${performance['entry_price']:.6f}, "
                          f"当前${performance['current_price']:.6f}, "
                          f"盈亏{performance['pnl_percent']:+.2f}%, "
                          f"持有{performance['holding_hours']:.1f}小时")
            
            print("✅ 所有持仓状态重新计算完成")
            
        except Exception as e:
            print(f"❌ 重新计算持仓状态失败: {e}")

    def _print_recovery_validation_report(self, validation_report):
        """打印恢复验证报告 - 新增功能"""
        print("\n" + "="*80)
        print("📋 恢复数据验证报告")
        print("="*80)
        
        stats = validation_report['accuracy_statistics']
        print(f"📊 数据准确性统计:")
        print(f" 总持仓数量: {stats['total_positions']}个")
        print(f" 准确成本价: {stats['accurate_positions']}个")
        print(f" 估算成本价: {stats['estimated_positions']}个")
        print(f" 数据准确率: {stats['accuracy_rate']:.1f}%")
        
        if stats['estimated_positions'] > 0:
            print(f"\n⚠️ 注意: {stats['estimated_positions']}个持仓使用估算成本价")
            print(" 这些持仓的盈亏计算可能不准确")
            print(" 程序会尝试从交易历史获取实际成本价")
        
        print("="*80 + "\n")

    def _execute_emergency_recovery(self):
        """执行紧急恢复"""
        print("🆘 执行紧急恢复...")
        
        try:
            # 清空所有状态
            self.position_manager.positions = {}
            
            # 从交易所重新获取所有资产
            actual_assets = self.disaster_recovery.orphan_recovery._get_all_actual_assets()
            if actual_assets:
                for asset, quantity in actual_assets.items():
                    symbol, price = self.disaster_recovery.orphan_recovery._find_best_trading_pair(asset)
                    if symbol and price:
                        self.position_manager.positions[symbol] = {
                            'symbol': symbol,
                            'quantity': quantity,
                            'entry_price': price,
                            'total_invested': quantity * price,
                            'entry_time': datetime.now(),
                            'original_entry_time': datetime.now(), # 保存原始时间
                            'original_entry_price': price, # 保存原始价格
                            'emergency_recovered': True,
                            'cost_known': False # 🆕 标记为估算成本
                        }
                        print(f"🆘 紧急恢复: {symbol}")
            
            print("✅ 紧急恢复完成")
            
        except Exception as e:
            print(f"❌ 紧急恢复失败: {e}")

    def _validate_enhanced_recovery_result(self):
        """验证增强恢复结果"""
        print("🔍 验证增强恢复结果...")
        
        actual_count = len(self.disaster_recovery.orphan_recovery._get_all_actual_assets() or {})
        recovered_count = len(self.position_manager.positions)
        
        print(f"📊 增强恢复验证: 实际资产{actual_count}个 vs 恢复持仓{recovered_count}个")
        
        if actual_count == recovered_count:
            print("✅ 增强恢复验证通过!")
        else:
            print(f"⚠️ 增强恢复验证警告: 数量不匹配 (实际{actual_count} vs 恢复{recovered_count})")
            
            # 显示详细信息
            actual_assets = self.disaster_recovery.orphan_recovery._get_all_actual_assets() or {}
            recovered_symbols = set(self.position_manager.positions.keys())
            
            print("🔍 详细对比:")
            for asset in actual_assets:
                found = False
                for symbol in recovered_symbols:
                    if asset in symbol:
                        found = True
                        break
                status = "✅" if found else "❌"
                print(f" {status} {asset}: {actual_assets[asset]:.8f}")
    
    def _print_enhanced_system_info(self):
        """打印增强系统信息"""
        total_assets = self.live_capital_manager.total_assets
        trading_capital = self.live_capital_manager.trading_capital
        running_hours = (datetime.now() - self.start_time).total_seconds() / 3600
        
        print("🎯 增强系统配置信息:")
        print(f" 运行模式: {self.mode_selector.mode.upper()}模式")
        print(f" 总资产: ${total_assets:.2f}")
        print(f" 交易资金: ${trading_capital:.2f} (50%)")
        print(f" 动态币种: {len(self.coin_list_manager.coin_universe)}个USDT交易对 (已过滤稳定币)")
        print(f" 已运行: {running_hours:.1f}小时")
        print(f" 紧急停止: ✅ 已激活")
        print(f" 事务性资金管理: ✅ 已激活")
        print(f" 性能监控: ✅ 已激活")
        print(f" 持仓保护: ✅ 8小时最小持仓")
        print(f" 智能灾难恢复: ✅ 已激活")
        print(f" 智能孤儿持仓恢复: ✅ 已激活")
        print(f" 退市币种屏蔽: ✅ 已激活 (AUTO, NEBL等)")
        print(f" 智能持仓管理: ✅ 已激活")
        print(f" 多级止盈止损: ✅ 已激活")
        print(f" 全仓风险控制: ✅ 已激活")
        print(f" 动态止损: ✅ 已激活")
        print(f" 多维度选币: ✅ 已激活")
        print(f" 板块分散: ✅ 已激活")
        print(f" 🆕 重启修复: ✅ 已激活")
        print(f" 🆕 卖出修复: ✅ 已激活")
        print(f" 🆕 币种评分定期更新: ✅ 已激活 (1小时间隔)")
        print(f" 🆕 真实可交易USDT币种列表: ✅ 已激活 (直接从币安获取)")
        print(f" 🆕 资金面权重60%: ✅ 已激活 (15分钟K线成交量分析)")
        print(f" 🆕 24小时成交量>400万USDT: ✅ 已激活")
        print(f" 🆕 稳定币过滤: ✅ 已激活 (过滤USDCUSDT, BUSDUSDT等)")
        print(f" 🆕 增强精度处理: ✅ 已激活 (避免买入卖出失败)")
        print(f" 🆕 粉尘持仓管理: ✅ 已激活 (<$0.3自动识别)")
        print(f" 🆕 持仓数据持久化: ✅ 已激活 (解决重启计算问题)")
        print(f" 🆕 200币种市场广度分析: ✅ 已激活 (更准确的市场状态判断)")
        print(f" 🆕 增强数据持久化: ✅ 已激活 (交易后立即保存)")
        print(f" 🆕 成本价准确性标记: ✅ 已激活 (区分已知/估算成本)")
        print(f" 🆕 恢复验证报告: ✅ 已激活 (数据准确性统计)")
        print(f" 🆕 主动持仓优化: ✅ 已激活 (12小时保护, 20分阈值)")
        print(f" 🆕 动态频率调整: ✅ 已激活 (根据波动性调整)")
        print(f" 🆕 独立市场分析: ✅ 已激活 (独立于主策略周期)")
        
        api_stats = self.data_collector.get_api_health_report()
        print(f" API成功率: {api_stats['success_rate']}")
    
    def calculate_total_assets(self):
        """统一计算总资产 - 修正逻辑"""
        try:
            if self.mode_selector.mode == 'live':
                total_assets = self.live_capital_manager.total_assets
                
                self.position_manager.total_balance = total_assets
                
                positions_value = 0
                for symbol, position in self.position_manager.positions.items():
                    current_price = self.data_collector.get_price(symbol)
                    if current_price and current_price > 0:
                        positions_value += position['quantity'] * current_price
                
                self.position_manager.available_balance = max(0, total_assets - positions_value)
                self.position_manager.update_allocated_capital(self.live_capital_manager)
                
                return total_assets
            else:
                positions_value = 0
                for symbol, position in self.position_manager.positions.items():
                    current_price = self.data_collector.get_price(symbol)
                    if current_price and current_price > 0:
                        positions_value += position['quantity'] * current_price
                
                total_assets = self.position_manager.available_balance + positions_value
                
                self.live_capital_manager.total_assets = total_assets
                self.live_capital_manager.usdt_balance = self.position_manager.available_balance
                self.live_capital_manager.trading_capital = total_assets * 0.5
                self.live_capital_manager.allocated_capital = positions_value
                self.position_manager.total_balance = total_assets
                
                return total_assets
                
        except Exception as e:
            print(f"❌ 计算总资产失败: {e}")
            return self.live_capital_manager.total_assets
    
    def update_available_balance(self):
        """统一更新可用资金"""
        try:
            total_assets = self.calculate_total_assets()
            
            positions_value = 0
            for symbol, position in self.position_manager.positions.items():
                current_price = self.data_collector.get_price(symbol)
                if current_price and current_price > 0:
                    positions_value += position['quantity'] * current_price
            
            self.position_manager.available_balance = max(0, total_assets - positions_value)
            
            print(f"💰 资金更新: 总资产${total_assets:.2f}, 持仓价值${positions_value:.2f}, 可用资金${self.position_manager.available_balance:.2f}")
            
        except Exception as e:
            print(f"❌ 更新可用资金失败: {e}")
    
    def enhanced_coin_validation(self, symbol):
        """增强币种验证 - 确保是USDT交易对"""
        try:
            # 🛠️ 关键修复：首先检查是否是USDT交易对
            if not symbol.endswith('USDT'):
                return False, "非USDT交易对"
            
            # 🚫 检查是否在稳定币黑名单中
            stable_blacklist = [
                'USDCUSDT', 'BUSDUSDT', 'TUSDUSDT', 'USDPUSDT', 'DAIUSDT', 
                'FDUSDUSDT'
            ]
            if symbol in stable_blacklist:
                return False, "稳定币交易对"
            
            price = self.data_collector.get_price(symbol)
            if price is None or price <= 0:
                return False, "无法获取价格"
            
            # 获取24小时成交量数据
            klines_24h = self.data_collector.get_klines(symbol, '1h', 24)
            if klines_24h is None or len(klines_24h) < 24:
                return False, "24小时数据不足"
            
            # 🆕 检查24小时成交量 > 400万USDT
            total_volume_24h = (klines_24h['volume'] * klines_24h['close']).sum()
            if total_volume_24h < 4000000: # 400万USDT
                return False, f"24小时成交量不足: ${total_volume_24h:,.0f}"
            
            # 其他验证条件保持不变
            klines_daily = self.data_collector.get_klines(symbol, '1d', 30)
            if klines_daily is None or len(klines_daily) < 20:
                return False, "数据不足"
            
            volatility = klines_daily['close'].pct_change().std()
            if volatility > 0.15:
                return False, f"波动性过高: {volatility:.2%}"
            
            return True, "验证通过"
            
        except Exception as e:
            return False, f"验证异常: {e}"
    
    def _print_condensed_status(self):
        """简洁状态打印"""
        report = self.asset_stats_manager.generate_complete_asset_report()
        running_hours = (datetime.now() - self.start_time).total_seconds() / 3600
        api_stats = self.data_collector.get_api_health_report()
        
        status_line = (f"🔄 [{datetime.now().strftime('%H:%M')}] "
                       f"运行: {running_hours:.1f}h | "
                       f"资产: ${report['total_assets']:.2f} | "
                       f"持仓: {report['positions_count']} | "
                       f"盈亏: {report['total_pnl_percent']:+.2f}% | "
                       f"API: {api_stats['success_rate']}")
        
        print(status_line)
        
        if self.trade_count % 10 == 0:
            buying_power = report['buying_power']
            if buying_power['can_buy_more']:
                print(f" 🛒 可买: {buying_power['available_slots']}仓位, "
                      f"${buying_power['actual_buying_power']:.2f}可用")
    
    def run_continuous_trading(self):
        """运行连续交易 - 增强版（包含主动优化）"""
        print("🚀 启动无限循环交易模式（增强版）...")
        print("💡 新增功能：主动持仓优化 + 动态频率调整")
        
        # 使用动态间隔
        intervals = self.adaptive_interval_manager.get_current_intervals()
        
        main_cycle_interval = intervals['main_cycle']
        realtime_monitor_interval = intervals['realtime_monitor']
        portfolio_optimize_interval = intervals['portfolio_optimize']
        market_analysis_interval = intervals['market_analysis']
        
        health_check_interval = 600 # 健康检查10分钟
        
        # 初始化时间记录
        last_main_run = datetime.now() - timedelta(seconds=main_cycle_interval)
        last_realtime_run = datetime.now() - timedelta(seconds=realtime_monitor_interval)
        last_portfolio_optimize = datetime.now() - timedelta(seconds=portfolio_optimize_interval)
        last_market_analysis = datetime.now() - timedelta(seconds=market_analysis_interval)
        last_health_check = datetime.now()
        last_memory_cleanup = datetime.now()
        last_dust_check = datetime.now()
        last_persistence_save = datetime.now()
        last_interval_update = datetime.now()
        
        cycle_count = 0
        self.continuous_errors = 0
        
        print("🚀 开始增强版无限循环策略运行...")
        print(f"📊 初始监控间隔: 主策略{main_cycle_interval//60}分钟, 实时监控{realtime_monitor_interval//60}分钟, 优化{portfolio_optimize_interval//60}分钟")
        
        while self.is_running:
            current_time = datetime.now()
            
            try:
                # 检查紧急停止
                if self.emergency_stop_manager.check_emergency_stop():
                    print("🚨 紧急停止触发，程序终止")
                    break
                
                # 🆕 每6小时更新一次监控间隔
                if (current_time - last_interval_update).total_seconds() >= 21600: # 6小时
                    intervals = self.adaptive_interval_manager.get_current_intervals()
                    main_cycle_interval = intervals['main_cycle']
                    realtime_monitor_interval = intervals['realtime_monitor']
                    portfolio_optimize_interval = intervals['portfolio_optimize']
                    market_analysis_interval = intervals['market_analysis']
                    last_interval_update = current_time
                    print(f"🔄 更新监控间隔: 主策略{main_cycle_interval//60}分钟, 实时监控{realtime_monitor_interval//60}分钟, 优化{portfolio_optimize_interval//60}分钟")
                
                # 定时详细账户报告
                if self.account_detail_manager.should_print_detailed_report():
                    self.account_detail_manager.print_detailed_account_report("定时报告")
                
                # 健康检查
                if (current_time - last_health_check).total_seconds() >= health_check_interval:
                    perf_health = self.performance_monitor.check_performance_health()
                    if perf_health != "HEALTHY":
                        print(f"⚠️ 性能警告: {perf_health}")
                    last_health_check = current_time
                
                # 内存清理
                if (current_time - last_memory_cleanup).total_seconds() >= 3600:
                    gc.collect()
                    memory_info = psutil.virtual_memory()
                    print(f"🧠 内存维护: 使用{memory_info.percent}%, 可用{memory_info.available//1024//1024}MB")
                    last_memory_cleanup = current_time
                
                # 粉尘持仓检查（每6小时一次）
                if (current_time - last_dust_check).total_seconds() >= 21600: # 6小时
                    self.dust_manager.identify_dust_positions()
                    last_dust_check = current_time
                
                # 持仓数据持久化（每2小时一次）
                if (current_time - last_persistence_save).total_seconds() >= 7200: # 2小时
                    self.persistence_manager.save_position_details()
                    last_persistence_save = current_time
                
                # 控制状态打印频率
                if (current_time - self.last_status_print).total_seconds() >= self.status_print_interval * 60:
                    self._print_condensed_status()
                    self.last_status_print = current_time
                
                # 更新实盘资金
                if self.mode_selector.mode == 'live':
                    self.live_capital_manager.update_trading_capital()
                
                # 🆕 市场状态分析（独立于主策略周期）
                if (current_time - last_market_analysis).total_seconds() >= market_analysis_interval:
                    print("🔄 执行独立市场状态分析...")
                    old_market_state = self.current_market_state
                    
                    # 专业市场分析
                    market_state, confidence, market_score, breadth_result = self.market_analyst.professional_market_analysis()
                    
                    # 更新市场状态
                    if old_market_state != market_state:
                        print(f"🎯 检测到市场状态转换: {old_market_state} → {market_state}")
                        self.current_market_state = market_state
                        self.position_manager.current_market_state = market_state
                        self.handle_market_state_transition(old_market_state, market_state)
                    
                    last_market_analysis = current_time
                
                # 运行主策略
                if (current_time - last_main_run).total_seconds() >= main_cycle_interval:
                    self.run_smart_trading_cycle()
                    last_main_run = current_time
                    cycle_count += 1
                    self.continuous_errors = 0
                
                # 运行实时监控
                if (current_time - last_realtime_run).total_seconds() >= realtime_monitor_interval:
                    self.run_real_time_monitoring(verbose=False)
                    last_realtime_run = current_time
                
                # 🆕 主动持仓优化
                if (current_time - last_portfolio_optimize).total_seconds() >= portfolio_optimize_interval:
                    if len(self.position_manager.positions) > 0: # 只有有持仓时才优化
                        self.portfolio_optimizer.optimize_portfolio()
                    last_portfolio_optimize = current_time
                
                time.sleep(30) # 主循环休眠30秒
                
            except Exception as e:
                self.continuous_errors += 1
                self.error_handler.handle_error('logic_error', e, "主循环执行")
                
                if self.continuous_errors >= self.max_continuous_errors:
                    print("🚨 连续错误过多，进入安全模式")
                    self.state_manager.save_complete_state(self)
                    self.persistence_manager.save_position_details()
                    time.sleep(300)
                    self.continuous_errors = 0
                else:
                    time.sleep(60)

    def run_smart_trading_cycle(self):
        """运行智能交易周期 - 只管理程序识别的持仓"""
        print("🔄 运行智能交易周期...")
        
        self.trade_count += 1
        current_time = datetime.now()
        
        print(f" {'='*60}")
        print(f"🔄 第 {self.trade_count} 次交易决策 - {current_time}")
        print(f"{'='*60}")
        
        try:
            # 交易前账户详情
            print("📊 交易前状态:")
            self.account_detail_manager.print_immediate_funds_snapshot()
            
            total_assets = self.calculate_total_assets()
            trading_capital = self.live_capital_manager.trading_capital
            
            print(f"💰 资金概况: 总资产${total_assets:.2f}, 交易资金${trading_capital:.2f}")
            
            # 专业市场分析 - 现在使用200个币种进行市场广度分析
            market_state, confidence, market_score, breadth_result = self.market_analyst.professional_market_analysis()
            
            # 🛡️ 关键修改：检测市场状态转换并处理
            old_market_state = self.current_market_state
            self.current_market_state = market_state
            self.position_manager.current_market_state = market_state
            
            # 如果市场状态发生变化，执行状态转换处理
            if old_market_state != market_state:
                print(f"🎯 检测到市场状态转换: {old_market_state} → {market_state}")
                self.handle_market_state_transition(old_market_state, market_state)
            
            # 账户报告
            self.print_account_report(total_assets, trading_capital)
            
            # 检查整体止损
            stop_loss_triggered, drawdown = self.position_manager.check_overall_stop_loss()
            if stop_loss_triggered:
                print(f"🚨 触发整体止损，回撤: {drawdown:.2%}")
                self.position_manager.execute_overall_stop_loss(self.data_collector)
                return
            
            # 更新可用资金
            self.update_available_balance()
            
            # 智能持仓限制 - 先清理程序管理的持仓
            position_cleanup_success = self.position_manager.enforce_smart_position_limits(market_state, self)
            if not position_cleanup_success:
                print("🚨 程序持仓清理失败，跳过本次交易周期")
                return
            
            # 重新计算可用资金（清理后）
            total_assets = self.calculate_total_assets()
            true_available = self.live_capital_manager.get_available_trading_capital()
            
            # 检查是否允许买入（只考虑程序管理持仓）
            program_positions_count = self.position_manager.get_program_managed_positions_count()
            max_positions = self.position_manager.get_max_positions(market_state)
            available_slots = max(0, max_positions - program_positions_count)
            
            print(f"📊 智能交易条件检查:")
            print(f" 程序管理持仓: {program_positions_count}/{max_positions}")
            print(f" 可用仓位: {available_slots}")
            print(f" 真实可用资金: ${true_available:.2f}")
            print(f" 最小投资: ${self.order_manager.min_investment:.2f}")
            
            # 只有满足条件才执行买入
            if (available_slots > 0 and 
                true_available >= self.order_manager.min_investment and
                program_positions_count < max_positions):
                
                print("✅ 满足智能买入条件，执行选币逻辑")
                
                # 使用增强选币器，传入当前持仓以实现板块分散
                current_positions = list(self.position_manager.positions.keys())
                candidates = self.coin_selector.professional_screening(market_state, current_positions)
                
                if candidates:
                    self._execute_smart_trading_logic(market_state, true_available, candidates, available_slots)
                else:
                    print("🔍 无合适候选币种")
            else:
                reason = []
                if available_slots <= 0:
                    reason.append("程序管理持仓已达上限")
                if true_available < self.order_manager.min_investment:
                    reason.append(f"资金不足 (${true_available:.2f} < ${self.order_manager.min_investment:.2f})")
                if program_positions_count >= max_positions:
                    reason.append("持仓已达上限")
                    
                print(f"⏹️ 不满足智能买入条件: {', '.join(reason)}")
            
            print("🎉 智能交易决策完成!")
            
            # 🆕 交易后立即保存持仓数据
            self.persistence_manager.save_position_details()
            
            # 交易后账户详情
            print("📊 交易后状态:")
            self.account_detail_manager.print_immediate_funds_snapshot()
            
        except Exception as e:
            print(f"❌ 智能交易执行失败: {e}")
            self.error_handler.handle_error('trading_error', e, "智能交易周期执行")
            self.state_manager.save_complete_state(self)
            self.persistence_manager.save_position_details()
    
    def handle_market_state_transition(self, old_state, new_state):
        """处理市场状态转换 - 智能持仓调整"""
        print(f"🔄 市场状态转换处理: {old_state} → {new_state}")
        
        # 获取新旧状态配置
        old_config = OptimizedLongRunConfig.POSITION_CONFIG[old_state]
        new_config = OptimizedLongRunConfig.POSITION_CONFIG[new_state]
        
        old_max_positions = old_config["max_positions"]
        new_max_positions = new_config["max_positions"]
        old_position_size = old_config["position_size"]
        new_position_size = new_config["position_size"]
        
        print(f"📊 状态转换详情:")
        print(f" 最大持仓: {old_max_positions} → {new_max_positions}")
        print(f" 仓位规模: {old_position_size:.0%} → {new_position_size:.0%}")
        
        # 情况1：从宽松状态转向严格状态（如牛市→熊市）
        if new_max_positions < old_max_positions:
            self._handle_restrictive_transition(old_state, new_state, old_max_positions, new_max_positions)
        
        # 情况2：从严格状态转向宽松状态（如熊市→牛市）
        elif new_max_positions > old_max_positions:
            self._handle_expansive_transition(old_state, new_state, new_max_positions)
        
        # 情况3：仓位规模变化
        if abs(new_position_size - old_position_size) > 0.05: # 变化超过5%
            self._adjust_position_sizes(new_position_size)
    
    def _handle_restrictive_transition(self, old_state, new_state, old_max, new_max):
        """处理限制性转换（减少持仓）"""
        current_positions = len(self.position_manager.positions)
        positions_to_sell = current_positions - new_max
        
        if positions_to_sell > 0:
            print(f"🚨 需要减少 {positions_to_sell} 个持仓以适应新市场状态")
            
            # 选择要卖出的持仓（自动保护未达到时间的）
            positions_to_sell_list = self._select_positions_for_state_transition(positions_to_sell)
            
            # 如果可卖出的数量不足，调整目标
            if len(positions_to_sell_list) < positions_to_sell:
                actual_new_max = current_positions - len(positions_to_sell_list)
                print(f"⚠️ 持仓保护机制生效: 实际最大持仓调整为{actual_new_max} (原计划{new_max})")
                print(f" 原因: {positions_to_sell - len(positions_to_sell_list)}个持仓未达到最短持有时间")
            
            # 执行卖出 - 使用修复的卖出管理器
            sold_count = 0
            for symbol in positions_to_sell_list:
                position = self.position_manager.positions[symbol]
                current_price = self.data_collector.get_price(symbol)
                if current_price:
                    if self.sell_manager.safe_sell_position(symbol, position['quantity'], f"状态转换: {old_state}→{new_state}"):
                        sold_count += 1
            
            print(f"📊 状态转换完成: 卖出 {sold_count} 个持仓")
            
            # 重新分配资金
            self._redistribute_capital_after_sell(new_state)
    
    def _handle_expansive_transition(self, old_state, new_state, new_max):
        """处理扩张性转换（可增加持仓）"""
        current_positions = len(self.position_manager.positions)
        available_slots = new_max - current_positions
        
        if available_slots > 0:
            print(f"🎯 可以增加 {available_slots} 个持仓以适应新市场状态")
            
            # 检查可用资金
            available_funds = self.live_capital_manager.get_available_trading_capital()
            if available_funds >= self.order_manager.min_investment:
                print(f"💰 可用资金 ${available_funds:.2f}，可以考虑新增持仓")
                # 注意：实际买入会在下一个交易周期执行
    
    def _select_positions_for_state_transition(self, count):
        """为状态转换选择要卖出的持仓 - 添加持仓时间保护"""
        position_scores = []
        
        for symbol, position in self.position_manager.positions.items():
            try:
                current_price = self.data_collector.get_price(symbol)
                if not current_price:
                    continue
                    
                entry_price = position['entry_price']
                pnl_percent = (current_price - entry_price) / entry_price
                holding_hours = self.holding_manager.get_holding_hours(symbol)
                
                # 🛡️ 关键修改：检查是否达到最短持仓时间
                min_holding_hours = self.holding_manager.min_holding_hours
                if holding_hours < min_holding_hours:
                    # 未达到最短持仓时间，给予保护分数
                    protection_score = -100 # 负分数，不会被选中
                    print(f"🛡️ {symbol} 持仓保护: 仅持有{holding_hours:.1f}小时 (<{min_holding_hours}小时)")
                    position_scores.append((symbol, protection_score, pnl_percent, holding_hours))
                    continue
                
                # 正常计算卖出优先级
                score = self._calculate_transition_sell_score(
                    symbol, pnl_percent, holding_hours, position, 
                    self.current_market_state
                )
                position_scores.append((symbol, score, pnl_percent, holding_hours))
                
            except Exception as e:
                print(f"❌ 计算持仓 {symbol} 转换分数失败: {e}")
                continue
        
        # 按分数排序（分数高的优先卖出）
        position_scores.sort(key=lambda x: x[1], reverse=True)
        
        # 过滤掉被保护的持仓（负分数的）
        eligible_positions = [item for item in position_scores if item[1] >= 0]
        
        # 如果需要卖出的数量超过可卖出的数量，只卖出可卖出的
        actual_sell_count = min(count, len(eligible_positions))
        selected = [item[0] for item in eligible_positions[:actual_sell_count]]
        
        print(f"📋 状态转换卖出列表:")
        print(f" 需要卖出: {count}个")
        print(f" 可卖出: {len(eligible_positions)}个 (排除{len(position_scores)-len(eligible_positions)}个受保护的)")
        print(f" 实际卖出: {actual_sell_count}个")
        
        for symbol, score, pnl, hours in eligible_positions[:actual_sell_count]:
            status = "盈利" if pnl >= 0 else "亏损"
            print(f" 🗑️ {symbol}: 分数{score:.1f}, {status}{abs(pnl):.2%}, 持有{hours:.1f}小时")
            
        return selected
    
    def _calculate_transition_sell_score(self, symbol, pnl_percent, holding_hours, position, new_market_state):
        """计算状态转换时的卖出优先级分数"""
        score = 0
        
        # 1. 亏损持仓优先卖出（权重最高）
        if pnl_percent < 0:
            score += 100 + abs(pnl_percent) * 500 # 亏损越大优先级越高
        
        # 2. 持有时间长的优先考虑（避免卖出刚买入的）
        score += min(holding_hours / 24 * 10, 50) # 每持有1天加10分，最多50分
        
        # 3. 根据新市场状态调整
        if new_market_state == 'bear':
            # 熊市：卖出高波动性币种
            volatility = self._estimate_volatility(symbol)
            if volatility > 0.08:
                score += 30
        elif new_market_state == 'bull':
            # 牛市：保留强势币种，卖出弱势币种
            if pnl_percent < 0.05: # 盈利不足5%的视为弱势
                score += 20
        
        # 4. 恢复的持仓优先级较高
        if position.get('recovered'):
            score += 25
        
        return score
    
    def _estimate_volatility(self, symbol):
        """估算币种波动率"""
        try:
            data = self.data_collector.get_klines(symbol, '1d', 20)
            if data is not None and len(data) >= 20:
                returns = data['close'].pct_change().dropna()
                return returns.std()
        except:
            pass
        return 0.05 # 默认波动率
    
    def _redistribute_capital_after_sell(self, new_market_state):
        """卖出后重新分配资金"""
        try:
            # 更新可用资金
            self.update_available_balance()
            
            # 重新计算交易资金
            total_assets = self.calculate_total_assets()
            self.live_capital_manager.trading_capital = total_assets * self.live_capital_manager.capital_ratio
            
            print(f"💰 资金重新分配: 总资产${total_assets:.2f}, 交易资金${self.live_capital_manager.trading_capital:.2f}")
            
        except Exception as e:
            print(f"❌ 资金重新分配失败: {e}")
    
    def _adjust_position_sizes(self, new_position_size_ratio):
        """调整仓位规模（如果需要的话）"""
        print(f"📏 检测到仓位规模变化，新规模: {new_position_size_ratio:.0%}")
        # 这里可以添加逻辑来调整现有持仓的规模
        # 但通常我们只对新买入的持仓应用新的仓位规模
    
    def _execute_smart_trading_logic(self, market_state, available_funds, candidates, available_slots):
        """智能交易逻辑 - 每个仓位向下尝试多个候选币种"""
        successful_buys = 0
        max_buy_count = min(available_slots, len(candidates))
        
        print(f"🛒 计划尝试 {max_buy_count} 个仓位，可用资金 ${available_funds:.2f}")
        print(f"🔄 每个仓位向下尝试3个候选币种")
        
        # 按评分排序的候选币种列表
        sorted_candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)
        
        for slot in range(max_buy_count):
            if successful_buys >= available_slots:
                break
                
            if available_funds < self.order_manager.min_investment:
                print(f"💰 可用资金不足，停止买入")
                break
            
            print(f"\n🎯 处理第 {slot+1} 个可买仓位:")
            
            # 为这个仓位尝试最多3个候选币种
            slot_success = False
            attempts = 0
            max_attempts_per_slot = 3
            
            for attempt in range(max_attempts_per_slot):
                candidate_index = slot * max_attempts_per_slot + attempt
                
                if candidate_index >= len(sorted_candidates):
                    print(f"📭 候选币种已用尽，无法为仓位 {slot+1} 找到更多候选")
                    break
                    
                candidate = sorted_candidates[candidate_index]
                symbol = candidate['symbol']
                attempts += 1
                
                print(f" 🔄 尝试第 {attempt+1} 个候选: {symbol} (评分: {candidate['score']:.1f})")
                
                # 检查是否已持有
                if symbol in self.position_manager.positions:
                    print(f" ⏩ 已持有 {symbol}，尝试下一个候选")
                    continue
                
                # 增强验证
                is_valid, reason = self.enhanced_coin_validation(symbol)
                if not is_valid:
                    print(f" ⏩ 验证失败 {symbol}: {reason}，尝试下一个候选")
                    continue
                
                # 检查市场状态
                if not self.order_manager.check_market_status(symbol):
                    print(f" ⏩ 市场未开放 {symbol}，尝试下一个候选")
                    continue
                
                # 计算投资金额
                position_size = self.position_manager.get_position_size(market_state)
                investment = min(available_funds, position_size)
                
                if investment < self.order_manager.min_investment:
                    print(f" 💰 投资金额不足: ${investment:.2f}")
                    break
                    
                quantity = investment / candidate['current_price']
                
                # 调整数量精度 - 使用增强的精度处理
                adjusted_quantity, adjusted_investment = self.order_manager._enhanced_adjust_quantity(
                    symbol, quantity, candidate['current_price'], 'BUY'
                )
                
                if adjusted_quantity <= 0:
                    print(f" ⚠️ 调整后数量无效: {symbol}，尝试下一个候选")
                    continue
                
                # 使用事务性资金管理
                if not self.transaction_manager.reserve_funds(symbol, adjusted_investment):
                    print(f" 💰 资金预留失败: {symbol}，尝试下一个候选")
                    continue
                
                print(f" 🔄 尝试买入 {symbol}...")
                
                # 尝试买入
                buy_order = self.order_manager.create_buy_order(
                    symbol, adjusted_quantity, candidate['current_price'],
                    self.position_manager, self.live_capital_manager
                )
                
                if buy_order:
                    self.holding_manager.record_buy(symbol)
                    self.transaction_manager.commit_transaction(symbol, True)
                    successful_buys += 1
                    slot_success = True
                    
                    print(f" ✅ 成功买入: {symbol}")
                    
                    # 记录原始买入数据用于持久化
                    if symbol in self.position_manager.positions:
                        position = self.position_manager.positions[symbol]
                        position['original_entry_time'] = datetime.now()
                        position['original_entry_price'] = candidate['current_price']
                        position['cost_known'] = True # 🆕 标记为已知成本
                    
                    # 🆕 立即保存持仓数据
                    self.persistence_manager.save_position_details()
                    
                    # 买入后立即输出账户详情
                    self.account_detail_manager.print_transaction_summary(
                        "买入完成", symbol, adjusted_investment, candidate['current_price']
                    )
                    
                    # 更新可用资金
                    self.update_available_balance()
                    available_funds = self.position_manager.get_available_funds()
                    
                    # 成功买入后跳出当前仓位的尝试循环
                    break
                else:
                    self.transaction_manager.commit_transaction(symbol, False)
                    print(f" ❌ 买入失败: {symbol}")
                    # 继续尝试下一个候选币种
            
            if slot_success:
                print(f" 🎉 仓位 {slot+1} 买入成功")
            else:
                print(f" ⏩ 仓位 {slot+1} 的所有候选币种都失败，跳过这个仓位")
        
        print(f"\n🎉 智能买入完成: 成功 {successful_buys} 个仓位，尝试了 {available_slots} 个可用仓位")
        
        # 🆕 所有买入完成后再次保存持仓数据
        self.persistence_manager.save_position_details()
        
        # 所有买入完成后打印报告
        if self.account_detail_manager.should_print_detailed_report():
            self.account_detail_manager.print_detailed_account_report("智能买入周期完成")
        else:
            self.account_detail_manager.print_quick_status_update()
    
    def print_account_report(self, total_assets, trading_capital):
        """打印账户报告"""
        self.calculate_total_assets()
        
        print(f" {'='*50}")
        print(f"📊 账户报告 - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"💰 总资产: ${total_assets:.2f}")
        print(f"💵 交易资金: ${trading_capital:.2f} (50%)")
        print(f"💸 可用资金: ${self.position_manager.available_balance:.2f}")
        print(f"📦 程序管理持仓: {self.position_manager.get_program_managed_positions_count()}个")
        print(f"🌐 市场状态: {self.current_market_state.upper()}")
        print(f"{'='*50}")
    
    def run_real_time_monitoring(self, verbose=True):
        """运行实时监控 - 使用修复的风险管理器"""
        try:
            if verbose:
                print("=== 实时监控 ===")
            
            # 检查是否需要输出账户详情
            if self.account_detail_manager.should_print_quick_update():
                self.account_detail_manager.print_quick_status_update()
            
            # 检查全仓风险
            portfolio_risk = self.portfolio_risk_controller.check_portfolio_risk()
            if portfolio_risk['action'] != 'NORMAL':
                print(f"🚨 全仓风险控制: {portfolio_risk['reason']}")
            
            # 监控持仓风险 - 使用修复的风险管理器
            self.monitor_positions_risk_fixed(verbose=verbose)
            
            if verbose:
                print("✅ 实时监控完成")
                
        except Exception as e:
            if verbose:
                print(f"❌ 实时监控失败: {e}")
    
    def monitor_positions_risk_fixed(self, verbose=True):
        """修复的持仓风险监控 - 使用FixedRiskManager"""
        if verbose:
            print(" === 修复的持仓风险监控 ===")
        
        for symbol, position in list(self.position_manager.positions.items()):
            try:
                current_price = self.data_collector.get_price(symbol)
                if current_price is None or current_price <= 0:
                    continue
                    
                entry_price = position['entry_price']
                current_pnl = (current_price - entry_price) / entry_price
                holding_hours = self.holding_manager.get_holding_hours(symbol)
                
                if verbose:
                    print(f"📊 {symbol} 状态:")
                    print(f" 成本: ${entry_price:.6f}, 当前: ${current_price:.6f}")
                    print(f" 盈亏: {current_pnl:+.2%}")
                    print(f" 持有: {holding_hours:.1f}小时")
                
                # 使用修复的风险检查
                risk_action = self.risk_manager.check_enhanced_risk(
                    symbol, position, self.current_market_state, 
                    holding_hours, current_price
                )
                
                if risk_action and risk_action['action'] != 'HOLD':
                    # 🆕 使用统一的卖出方法
                    symbol = risk_action['symbol']
                    quantity = risk_action.get('quantity')
    
                    # 如果没有提供数量，使用持仓的全部数量
                    if quantity is None:
                        if symbol in self.position_manager.positions:
                            quantity = self.position_manager.positions[symbol]['quantity']
                        else:
                            print(f"❌ 无法找到持仓 {symbol}，无法卖出")
                            continue
    
                    success = self.safe_sell_and_check(
                        symbol, 
                        quantity, 
                        f"{risk_action['action']}: {risk_action['reason']}"
                    )
    
                    if success:
                        if verbose:
                            action_type = risk_action['action']
                            print(f"✅ 执行{action_type}: {symbol} - {risk_action['reason']}")
                    else:
                        if verbose:
                            print(f"❌ 风险控制执行失败: {symbol}")
            except Exception as e:
                if verbose:
                    print(f"❌ 持仓 {symbol} 风险检查失败: {e}")
                continue
    
    def _execute_fixed_risk_action(self, risk_action):
        """执行修复的风险操作 - 使用统一的卖出方法"""
        try:
            symbol = risk_action['symbol']
            quantity = risk_action['quantity']
            reason = risk_action['reason']
        
            # 🆕 使用统一的卖出方法
            if risk_action['action'] in ['STOP_LOSS', 'TAKE_PROFIT', 'TRAILING_STOP']:
                return self.safe_sell_and_check(symbol, quantity, f"{risk_action['action']}: {reason}")
        
            return False
        
        except Exception as e:
            print(f"❌ 执行风险操作异常 {symbol}: {e}")
            return False
    def execute_smart_recovery(self):
        """执行智能恢复"""
        print("🧠 执行智能恢复程序...")
        
        try:
            # 1. 备份当前状态
            self.state_manager.save_complete_state(self)
            self.persistence_manager.save_position_details()
            
            # 2. 执行智能孤儿持仓恢复（包含退市币种屏蔽）
            recovery_success, recovered_positions = self.disaster_recovery.orphan_recovery.recover_orphan_positions()
            
            # 3. 重新计算资金
            self.live_capital_manager._update_live_balance_correct()
            
            # 4. 强制执行持仓限制（只针对程序管理持仓）
            market_state = self.current_market_state
            self.position_manager.enforce_smart_position_limits(market_state, self)
            
            # 5. 重新初始化资产统计
            self.asset_stats_manager.initialize_statistics()
            
            print("🎉 智能恢复完成!")
            print(f"📊 恢复结果:")
            print(f" 成功恢复持仓: {len(recovered_positions)}个")
            print(f" 程序管理持仓: {self.position_manager.get_program_managed_positions_count()}个")
            print(f" 总资产: ${self.live_capital_manager.total_assets:.2f}")
            
        except Exception as e:
            print(f"❌ 智能恢复失败: {e}")

    def manual_asset_recovery(self, specific_assets=None):
        """手动资产恢复接口"""
        print("🔧 执行手动资产恢复...")
        
        if specific_assets:
            print(f"🔄 恢复指定资产: {specific_assets}")
            # 这里可以添加针对特定资产的恢复逻辑
        else:
            # 执行完整的智能灾难恢复
            return self.disaster_recovery.execute_disaster_recovery()
        
        return True

# ========== 主程序入口 ==========
def main():
    """主程序入口"""
    print("🚀 启动实盘就绪版交易机器人 v8.1 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化...")
    
    try:
        bot = FullyOptimizedTradingBot()
        bot.run_continuous_trading()
        
    except KeyboardInterrupt:
        print("\n🛑 用户中断程序")
        if 'bot' in locals():
            bot.state_manager.save_complete_state(bot)
            bot.persistence_manager.save_position_details()
    except Exception as e:
        print(f"❌ 程序异常: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


C:\Users\ThinkPad\.conda\envs\crypto_quant\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


🚀 启动实盘就绪版交易机器人 v8.1 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化...
🛡️ 初始化实盘就绪版交易机器人 v8.1 + 重启修复 + 卖出修复 + 币种列表修复 + 稳定币过滤 + 增强精度处理 + 完整问题修复 + 200币种市场广度 + 增强数据持久化...
🎯 请选择交易模式:
1. 测试网模式 (Testnet)
2. 实盘模式 (Live Trading)


请输入选择 (1 或 2):  2


🚀 已选择实盘模式 - 请注意风险!
🔄 正在连接交易所...
✅ 实盘客户端创建成功
✅ 交易所连接测试成功
✅ 交易所连接成功
🔄 获取实盘资金信息...
📋 账户所有资产:
💰 稳定币 USDT: 182.361500
💰 NEBL: 0.081500 ≈ $0.03 (USDT)
💰 AUTO: 0.000243 ≈ $0.02 (USDT)
🚀 资金统计完成:
 总资产: $182.41
 USDT余额: $182.36
 交易资金: $91.21
 已分配资金: $0.05
 识别资产: 5个
💰 资金初始化: 总资产$182.41, USDT余额$182.36, 交易资金$91.21 (50%)
🔄 初始化动态币种列表 - 从币安获取USDT交易对，正确过滤稳定币...
📊 从币安获取交易量排名前200的USDT交易对...
📈 获取到USDT交易对前200的币种 (已过滤稳定币)
✅ 成功从币安获取 200 个USDT交易对 (已过滤稳定币)
🔍 执行严格币种验证...
🔄 第 1 轮严格验证...
❌ PEOPLEUSDT: 24小时成交量不足 - $2,401,835 < $4,000,000
✅ HBARUSDT: 严格验证通过 - 价格$0.2027, 24小时成交量$168,077,480
❌ JASMYUSDT: 24小时成交量不足 - $2,320,460 < $4,000,000
✅ SOLUSDT: 严格验证通过 - 价格$198.3100, 24小时成交量$687,002,739
✅ ETHFIUSDT: 严格验证通过 - 价格$0.9870, 24小时成交量$12,685,939
❌ ACTUSDT: 24小时成交量不足 - $2,287,860 < $4,000,000
✅ GALAUSDT: 严格验证通过 - 价格$0.0111, 24小时成交量$6,989,895
❌ BANANAUSDT: 24小时成交量不足 - $3,302,510 < $4,000,000
✅ OPUSDT: 严格验证通过 - 价格$0.4525, 24小时成交量$7,583,149
❌ TWTUSDT: 24小时成交量不足 - $3,800,077 < $4,000,000
❌ JTOUSDT: 24小时成交量不足 - $2,276,167 < $